<a href="https://colab.research.google.com/github/dcownden/PerennialProblemsOfLifeWithABrain/blob/P1C3_S1_Pop_Inherit/sequences/P1C3_RealEvolution/P1C3_Sequence1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following is part of a test for an upcoming text book on computational neuroscience from an optimization and learning perspective. The book will start with evolution because ultimately, all aspects of the brain are shaped by evolution and, as we will see, evolution can also be seen as an optimization algorithm. We are sharing it now to get feedback on what works and what does not and the developments we should do.

___
# **1.3.1: Populations and Inheritence**
### Objective: We concluded the last chapter by discussing how neuroscience has traditionally focused on neural mechanisms of behaviour. While neural ontogeny has also been a focus of neuroscience we argued that even greater focus on ontogeny could benefit both brain and behavioural sciences. Evolutionary thinking underpins this argument, and frames the brain as an evolved tool for rapid adaption. To fully grasp this perspective, we first need to undertand the core principles of evolutionary theory. Our aim here is to set out and explore these core principles of evolution through simple examples.

In this sequence we will:

* Outline the fundamental concepts of evolution and natural selection:
  1. Identify the Key Players in Evolution:
    * Populations and Individuals: The collective and singular units of evolution. Individuals are selected to reproduce, but it is the allele (a variant form of a gene at a specific gene locus) and trait frequencies within the population that evolve over time and generations.
    * Genotype and Phenotype: Genetic makeup of an individual and its observable traits.
    * Genotype-Phenotype Map (Ontogeny): How genotypes translate to phenotypes over an individual's lifespan.
    * Genes and Alleles: Genes are the basic units of heredity that dictate traits through coded instructions. Alleles are the different variants of a gene, located at specific positions (loci) on a chromosome.
  2. Unpack the Dynamics of Evolution:
    * Reproduction and Fitness: Who gets to pass on how many of their alleles, how many copies they pass on, to whom, and why. This process is not random but is guided by environmental conditions that act as a filter on the traits of individuals.
    * Selection: The natural process that filters individuals based on their fitness, essentially acting as an environmental sieve that determines which traits are favorable for reproduction.
    * Heritable Variation: Genetic differences that can be passed down to the next generation.
  3. Where Variation Comes from:  
    * Mutation: Random changes in genetic material, that can produce wholly new genotypes.
    * Recombination: The reshuffling of existing genes to create new genetic combinations.
* Use a simple example throughout to show how these elements fit together in an evolutionary process.
* Show how this evolutionary process is fundamentally a form of optimization, tuning a population for better survival and reproduction over time.


# Setup

Run the following cell to setup and install the various dependencies and helper functions for this .

In [1]:
# @title Dependencies, Imports and Setup
# @markdown You don't need to worry about how this code works – but you do need to **run the cell**

!pip install ipympl vibecheck datatops jupyterquiz > /dev/null 2> /dev/null #google.colab

import requests
from requests.exceptions import RequestException
import numpy as np
import itertools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.animation import FuncAnimation
import seaborn as sns
import pandas as pd
import ipywidgets as widgets
import time
import logging
import random
import os
import collections
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from io import BytesIO
from enum import Enum
from copy import copy
from scipy.spatial.distance import cdist
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from tabulate import tabulate
from IPython.display import display, clear_output, Markdown, HTML
from jupyterquiz import display_quiz
from vibecheck import DatatopsContentReviewContainer
from pathlib import Path
from typing import List, Dict
from tqdm.notebook import tqdm


# random seed settings and
# getting torch to use gpu if it's there


def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness. NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')


def seed_worker(worker_id):
  """
  DataLoader will reseed workers following randomness in
  multi-process data loading algorithm.

  Args:
    worker_id: integer
      ID of subprocess to seed. 0 means that
      the data will be loaded in the main process
      Refer: https://pytorch.org/docs/stable/data.html#data-loading-randomness for more details

  Returns:
    Nothing
  """
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)


def set_device():
  """
  Set the device. CUDA if available, CPU otherwise

  Args:
    None

  Returns:
    Nothing
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("This notebook isn't using and doesn't need a GPU. Good.")
  else:
    print("GPU is enabled in this notebook but not needed.")
    print("If possible, in the menu under `Runtime` -> ")
    print("`Change runtime type.`  select `CPU`")

  return device


SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()


def printmd(string):
  display(Markdown(string))


# the different utility .py files used in this notebook
filenames = ['gw_plotting.py', 'gw_board.py', 'gw_game.py',
             'gw_widgets.py', 'gw_NN_RL.py']
#filenames = []
# just run the code straight out of the response, no local copies needed!
for filename in filenames:
  url = f'https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/utils/{filename}'
  response = requests.get(url)
  # Check that we got a valid response
  if response.status_code == 200:
    code = response.content.decode()
    exec(code)
  else:
    print(f'Failed to download {url}')

# environment contingent imports
try:
  print('Running in colab')
  from google.colab import output
  output.enable_custom_widget_manager()
  from google.colab import data_table
  data_table.disable_dataframe_formatter()
  #from google.colab import output as colab_output
  #colab_output.enable_custom_widget_manager()
  IN_COLAB = True
except:
  IN_COLAB = False
  print('Not running in colab')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib widget
plt.style.use("https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/pplb.mplstyle")
plt.ioff() #need to use plt.show() or display explicitly
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)


def content_review(notebook_section: str):
  return DatatopsContentReviewContainer(
    "",  # No text prompt
    notebook_section,
    {
      "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
      "name": "neuro_book",
      "user_key": "xuk960xj",
    },
  ).render()
feedback_prefix = "P1C3_S3"




################################################################
# refactor Monte Carlo for boards that support multiple critters
################################################################




class MonteCarlo():
  """
  Implementation of Monte Carlo Algorithm
  """


  def __init__(self, game, nnet, default_depth=5, random_seed=None):
    """
    Initialize Monte Carlo Parameters

    Args:
      game: Gridworld Game instance
        Instance of the gridworldGame class above;
      nnet: gridworldNet instance
        Instance of the gridworldNNet class above;
      args: dictionary
        Instantiates number of iterations and episodes, controls temperature threshold, queue length,
        arena, checkpointing, and neural network parameters:
        learning-rate: 0.001, dropout: 0.3, epochs: 10, batch_size: 64,
        num_channels: 512

    Returns:
      Nothing
    """
    self.game = game
    self.nnet = nnet
    self.default_depth = default_depth
    self.rng = np.random.default_rng(seed=random_seed)


  def pis_vs_from_board(self, board, critter):
    #helper function, to put board in canonical form that nn was trained on
    batch_size, n_rows, n_cols = board['pieces'].shape
    co_pieces = board['pieces'].copy()
    this_critter_locs = np.where(co_pieces == critter)
    all_critter_locs = np.where(co_pieces >= 1)
    # other critters are invisible to this player
    co_pieces[all_critter_locs] = 0
    # nnet trained to see self as 1
    co_pieces[this_critter_locs] = 1
    scalar_rounds_left = board['rounds_left'][0]
    co_rounds_left = scalar_rounds_left // self.game.num_critters
    if critter-1 < scalar_rounds_left % self.game.num_critters:
       # add an extra if we haven't had this players turn yet in the round cycle
       co_rounds_left = co_rounds_left + 1
    co_rounds_left = np.array([co_rounds_left]*batch_size)
    pis, vs = self.nnet.predict(co_pieces,
                                board['scores'][:,critter-1],
                                co_rounds_left)
    return pis, vs


  def simulate(self, board, actions, action_indexes, critter=1, depth=None):
    """
    Helper function to simulate one Monte Carlo rollout

    Args:
      board: triple (batch_size x x_size x y_size np.array of board position,
                     scalar of current score,
                     scalar of rounds left
      actions: batch size list/array of integer indexes for moves on each board
      these are assumed to be legal, no check for validity of moves
    Returns:
      temp_v:
        Terminal State
    """
    batch_size, x_size, y_size = board['pieces'].shape
    next_board = self.game.get_next_state(board, critter,
                                          actions, action_indexes)
    # in this version of the mc player, the existence of other players is
    # ignored, in another version of mc other players moves might be simulated
    next_board['active_player'] = critter-1

    if depth is None:
      depth = self.default_depth
    # potentially expand the game tree here,
    # but just do straight rollouts after this
    # doesn't expand to deal with all random food generation possibilities
    # just expands based on the actions given
    expand_bs, _, _ = next_board['pieces'].shape

    for i in range(depth):  # maxDepth
      if next_board['rounds_left'][0] <= 0:
        # check that game isn't over
        # assumes all boards have the same rounds left
        # no rounds left return scores as true values
        terminal_vs = next_board['scores'][:,critter-1].copy()
        return terminal_vs
      else:
        #pis, vs = self.nnet.predict(next_board['pieces'], next_board['scores'], next_board['rounds_left'])
        pis, vs = self.pis_vs_from_board(next_board, critter)
        valids = self.game.get_valid_actions(next_board, critter)
        masked_pis = pis * valids
        sum_pis = np.sum(masked_pis, axis=1)
        probs = np.array(
            [masked_pi / masked_pi.sum() if masked_pi.sum() > 0
             else valid / valid.sum()
             for valid, masked_pi in zip(valids, masked_pis)])
        samp = self.rng.uniform(size = expand_bs).reshape((expand_bs,1))
        sampled_actions = np.argmax(probs.cumsum(axis=1) > samp, axis=1)
      next_board = self.game.get_next_state(next_board, critter,
                                            sampled_actions)
      # in this version of the mc player, existence of other players is ignored
      # in another better version other players moves might be simulated, either
      # as copies of self, or as distinct environmental dynamics
      next_board['active_player'] = critter-1


    pis, vs = self.pis_vs_from_board(next_board, critter)
    #pis, vs = self.nnet.predict(next_board['pieces'], next_board['scores'],
    #                            next_board['rounds_left'])
    #print(vs.shape)
    return vs




# @title plotting functions
#################################################
# More plotting functions
#################################################


def plot_directions(fig, ax, loc_prob_dict, critter, deterministic=False,
                    name=None):
  """
  Plot vector field indicating critter direction probabilities.

  Args:
    fig, ax (matplotlib objects): Figure and axes objects for plotting.
    loc_prob_dict (dict): Dictionary with keys as (row, col) location tuples
      and values as lists of direction probabilities corresponding to the
      directions ['right', 'down', 'left', 'up'].
    critter (int): Identifier for which critter directions are associated with.
    deterministic (bool, optional): If True, the probabilities array is
      converted to 1-hot, and the arrows are plotted at the center of the cell
      and are larger. Defaults to False.
  """

  #looks like direction ignores inverted axis
  direction_vectors = {'right': (1, 0), 'down': (0, -1),
                       'left': (-1, 0), 'up': (0, 1)}
  # but offsets need to be aware of inverted
  direction_offsets = {'right': (0.1, 0), 'down': (0, 0.1),
                       'left': (-0.1, 0), 'up': (0, -0.1)}
  # Offsets for each critter type 1 and 2 to be used together, 0 by itself
  critter_offsets = {0: (0, 0), 1: (-0.05, -0.05), 2: (0.05, 0.05)}
  # same logic for colors
  critter_colors = {0: 'black', 1: 'red', 2: 'blue'}
  # Get the offset and color for this critter
  critter_offset = critter_offsets[critter]
  critter_color = critter_colors[critter]

  # Add legend only if critter is not 0
  custom_leg_handles = []
  if critter != 0:
    if name is None:
      name = f'Critter {critter}'
    legend_patch = mpatches.Patch(color=critter_color, label=name)
    # Add the legend for this critter
    custom_leg_handles.append(legend_patch)

  C, R, U, V, A = [], [], [], [], []

  for loc in loc_prob_dict.keys():
    row, col = loc
    probs = loc_prob_dict[loc]
    for dir_key, prob in probs.items():
      C.append(col + critter_offset[0] + direction_offsets[dir_key][0])
      R.append(row + critter_offset[1] + direction_offsets[dir_key][1])
      U.append(direction_vectors[dir_key][0])
      V.append(direction_vectors[dir_key][1])

      if deterministic:
        A.append(1 if prob == max(probs.values()) else 0)
      else:
        A.append(prob)

  linewidth = 1.5 if deterministic else 0.5
  scale = 15 if deterministic else 30

  ax.quiver(C, R, U, V, alpha=A, color=critter_color,
            scale=scale, linewidth=linewidth)
  return fig, ax, custom_leg_handles


def make_grid(num_rows, num_cols, figsize=(7,6), title=None):
  """Plots an n_rows by n_cols grid with cells centered on integer indices and
  returns fig and ax handles for further use
  Args:
    num_rows (int): number of rows in the grid (vertical dimension)
    num_cols (int): number of cols in the grid (horizontal dimension)

  Returns:
    fig (matplotlib.figure.Figure): figure handle for the grid
    ax: (matplotlib.axes._axes.Axes): axes handle for the grid
  """
  # Create a new figure and axes with given figsize
  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  # Set width and height padding, remove horizontal and vertical spacing
  fig.get_layout_engine().set(w_pad=4 / 72, h_pad=4 / 72, hspace=0, wspace=0)
  # Show right and top borders (spines) of the plot
  ax.spines[['right', 'top']].set_visible(True)
  # Set major ticks (where grid lines will be) on x and y axes
  ax.set_xticks(np.arange(0, num_cols, 1))
  ax.set_yticks(np.arange(0, num_rows, 1))
  # Set labels for major ticks with font size of 8
  ax.set_xticklabels(np.arange(0, num_cols, 1),fontsize=8)
  ax.set_yticklabels(np.arange(0, num_rows, 1),fontsize=8)
  # Set minor ticks (no grid lines here) to be between major ticks
  ax.set_xticks(np.arange(0.5, num_cols-0.5, 1), minor=True)
  ax.set_yticks(np.arange(0.5, num_rows-0.5, 1), minor=True)
  # Move x-axis ticks to the top of the plot
  ax.xaxis.tick_top()
  # Set grid lines based on minor ticks, make them grey, dashed, and half transparent
  ax.grid(which='minor', color='grey', linestyle='-', linewidth=2, alpha=0.5)
  # Remove minor ticks (not the grid lines)
  ax.tick_params(which='minor', bottom=False, left=False)
  # Set limits of x and y axes
  ax.set_xlim(( -0.5, num_cols-0.5))
  ax.set_ylim(( -0.5, num_rows-0.5))
  # Invert y axis direction
  ax.invert_yaxis()
  # If title is provided, set it as the figure title
  if title is not None:
    fig.suptitle(title)
  # Hide header and footer, disable toolbar and resizing of the figure
  fig.canvas.header_visible = False
  fig.canvas.toolbar_visible = False
  fig.canvas.resizable = False
  fig.canvas.footer_visible = False
  # Redraw the figure with these settings
  fig.canvas.draw()
  # Return figure and axes handles for further customization
  return fig, ax


def plot_food(fig, ax, rc_food_loc, food=None):
  """
  Plots "food" on a grid implied by the given fig, ax arguments

  Args:
    fig, ax: matplotlib figure and axes objects
    rc_food_loc: ndarry(int) of shape (N:num_food x 2:row,col)
    food: a handle for the existing food matplotlib PatchCollection object
    if one exists
  Returns:
    a handle for matplotlib PathCollection object of food scatter plot, either
    new if no handle was passed or updated if it was
  """
  # if no PathCollection handle passed in:
  if food is None:
    food = ax.scatter([], [], s=150, marker='o', color='red', label='Food')
  rc_food_loc = np.array(rc_food_loc, dtype=int)
  #matrix indexing convention is is [row-vertical, col-horizontal]
  #plotting indexing convention is (x-horizontal,y-vertical), hence flip
  food.set_offsets(np.fliplr(rc_food_loc))
  return food


def plot_critters(fig, ax, critter_specs: List[Dict[str, object]]) -> List[Dict[str, object]]:
  """
  Plots multiple types of "critters" on a grid implied by the given
  fig, ax arguments.

  Args:
    fig, ax: matplotlib figure and axes objects.
    critter_specs: List of dictionaries with keys 'location', 'name', 'color',
    'marker', 'int_id', 'rc_critter_loc' and optionally 'handle' for each
    critter.

  Returns:
    Updated critter_specs with handles.
  """
  for spec in critter_specs:
    # Ensure required keys are present
    for key in ['marker', 'color', 'name', 'rc_loc']:
      if key not in spec:
        raise ValueError(f"Key '{key}' missing in critter spec.")
    handle_ = spec.get('handle')
    if handle_ is None:
      handle_ = ax.scatter([], [], s=250, marker=spec['marker'],
                           color=spec['color'], label=spec['name'])
    handle_.set_offsets(np.flip(spec['rc_loc']))
    spec.update({'handle': handle_})
  return critter_specs


def plot_critter(fig, ax, rc_critter_loc,
                 critter=None, critter_name='Critter'):
  """
  Plots "critter" on a grid implied by the given fig, ax arguments

  Args:
    fig, ax: matplotlib figure and axes objects
    rc_critter_loc: ndarry(int) of shape (N:num_critters x 2:row,col)
    critter: a handle for the existing food matplotlib PatchCollection object
    if one exists
  Returns:
    a handle for matplotlib PathCollection object of critter scatter plot,
    either new if no handle was passed in or updated if it was.
  """
  if critter is None:
    critter = ax.scatter([], [], s=250, marker='h',
                         color='blue', label=critter_name)
  # matrix indexing convention is is [row-vertical, col-horizontal]
  # plotting indexing convention is (x-horizontal,y-vertical), hence flip
  critter.set_offsets(np.flip(rc_critter_loc))
  return critter


def plot_fov(fig, ax, rc_critter, n_rows, n_cols, radius, has_fov,
             opaque=False, fov=None):
  """
  Plots a mask on a grid implied by the given fig, ax arguments

  Args:
    fig, ax: matplotlib figure and axes objects
    rc_critter: ndarry(int) (row,col) of the critter
    mask: a handle for the existing mask matplotlib Image object if one exists
  Returns:
    a handle for matplotlib Image object of mask, either new if no handle
    was passed in or updated if it was.
  """

  # Initialize mask as a semi-transparent overlay for the entire grid
  mask_array = np.ones((n_rows, n_cols, 4))
  mask_array[:, :, :3] = 0.5  # light grey color
  if has_fov == True:
    if opaque:
      mask_array[:, :, 3] = 1.0  # 50% opacity
    else:
      mask_array[:, :, 3] = 0.5  # 50% opacity
    # Create arrays representing the row and column indices
    rows = np.arange(n_rows)[:, np.newaxis]
    cols = np.arange(n_cols)[np.newaxis, :]
    # Iterate over each critter location
    dist = np.abs(rows - rc_critter[0]) + np.abs(cols - rc_critter[1])
    # Set the region within the specified radius around the critter to transparent
    mask_array[dist <= radius, 3] = 0
  else:
    mask_array[:, :, 3] = 0

  if fov is None:
    fov = ax.imshow(mask_array, origin='lower', zorder=2)
  else:
    fov.set_data(mask_array)

  return fov


def remove_ip_clutter(fig):
  fig.canvas.header_visible = False
  fig.canvas.toolbar_visible = False
  fig.canvas.resizable = False
  fig.canvas.footer_visible = False
  fig.canvas.draw()




# @title GridworldBoard class
#######################################################################
# extend GridworldGame class locally before integrating in shared utils
#######################################################################




class GridworldBoard():
  """
  A collection methods and parameters of a gridworld game board that
  define the logic of the game, and allows for multiple critters on the same
  board

  board state is represented by primarily by pieces, score, and rounds left
  pieces is a batch x n_rows x n_cols numpy array positive integers are critter
  locations 0's are empty space and -1's are food.

  For pieces first dim is batch, second dim row , third is col,
  so pieces[0][1][7] is the square in row 2, in column 8 of the first board in
  the batch of boards.

  scores is a batchsize x num_critters numpy array giving the scores for each
  critter on each board in the batch (note off by one indexing)

  rounds_left is how many rounds are left in the game.

  active_player keeps track of which players turn it is

  Note:
    In 2d np.array first dim is row (vertical), second dim is col (horizontal),
    i.e. top left corner is (0,0), so take care when visualizing/plotting
    as np.array visualization inline with typical tensor notation but at odds
    with conventional plotting where (0,0) is bottom left, first dim, x, is
    horizontal, second dim, y, is vertical
  """


  class CritterFoodType(Enum):
    FOOD = "food"
    PREY = "prey"
    PREDATOR = "predator"

  ARRAY_PAD_VALUE = -200

  def __init__(self, batch_size=1,
               n_rows=7, n_cols=7,
               num_food=10, num_prey=1, num_pred=1,
               lifetime=30, rng=None):
    """Set the parameters of the game."""
    self.n_rows = n_rows
    self.n_cols = n_cols
    self.batch_size = batch_size
    self.num_food = num_food
    self.num_prey = num_prey
    self.num_pred = num_pred
    self.num_critters = num_pred + num_prey
    self.lifetime = lifetime
    self.pred_prey_threshold = self.num_prey
    if rng is None:
      self.rng = np.random.default_rng(seed=SEED)
    else:
      self.rng = rng
    self.check_sum = np.sum(np.arange(start=-self.num_food,
                                      stop=self.num_critters+1))


  def init_loc(self, n_rows, n_cols, num, rng=None):
    """
    Samples random 2d grid locations without replacement

    Args:
      n_rows: int, number of rows in the grid
      n_cols: int, number of columns in the grid
      num:    int, number of samples to generate. Should throw an error if num > n_rows x n_cols
      rng:    instance of numpy.random's default rng. Used for reproducibility.

    Returns:
      int_loc: ndarray(int) of shape (num,), flat indices for a 2D grid flattened into 1D
      rc_index: tuple(ndarray(int), ndarray(int)), a pair of arrays with the first giving
        the row indices and the second giving the col indices. Useful for indexing into
        an n_rows by n_cols numpy array.
      rc_plotting: ndarray(int) of shape (num, 2), 2D coordinates suitable for matplotlib plotting
    """

    # Set up default random generator, use the boards default if none explicitly given
    if rng is None:
      rng = self.rng
    # Choose 'num' unique random indices from a flat 1D array of size n_rows*n_cols
    int_loc = rng.choice(n_rows * n_cols, num, replace=False)
    # Convert the flat indices to 2D indices based on the original shape (n_rows, n_cols)
    rc_index = np.unravel_index(int_loc, (n_rows, n_cols))
    # Transpose indices to get num x 2 array for easy plotting with matplotlib
    rc_plotting = np.array(rc_index).T
    # Return 1D flat indices, 2D indices for numpy array indexing and 2D indices for plotting
    return int_loc, rc_index, rc_plotting


  def get_init_board_state(self):
    """Set up starting board using game parameters"""
    #set rounds_left and score
    self.rounds_left = (np.ones(self.batch_size) *
                        self.lifetime)
    self.is_over = np.zeros(self.batch_size, dtype=bool)
    self.scores = np.zeros((self.batch_size, self.num_critters))
    # create an empty board array.
    self.pieces = np.zeros((self.batch_size, self.n_rows, self.n_cols),
                           dtype=int)
    # Place critter and initial food items on the board randomly
    for ii in np.arange(self.batch_size):
      # num_food+num_critter because we want critter and food locations
      int_loc, rc_idx, rc_plot = self.init_loc(
        self.n_rows, self.n_cols, self.num_food+self.num_critters)
      # critter random start locations
      for c_ in np.arange(self.num_critters):
        self.pieces[(ii, rc_idx[0][c_], rc_idx[1][c_])] = c_ + 1
      # food random start locations
      for f_ in np.arange(self.num_food):
        self.pieces[(ii, rc_idx[0][self.num_critters + f_],
                         rc_idx[1][self.num_critters + f_])] = -f_ - 1
    state = {'pieces': self.pieces.copy(),
             'scores': self.scores.copy(),
             'rounds_left': self.rounds_left.copy(),
             'is_over': self.is_over.copy()}
    return state


  def set_state(self, board):
    """ board is dictionary giving game state a triple of np arrays
      pieces:        numpy array (batch_size x n_rows x n_cols),
      scores:        numpy array (batch_size x num_critters)
      rounds_left:   numpy array (batch_size)
    """
    self.pieces = board['pieces'].copy()
    self.scores = board['scores'].copy()
    self.rounds_left = board['rounds_left'].copy()
    self.is_over = board['is_over'].copy()


  def get_state(self):
    """ returns a board state, which is a triple of np arrays
    pieces,       - batch_size x n_rows x n_cols
    scores,       - batch_size
    rounds_left   - batch_size
    """
    state = {'pieces': self.pieces.copy(),
             'scores': self.scores.copy(),
             'rounds_left': self.rounds_left.copy(),
             'is_over': self.is_over.copy()}
    return state


  def __getitem__(self, index):
    return self.pieces[index]


  ################# CORE GAME STATE UPDATE LOGIC ##############################
  ################# execute_moves is main, uses these helper functions ########
  def get_critter_food_type(self, critter_food):
    if critter_food <= -1:
        critter_food_type = self.CritterFoodType.FOOD
    elif critter_food > self.pred_prey_threshold:
        critter_food_type = self.CritterFoodType.PREDATOR
    else:
        critter_food_type = self.CritterFoodType.PREY
    return critter_food_type


  def get_type_masks(self):
    """
    Returns masks indicating the position types on the board.
    Returns:
        tuple: Tuple containing masks for empty spaces, food, prey, and predator.
    """
    empt_mask = self.pieces == 0
    food_mask = self.pieces <= -1
    prey_mask = (1 <= self.pieces) & (self.pieces <= self.pred_prey_threshold)
    pred_mask = self.pred_prey_threshold < self.pieces
    return empt_mask, food_mask, prey_mask, pred_mask


  def get_collisions(self, moves, critter_food, critter_food_type):
    """
    Determine the collision results and update scores accordingly.
    Args:
        moves (tuple): Tuple of arrays indicating the moves.
        critter_food (int): Index to identify the critter or food.
        critter_food_type (enum): Type of the critter or food
    Returns:
        tuple: Tuple containing move collision messages and separates out the
        moves by where they land i.e., empty spaces, food, prey, and predator.
    """
    batch_size, n_rows, n_cols = self.pieces.shape
    move_mask = np.zeros(self.pieces.shape, dtype=bool)
    move_mask[moves] = True
    (empt_mask, food_mask,
     prey_mask, pred_mask) = self.get_type_masks()

    move_coll_msg = np.zeros(batch_size)
    empt_moves = np.where(empt_mask & move_mask)
    food_moves = np.where(food_mask & move_mask)
    prey_moves = np.where(prey_mask & move_mask)
    pred_moves = np.where(pred_mask & move_mask)
    move_coll_msg[empt_moves[0]] = 1

    if critter_food_type == self.CritterFoodType.PREY:
      move_coll_msg[food_moves[0]] = 2
    elif critter_food_type == self.CritterFoodType.PREDATOR:
      move_coll_msg[food_moves[0]] = 3
      move_coll_msg[prey_moves[0]] = 4
    # all collision types are blocking for food types

    return (move_coll_msg, empt_moves, food_moves, prey_moves, pred_moves)


  def update_scores(self, move_coll_msg, critter_food,
                    critter_food_type, prey_moves):
    if critter_food_type == self.CritterFoodType.PREY:
      self.scores[:, critter_food-1] += (move_coll_msg == 2)
    elif critter_food_type == self.CritterFoodType.PREDATOR:
      # predators that eat get a point
      self.scores[:, critter_food-1] += (move_coll_msg == 4)
      # prey that are eaten lose 10 points
      who_eaten = self.pieces[prey_moves]
      self.scores[prey_moves[0], who_eaten-1] -= 10
    # food types don't get a score, it's a neuro book


  def move_pieces(self, critter_food, move_coll_msg, moves):
    """
    Move the pieces on the board based on the collision messages.

    Args:
        critter_food (int): Index to identify the critter or food.
        move_coll_msg (np.array): Array of collision messages.
        moves (tuple): Tuple of arrays indicating the moves.
    """
    old_locs = np.where(self.pieces == critter_food)
    vacated_old_locs = np.column_stack(old_locs)[np.where(move_coll_msg > 0)]
    vacated_old_locs_idx = (vacated_old_locs[:,0],
                            vacated_old_locs[:,1],
                            vacated_old_locs[:,2])
    self.pieces[vacated_old_locs_idx] = 0
    new_locs = np.column_stack(moves)[np.where(move_coll_msg > 0)]
    new_locs_idx = (new_locs[:,0], new_locs[:,1], new_locs[:,2])
    self.pieces[new_locs_idx] = critter_food


  def replace_destroyed(self, destroying_moves, old_pieces):
    """
    Replace the destroyed pieces on the board.

    Args:
        destroying_moves (tuple): Tuple of arrays indicating the moves that
        resulted in destruction.
    """
    batch_size, n_rows, n_cols = old_pieces.shape
    g_gone = np.zeros(batch_size)
    g_gone[destroying_moves[0]] = 1
    which_gone = old_pieces[destroying_moves]
    if np.sum(g_gone) > 0:
      num_empty_after = (n_rows*n_cols - self.num_food - self.num_critters + 1)
      p_new_locs = np.where(np.logical_and(
        self.pieces == 0, g_gone.reshape(batch_size, 1, 1)))
      food_sample_ = self.rng.choice(num_empty_after, size=int(np.sum(g_gone)))
      food_sample = food_sample_ + np.arange(int(np.sum(g_gone)))*num_empty_after
      new_loc_vals = self.pieces[(p_new_locs[0][food_sample],
                   p_new_locs[1][food_sample],
                   p_new_locs[2][food_sample])]
      # this requires that p_new_locs and destroying moves are both
      # lexographically sorted... but they are not always
      self.pieces[(p_new_locs[0][food_sample],
                   p_new_locs[1][food_sample],
                   p_new_locs[2][food_sample])] = which_gone


  def execute_moves(self, moves, critter_food):
    """
    Execute the moves on the board, handle collisions, update scores,
    and replace destroyed/eaten pieces.

    Args:
      moves (tuple): Tuple of arrays indicating the moves.
      critter_food (int): Index to identify the critter or food.
    """
    # what type of critter is moving
    critter_food_type = self.get_critter_food_type(critter_food)
    # what do they land on when the move
    (move_coll_msg, empt_moves, food_moves,
     prey_moves, pred_moves) = self.get_collisions(
        moves, critter_food, critter_food_type)
    # based on what they move onto increment/decrement scores
    self.update_scores(move_coll_msg, critter_food,
                       critter_food_type, prey_moves)
    # move the pieces
    old_pieces = self.pieces.copy()
    self.move_pieces(critter_food, move_coll_msg, moves)
    # eaten food and prey respawn
    if critter_food_type == self.CritterFoodType.PREY:
      self.replace_destroyed(food_moves, old_pieces)
    elif critter_food_type == self.CritterFoodType.PREDATOR:
      self.replace_destroyed(food_moves, old_pieces)
      self.replace_destroyed(prey_moves, old_pieces)
    if np.any(np.sum(self.pieces, axis=(1,2)) != self.check_sum):
      print('something went terribly wrong')
      print(old_pieces)
      print(critter_food)
      print(moves)
      print(self.pieces)


  def get_neighbor_grc_indices(self, critter_food, radius, pad=False):
    """
    Returns all grid positions within a certain cityblock distance radius from
    the place corresponding to critter_food.

    Args:
        critter_food (int): The idex of the focal critter_food.
        radius (int): The cityblock distance.
        pad (bool): whether or not to pad the array, if padded all row, col
          indexes are valid for the padded array, useful for getting percept
          if not all indexes are correct for the original array, useful for
          figuring out legal moves.

    Returns:
        an array of indices, each row is a g, r, c index for the neighborhoods
        around the critters, can use the g value to know which board you are in.
        if pad=True also returns the padded array (the indices in that case) are
        for the padded array, so won't work on self.pieces, whereas if pad is
        False the indices will be for the offsets in reference to the original
        self.pieces, but note that some of these will be invalid, and will
        need to be filtered out (as we do in get_legal)
    """
    batch_size, n_rows, n_cols = self.pieces.shape
    # Create meshgrid for offsets
    if pad is True:
      padded_arr = np.pad(self.pieces, ((0, 0), (radius, radius),
        (radius, radius)), constant_values=self.ARRAY_PAD_VALUE)
      batch, rows, cols = np.where(padded_arr == critter_food)
    else:
      batch, rows, cols = np.where(self.pieces == critter_food)
    row_offsets, col_offsets = np.meshgrid(
        np.arange(-radius, radius + 1),
        np.arange(-radius, radius + 1),
        indexing='ij')

    # Filter for valid cityblock distances
    mask = np.abs(row_offsets) + np.abs(col_offsets) <= radius
    valid_row_offsets = row_offsets[mask]
    valid_col_offsets = col_offsets[mask]
    # Extend rows and cols dimensions for broadcasting
    extended_rows = rows[:, np.newaxis]
    extended_cols = cols[:, np.newaxis]
    # Compute all neighbors for each position in the batch
    neighbors_rows = extended_rows + valid_row_offsets
    neighbors_cols = extended_cols + valid_col_offsets

    indices = np.column_stack((np.repeat(np.arange(batch_size),
                                         neighbors_rows.shape[1]),
                               neighbors_rows.ravel(),
                               neighbors_cols.ravel()))
    if pad is False:
      return indices
    elif pad is True:
      return indices, padded_arr


  def get_legal_moves(self, critter_food, radius=1):
    """
    Identifies all legal moves for the critter, taking into acount which moves
    are blocking based on type.

    Returns:
      A numpy int array of size batch x 3(g,x,y) x 4(possible moves)

    Note:
      moves[0,1,3] is the x coordinate of the move corresponding to the
      fourth offset on the first board.
      moves[1,:,1] will give the g,x,y triple corresponding to the
      move on the second board and the second offset, actions are integers
    """

    critter_locs = np.array(np.where(self.pieces == critter_food))
    # turn those row, col offsets into a set of legal offsets
    legal_offsets = self.get_neighbor_grc_indices(critter_food, radius)
    legal_offsets = {tuple(m_) for m_ in legal_offsets}

    # Apply logic of where a successful move can be made, by which
    # type of critter, be they food, prey, predator or something else
    empt_mask, food_mask, prey_mask, pred_mask = self.get_type_masks()
    critter_food_type = self.get_critter_food_type(critter_food)
    #print(critter_food_type)
    if critter_food_type == self.CritterFoodType.FOOD:
      #food only drifts into empty places
      legal_destinations = np.where(empt_mask)
    elif critter_food_type == self.CritterFoodType.PREY:
      legal_destinations = np.where(empt_mask | food_mask)
    elif critter_food_type == self.CritterFoodType.PREDATOR:
      legal_destinations = np.where(empt_mask | food_mask | prey_mask)
    else:
      raise ValueError("Unexpected value for critter_food_type.")
    legal_destinations = {tuple(coords) for coords in zip(*legal_destinations)}
    # Add the current locations of the critters to legal_destinations
    current_locations = {tuple(loc) for loc in critter_locs.T}
    legal_destinations = legal_destinations.union(current_locations)

    # legal moves are both legal offsets and legal destinations
    legal_moves = legal_offsets.intersection(legal_destinations)
    return legal_moves


  def get_legal_offsets(self, critter_food, radius):
    """
    Identifies all legal offsets for a critter or food, so filter out moves
    that are off the board, but does not filter out collisions that would be
    blocking. For a random valid player likely better to use get_legal_moves,
    but this is much quicker, because it doesn't check collision types, for
    use by RL agents in training loops

    Returns:
      A numpy int array of size batch x 3(g,x,y) x 4(possible moves)

    Note:
      moves[0,1,3] is the x coordinate of the move corresponding to the
      fourth offset on the first board.
      moves[1,:,1] will give the g,x,y triple corresponding to the
      move on the second board and the second offset, actions are integers
    """
    batch_size, n_rows, n_cols = self.pieces.shape
    batch, rows, cols = np.where(self.pieces == critter_food)
    row_offsets, col_offsets = np.meshgrid(
        np.arange(-radius, radius + 1),
        np.arange(-radius, radius + 1),
        indexing='ij')
    # Filter for valid cityblock distances
    mask = np.abs(row_offsets) + np.abs(col_offsets) <= radius
    valid_row_offsets = row_offsets[mask]
    valid_col_offsets = col_offsets[mask]
    # Extend rows and cols dimensions for broadcasting
    extended_rows = rows[:, np.newaxis]
    extended_cols = cols[:, np.newaxis]
    # Compute all neighbors for each position in the batch
    potential_moves_rows = extended_rows + valid_row_offsets
    potential_moves_cols = extended_cols + valid_col_offsets

    # Filter offsets that would take the critter outside the board
    c1 = potential_moves_rows >= 0
    c2 = potential_moves_rows <= n_rows-1
    c3 = potential_moves_cols >= 0
    c4 = potential_moves_cols <= n_cols-1
    valid_move_mask = np.logical_and.reduce([c1, c2, c3, c4])

    legal_offsets_rows = potential_moves_rows[valid_move_mask]
    legal_offsets_cols = potential_moves_cols[valid_move_mask]
    batch_indexes = np.repeat(batch, valid_row_offsets.shape[0])
    legal_offsets = np.column_stack((batch_indexes[valid_move_mask.ravel()],
                                     legal_offsets_rows.ravel(),
                                     legal_offsets_cols.ravel()))
    return legal_offsets, valid_move_mask


  def get_perceptions(self, critter_food, radius):
    idx, pad_pieces = self.get_neighbor_grc_indices(critter_food,
                                                    radius, pad=True)
    #percept_mask = np.zeros(pad_pieces.shape, dtype=bool)
    #percept_mask[idx[:,0], idx[:,1]], idx[:,2]] = True
    percept = pad_pieces[idx[:,0], idx[:,1], idx[:,2]]
    return(percept.reshape(self.batch_size, -1))




# @title GridworldGame class
#######################################################################
# extend GridworldGame class locally before integrating in shared utils
#######################################################################



class GridworldGame():
  """
  A collection methods and parameters of a gridworld game that allow
  for interaction with and display of GridwordlBoard objects.
  """


  def __init__(self, batch_size=1, n_rows=7, n_cols=7,
               num_food=10, num_prey=1, num_pred=1,
               lifetime=30, rng=None, drift_player=None):
    """
    Initializes an instance of the class with the specified parameters.
    Args:
      batch_size (int, optional): Number of instances in a batch. Default is 1.
      n_rows (int, optional): Number of rows in the grid. Default is 7.
      n_cols (int, optional): Number of columns in the grid. Default is 7.
      num_food (int, optional): Number of food items. Default is 10.
      num_prey (int, optional): Number of different agents running around
        on each board in the batch eating food. Default is 1.
      num_pred (int, optional): Number of different agents running around
        on each board in the batch eating prey. Default is 1.
      lifetime (int, optional): Time before critter's life ends, in terms of
        time steps. Default is 30.
      rng (numpy random number generator, optional): Random number generator
        for reproducibility. If None, uses default RNG with a preset seed.
      drift_player (player object, optional): a 'player' who moves the food
        pieces around (drifting) if none, skip food movement
    """

    # Check for positive integer inputs
    assert all(isinstance(i, int) and i >= 0
               for i in [batch_size, n_rows, n_cols, num_food, num_prey, num_pred,
                         lifetime]), "All inputs must be non-negative integers."
    self.batch_size = batch_size
    self.n_rows = n_rows
    self.n_cols = n_cols
    self.num_food = num_food
    self.num_prey = num_prey
    self.num_pred = num_pred
    self.num_critters = num_pred + num_prey
    self.pred_prey_threshold = self.num_prey
    # Check for num_food exceeding maximum possible value
    max_food = n_rows * n_cols - self.num_critters
    if num_food > max_food:
      print(f'num_food is too large, setting it to maximum possible value: {max_food}')
      num_food = max_food
    self.num_food = num_food
    self.lifetime = lifetime
    # Set up random number generator
    if rng is None:
      self.rng = np.random.default_rng(seed=SEED)
    else:
      self.rng = rng
    self.drift_player = drift_player


  def get_init_board(self):
    """
    Generates a starting board given the parameters of the game.
    Returns a tuple giving current state of the game
    """
    # current score, and rounds left in the episode
    b = GridworldBoard(batch_size=self.batch_size, n_rows=self.n_rows,
                       n_cols=self.n_cols, num_food=self.num_food,
                       num_prey=self.num_prey, num_pred=self.num_pred,
                       lifetime=self.lifetime, rng=self.rng)
    return b.get_init_board_state()


  def get_board_shape(self):
    """Shape of a single board, doesn't give batch size"""
    return (self.n_rows, self.n_cols)


  def get_action_size(self):
    """
    Returns the number of all possible actions, even though only  2-4 of
    these will ever be valid on a given turn.
    Actions correspond to integer indexes of board locations,
    moves to g,r,c coordinate indexes of board locations
    """
    return self.n_rows * self.n_cols


  def get_batch_size(self):
    """
    Returns the number of actions, only 0-4 of these will ever be valid.
    Actions correspond to integer indexes of board locations,
    moves to r,c indexes of board locations
    """
    return self.batch_size


  def string_rep(self, board, g=0):
    """ A bytestring representation board g's state in the batch of boards"""
    return (board['pieces'][g].tobytes() + board['scores'][g].tobytes() +
            board['rounds_left'][g].tobytes())


  def get_square_symbol(self, piece):
    """ Translate integer piece value to symbol for display"""
    if piece == -1:
      return "X"
    elif piece == 0:
      return "-"
    elif piece >= 1:
      return "0"
    else:
      return "???????????????????????????"


  def string_rep_readable(self, board, g=0):
    """ A human readable representation of g-th board's state in the batch"""
    board_s = "".join([self.get_square_symbol(square)
                        for row in board['pieces'][g]
                          for square in row])
    board_s = board_s + '_' + str(board['scores'][g])
    board_s = board_s + '_' + str(board['rounds_left'][g])
    return board_s


  def get_scores(self, board):
    return board['scores'].copy()


  def get_rounds_left(self, board):
    return board['rounds_left'].copy()


  def display(self, board, g=0):
    """Displays the g-th games in the batch of boards"""
    print("   ", end="")
    for c_ in range(self.n_cols):
      print(c_, end=" ")
    print("")
    print("-----------------------")
    for c_ in range(self.n_cols):
      print(c_, "|", end="")    # Print the row
      for r_ in range(self.n_rows):
        piece = board['pieces'][g,c_,r_]    # Get the piece to print
        #print(piece)
        print(self.get_square_symbol(piece), end=" ")
      print("|")
    print("-----------------------")
    print("Rounds Left: " + str(board['rounds_left'][g]))
    print("Score: " + str(board['scores'][g]))


  def get_critter_rc(self, board, g, critter_index):
    return np.squeeze(np.array(np.where(board['pieces'][g]==critter_index)))


  def plot_moves(self, board, player0, g=0, player1=None,
                 fig=None, ax=None, p0_name='Player 0', p1_name='Player 1',
                 figsize=(6,5), critter_name='Critter', title=None,
                 deterministic=False):
    """
    Uses plotting functions to make picture of the current board state, and what
    a critter would do at each non-food location in the current board state
    """
    def make_prob_dict(critter_locs, play):
      offset_dict = {(0, 1): 'right',
                     (0,-1): 'left',
                     ( 1, 0): 'down',
                     (-1, 0): 'up'}
      index_probs = play[2].copy()
      loc_prob_dict = {}
      # for each non food locations
      for g, loc_ in enumerate(critter_locs):
        # this is the location as an r, c tuple
        rc_tup = tuple((loc_[1], loc_[2]))
        # the relevant probabilities
        raw_probs = index_probs[g]
        probs = raw_probs[raw_probs > 0]
        indexes = np.argwhere(raw_probs > 0)
        # turn the probability indexes into r, c coords
        rows = np.floor_divide(indexes, gwg.n_cols)
        cols = np.remainder(indexes, gwg.n_cols)
        moves = np.squeeze(np.array([z for z in zip(rows, cols)]), axis=2)
        #compute the offsets and turn them to strings
        offsets = moves - loc_[1:]
        str_offsets = np.array(list(map(offset_dict.get, map(tuple, offsets))))
        # update the loc_prob_dict for plotting
        prob_dict = dict(zip(str_offsets, probs))
        loc_prob_dict.update({rc_tup: prob_dict})
      return loc_prob_dict

    batch_size, n_rows, n_cols = board['pieces'].shape
    plt.ioff()
    if fig is None and ax is None:
      fig, ax = make_grid(n_rows, n_cols, figsize=figsize, title=title)

    rc_food_index = np.array(np.where(board['pieces'][g] <= -1))
    rc_food_plotting = np.array(rc_food_index).T
    food = plot_food(fig, ax, rc_food_plotting)

    expanded_board = self.critter_everywhere_state_expansion(
      board, player0.critter_index, to_expand=g)
    critter_locs = np.argwhere(expanded_board['pieces']==player0.critter_index)
    #play the expanded state
    p0_play = player0.play(expanded_board)
    #get the prob dict
    p0_loc_prob_dict = make_prob_dict(critter_locs, p0_play)
    # same for player1 if there is one
    if player1 is not None:
      p1_play = player1.play(expanded_board)
      p1_loc_prob_dict = make_prob_dict(critter_locs, p1_play)

    existing_handels, _ = ax.get_legend_handles_labels()
    if player1 is None:
      fig, ax, leg_handles_0 = plot_directions(fig, ax, p0_loc_prob_dict,
        critter=0, deterministic=deterministic)
      leg_handles = existing_handels
    else:
      fig, ax, leg_handles_0 = plot_directions(fig, ax, p0_loc_prob_dict,
        critter=1, deterministic=deterministic, name=p0_name)
      fig, ax, leg_handles_1 = plot_directions(fig, ax, p1_loc_prob_dict,
        critter=2, deterministic=deterministic, name=p1_name)
      leg_handles = existing_handels + leg_handles_0 + leg_handles_1

    fig.legend(handles=leg_handles, loc="outside right upper")
    fig.canvas.draw()
    return fig, ax


  def plot_board(self, board, g=0,
                 fig=None, ax=None, critter_specs=None, food=None, fov=None,
                 legend_type='included',
                 has_fov=False, #fog_of_war feild_of_view
                 fov_opaque=False, #let human see trhough fog of war or not
                 radius=2, figsize=(6,5), title=None,
                 name='Critter',
                 focal_critter_index = 0):
    """Uses plotting functions to make picture of the current board state"""
    batch_size, n_rows, n_cols = board['pieces'].shape
    plt.ioff()
    if fig is None and ax is None:
      fig, ax = make_grid(n_rows, n_cols, figsize=figsize, title=title)

    # generate critter plotting specs if we don't already have them
    if critter_specs is None:
      critter_specs = []
      markers = ['h', 'd']  # hexagon and diamond
      colors = sns.color_palette("colorblind")
      for i in range(self.num_critters):
        critter_name = name if self.num_critters == 1 else f'{name} {i+1}'
        spec = {'marker': markers[i % len(markers)],
                'color': colors[i // len(markers) % len(colors)],
                'name': critter_name,
                'int_id': i+1}
        critter_specs.append(spec)
    # get critter locs and plot them
    assert len(critter_specs) == self.num_critters, "More/fewer specs than critters"
    for spec in critter_specs:
      rc_loc = np.array(np.where(board['pieces'][g] == spec['int_id'])).T
      spec.update({'rc_loc': rc_loc})
    critter_specs = plot_critters(fig, ax, critter_specs)

    # get food locs and plot them
    rc_food_index = np.array(np.where(board['pieces'][g] <= -1))
    rc_food_plotting = np.array(rc_food_index).T
    if food is None:
      food = plot_food(fig, ax, rc_food_plotting)
    else:
      food = plot_food(fig, ax, rc_food_plotting, food)

    #plot field of view if doing that
    if has_fov:
      # plot field of view around the 'active player'
      if fov is None:
        fov = plot_fov(fig, ax, critter_specs[focal_critter_index]['rc_loc'][0],
                       n_rows, n_cols, radius, has_fov, opaque=fov_opaque)
      else:
        fov = plot_fov(fig, ax, critter_specs[focal_critter_index]['rc_loc'][0],
                       n_rows, n_cols, radius, has_fov, opaque=fov_opaque, fov=fov)
    # make legend and draw and return figure
    if legend_type == 'included':
      fig.legend(loc = "outside right upper", markerscale=0.8)
      fig.canvas.draw()
      return fig, ax, critter_specs, food, fov
    elif legend_type == 'separate':
      fig_legend, ax_legend = plt.subplots(figsize=(1.5,1.5), layout='constrained')
      fig_legend.get_layout_engine().set(w_pad=0, h_pad=0, hspace=0, wspace=0)
      handles, labels = ax.get_legend_handles_labels()
      ax_legend.legend(handles, labels, loc='center', markerscale=0.8)
      ax_legend.axis('off')
      fig_legend.canvas.header_visible = False
      fig_legend.canvas.toolbar_visible = False
      fig_legend.canvas.resizable = False
      fig_legend.canvas.footer_visible = False
      fig_legend.canvas.draw()
      return fig, ax, critter_specs, food, fov, fig_legend, ax_legend
    else: #no legend
      fig.canvas.draw()
      return fig, ax, critter_specs, food, fov


  def get_legal_moves(self, board, critter=1, radius=1):
    """
    A Helper function to get the legal moves, as set of batch, row, col triples
    giving for the given board. Does return moves that are technically legal
    but that will result in a blocking move, this is good for a random valid
    player, so that the don't have a high probability of staying still if
    there are lots of blocking moves.

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      critter (int): value of critter we are getting the valid actions for
      radius (int): how far, in city block distance the critter can move

    Returns:
      moves: set or tuples (g, r, c)
    """
    batch_size, n_rows, n_cols = board['pieces'].shape
    b = GridworldBoard(batch_size=self.batch_size, n_rows=self.n_rows,
                       n_cols=self.n_cols, num_food=self.num_food,
                       num_prey=self.num_prey, num_pred=self.num_pred,
                       lifetime=self.lifetime, rng=self.rng)
    b.set_state(board)
    legal_moves =  b.get_legal_moves(critter, radius)
    return legal_moves


  def get_legal_offsets(self, board, critter=1, radius=1):
    """
    A Helper function to the legal moves, as an array where each row is
    a batch, row, col index giving legal moves on a given board. Includes
    blocking moves, but excludes offsets that will take the critter off the
    board

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      critter (int): value of critter we are getting the valid actions for
      radius (int): how far, in city block distance the critter can move

    Returns:
      moves: set or tuples (g, r, c)
    """
    batch_size, n_rows, n_cols = board['pieces'].shape
    b = GridworldBoard(batch_size=self.batch_size, n_rows=self.n_rows,
                       n_cols=self.n_cols, num_food=self.num_food,
                       num_prey=self.num_prey, num_pred=self.num_pred,
                       lifetime=self.lifetime, rng=self.rng)
    b.set_state(board)
    legal_offsets, valid_moves_mask =  b.get_legal_offsets(critter, radius)
    return legal_offsets, valid_moves_mask


  def get_valid_actions(self, board, critter=1, radius=1):
    """
    A Helper function to translate the g,x,y, tuples provided the
    GridworldBoard.get_legal_moves method into valid actions, represented
    as binary vectors of len num_actions.

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      critter (int): value of critter we are getting the valid actions for
      radius (int): how far, in city block distance the critter can move

    Returns:
      valids: np.ndarray(binary) batch_size x num_actions, 1's represent
              valid moves
    """
    legal_moves =  self.get_legal_moves(board, critter, radius)
    g, r, c = zip(*legal_moves)
    valids = np.zeros((self.batch_size, self.n_rows * self.n_cols))
    valids[g, np.array(r) * self.n_cols + np.array(c)] = 1
    return valids


  def display_moves(self, board, critter=1, g=0):
    """Displays possible moves for the g-th games in the batch of boards"""
    batch_size, n_rows, n_cols = board['pieces'].shape
    A=np.reshape(self.get_valid_actions(board, critter)[g],
                 (n_rows, n_cols))
    print("  ")
    print("possible moves")
    print("   ", end="")
    for col in range(self.n_cols):
      print(col, end=" ")
    print("")
    print("-----------------------")
    for col in range(self.n_cols):
      print(col, "|", end="")    # Print the row
      for row in range(self.n_rows):
        piece = A[col][row]    # Get the piece to print
        print(self.get_square_symbol(piece), end=" ")
      print("|")
    print("-----------------------")


  def get_perceptions(self, board, radius, critter):
    batch_size, n_rows, n_cols = board['pieces'].shape
    b = GridworldBoard(batch_size=self.batch_size, n_rows=self.n_rows,
                       n_cols=self.n_cols, num_food=self.num_food,
                       num_prey=self.num_prey, num_pred=self.num_pred,
                       lifetime=self.lifetime, rng=self.rng)
    b.set_state(board)
    return(b.get_perceptions(radius, critter))


  def get_next_state(self, board, critter, actions, a_indx=None):
    """
    Helper function using GridworldBoard.execute_moves to update board state
    given actions on a batch of boards, for a given critter

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      critter: integer index of the critter type
      actions: list of flat integer indexes of critter's new board positions
      a_indx: list of integer indexes indicating which actions are being taken
        on which boards in the batch

    Returns:
      a board triple signifying next state

    Note:
      if len(actions) > batch_size of board the returned board state will have
      an expanded batch size, allowing multiple paths in the game tree to be
      explored in parallel

    """
    batch_size, n_rows, n_cols = board['pieces'].shape
    if board['rounds_left'][0] <= 0:
      # assumes all boards in the batch have the same rounds left
      # no rounds left return the board unchanged
      return board
    else:
      b = GridworldBoard(batch_size=len(actions), n_rows=n_rows,
                         n_cols=n_cols, num_food=self.num_food,
                         num_prey=self.num_prey, num_pred=self.num_pred,
                         lifetime=self.lifetime, rng=self.rng)

      if a_indx is None:
        # just one move on each board in the batch
        assert batch_size == len(actions)
        b.set_state(board)
      else:
        # potentially multiple moves on each board, expand the batch
        assert len(actions) == len(a_indx)
        new_pieces = np.array([board['pieces'][ai].copy() for ai in a_indx])
        new_scores = np.array([board['scores'][ai].copy() for ai in a_indx])
        new_rounds_left = np.array([board['rounds_left'][ai].copy() for ai in a_indx])
        new_active_player = copy(board['active_player'])
        new_state = {'pieces': new_pieces,
                     'scores': new_scores,
                     'rounds_left': new_rounds_left,
                     'active_player': new_active_player}
        b.set_state(new_state)
      moves = self.actions_to_moves(actions)
      b.execute_moves(moves, critter)
      return b.get_state()


  def actions_to_moves(self, actions):
    """
    Args:
      actions: a batch length list of integer indexes for the flattened boards,
      i.e. in the range(n_cols * n_rows) actions are often much easier to use
      as training targets for NN based RL agents.
    Returns
      moves: a 3-tuple of 1-d arrays each of length batch_size,
        the first array gives the specific board within the batch,
        the second array in the tuple gives the new row coord for each critter
        on each board and the third gives the new col coord. Note this is
        exactly the format expected by GridworldBoard.execute_moves, and
        is a canonical way of indexing arrays for quick numpy operations.
    """
    moves = (np.arange(len(actions)),
             np.floor_divide(actions, self.n_cols),
             np.remainder(actions, self.n_cols))
    return moves


  def moves_to_actions(self, moves):
    """
    Args:
      moves: a 3-tuple of 1-d arrays each of length batch_size,
        the first array gives the specific board within the batch,
        the second array in the tuple gives the new row coord for each critter
        on each board and the third gives the new col coord. Note this is
        exactly the format expected by GridworldBoard.execute_moves, and
        is a canonical way of indexing arrays for quick numpy operations.
    Returns:
      actions: a batch length list of integer indexes for the flattened boards,
      i.e. in the range(n_cols * n_rows) actions are often much easier to use
      as training targets for NN based RL agents.
    """
    _, rows, cols = moves
    actions = rows * self.n_cols + cols
    return actions


  def critter_oriented_get_next_state(self, board, critter, offsets):
    """
    Translates directions in reference to the critter's location into
    moves on the board in absolute terms, while checking for
    bouncing/reflecting then get's the next state.

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      offsets: batch length list of strings one 'up', 'down', 'left', 'right'

    Returns:
      a board triple signifying next state

    Note:
      Unlike get_next_state, this method does not allow for expansion
      of the game tree, i.e. len(offsets)==batch_size required
    """
    assert len(offsets) == board['pieces'].shape[0]
    batch_size, n_rows, n_cols = board['pieces'].shape
    b = GridworldBoard(batch_size=self.batch_size, n_rows=self.n_rows,
                       n_cols=self.n_cols, num_food=self.num_food,
                       num_prey=self.num_prey, num_pred=self.num_pred,
                       lifetime=self.lifetime, rng=self.rng)
    b.set_state(board)
    moves = self.critter_direction_to_move(board, offsets, critter)
    b.execute_moves(moves, critter)
    return(b.get_state())


  def critter_direction_to_move(self, board, offsets, critter):
    """
    Translates directions in reference to the critter's location into
    moves on the board in absolute terms, while checking for
    bouncing/reflecting then returns moves. Doesn't check for collisions with
    other critters though. In general player's move methods should be checking
    valid moves and only making legal ones.

    Args:
      board: dict of np arrays representing board state
        'pieces':       batch_size x n_rows x n_cols
        'scores':       batch_size
        'rounds_left':  batch_size
      offsets: batch length list of strings,
        one of 'up', 'down', 'left', 'right'
      critter: integer index for the critter we want moves for

    Returns:
      moves: a 3-tuple of 1-d arrays each of length batch_size,
        the first array gives the specific board within the batch,
        the second array in the tuple gives the new row coord for each critter
        on each board and the third gives the new col coord. Note this is
        exactly the format expected by GridworldBoard.execute_moves, and
        is a canonical way of indexing arrays for numpy.

    Note:
      Unlike get_next_state, this method does not allow for expansion
      of the game tree, i.e. len(offsets)==batch_size required
    """
    assert len(offsets) == board['pieces'].shape[0]
    batch_size, n_rows, n_cols = board['pieces'].shape
    offset_dict = {'up': (0, -1, 0),
                   'down': (0, 1, 0),
                   'left': (0, 0, -1),
                   'right': (0, 0, 1),
                   'still': (0, 0, 0)}
    this_critter_locs = np.where(board['pieces'] == critter)
    all_critter_locs = np.where(board['pieces'] >= 1)
    offsets_array = np.hstack([np.array(offset_dict[ost_]).reshape((3,1))
                           for ost_ in offsets])
    new_locs = np.array(this_critter_locs) + offsets_array
    #check bounces at boundaries
    new_locs[1,:] = np.where(new_locs[1] >=
                               n_rows, n_rows-2, new_locs[1])
    new_locs[2,:] = np.where(new_locs[2,:] >=
                               n_cols, n_cols-2, new_locs[2,:])
    new_locs[1,:] = np.where(new_locs[1,:] < 0, 1, new_locs[1,:])
    new_locs[2,:] = np.where(new_locs[2,:] < 0, 1, new_locs[2,:])
    moves = tuple(new_locs)
    return moves


  def direction_probs_to_flat_probs(self, board, direction_probs, critter):
    """
    Converts direction probabilities in reference to the critter's location into
    probability arrays on the flattened board.

    Args:
      board: dict of np arrays representing board state
        'pieces':       batch_size x n_rows x n_cols
        'scores':       batch_size
        'rounds_left':  batch_size
      direction_probs: batch length list of dictionaries with keys
        ['up', 'down', 'left', 'right'] and corresponding probabilities.

    Returns:
      probs_arrays: list of arrays, where each array is of length n_rows*n_cols
                    and represents the flattened probability distribution for
                    board in the batch.
    """
    batch_size, n_rows, n_cols = board['pieces'].shape
    offset_dict = {
        'up': np.array((0, -1, 0)),
        'down': np.array((0, 1, 0)),
        'left': np.array((0, 0, -1)),
        'right': np.array((0, 0, 1))}
    critter_locs = np.where(board['pieces'] == critter)
    probs_arrays = np.zeros((batch_size, n_rows * n_cols))
    for batch_index in range(batch_size):
      prob_array = np.zeros(n_rows * n_cols)
      for direction, prob in direction_probs[batch_index].items():
          offset = offset_dict[direction]
          new_loc = np.array(critter_locs)[:, batch_index] + offset
          # Check bounces at boundaries
          new_loc[1] = np.where(new_loc[1] >= n_rows, n_rows-2, new_loc[1])
          new_loc[2] = np.where(new_loc[2] >= n_cols, n_cols-2, new_loc[2])
          new_loc[1] = np.where(new_loc[1] < 0, 1, new_loc[1])
          new_loc[2] = np.where(new_loc[2] < 0, 1, new_loc[2])
          # Convert 2D location to flattened index
          flattened_index = new_loc[1] * n_cols + new_loc[2]
          prob_array[flattened_index] += prob
      probs_arrays[batch_index, :] = prob_array
    return list(probs_arrays)


  def action_to_critter_direction(self, board, critter, actions):
    """
    Translates an integer index action into up/down/left/right

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      actions: a batch size ndarry of integer indexes for actions on each board

    Returns:
      offsets: a batch length list of strings 'up', 'down', 'left', 'right', 'still'
    """
    offset_dict = {(0, 0, 0): 'still',
                   (0, 0, 1): 'right',
                   (0, 0,-1): 'left',
                   (0, 1, 0): 'down',
                   (0,-1, 0): 'up'}
    batch_size, n_rows, n_cols = board['pieces'].shape
    critter_locs = np.where(board['pieces'] == critter)
    moves = (np.arange(len(actions)),
               np.floor_divide(actions, n_cols),
               np.remainder(actions, n_cols))
    # need to reverse this from above, moves is equiv to new_locs
    # new_locs = np.array(critter_locs) + offsets_array
    offsets_array = np.array(moves) - np.array(critter_locs)
    offsets = [offset_dict[tuple(o_)] for o_ in offsets_array.T]
    return offsets


  def get_valid_directions(self, board, critter):
    """
    Transforms output of get_valid_actions to a list of the valid directions
    for each board in the batch for a given critter.
    """
    offset_dict = {( 0, 1): 'right',
                   ( 0,-1): 'left',
                   ( 1, 0): 'down',
                   (-1, 0): 'up',
                   ( 0, 0): 'still'}
    batch_size, n_rows, n_cols = board['pieces'].shape
    valid_actions = self.get_valid_actions(board, critter)
    if batch_size != len(valid_actions):
      raise ValueError("Need Exactly one set of valid actions per board in batch")
    critter_locs = np.column_stack(np.where(board['pieces'] == critter))
    valid_directions = []
    for g, batch_valid in enumerate(valid_actions):
      valid_int_indices = np.where(batch_valid==1)[0]
      critter_loc = critter_locs[critter_locs[:, 0] == g, 1:]
      # critter_loc shape is (1, 2)
      critter_loc = np.squeeze(critter_loc)
      moves = np.column_stack([valid_int_indices // n_cols, valid_int_indices % n_cols])
      offsets = moves - critter_loc
      batch_valid_directions = [offset_dict[tuple(offset)] for offset in offsets]
      valid_directions.append(batch_valid_directions)
    return valid_directions


  def get_game_ended(self, board):
    """
    Helper function to signify if game has ended
    Returns a batch size np.array of -1 if not ended, and scores for each game
    in the batch if it is ended, note only returns scores if all games in the
    batch have ended
    """
    rounds_left = board['rounds_left']
    scores = board['scores']
    if np.any(rounds_left >= 1):
      return np.ones(self.batch_size) * -1.0
    else:
      return scores


  def critter_everywhere_state_expansion(self, board_state,
                                         critter=1, to_expand=0):
    """
    Expand a given board state by placing a critter at each non-food location.

    The function takes a game state and returns an expanded version of it. For
    each board in the state, it creates a new version of the board for every
    non-food location, placing a critter at that location. The scores and
    remaining rounds are copied for each new board. The result is a new game state
    with a larger number of boards, each representing a possible configuration
    with a critter at a different location.

    Args:
      board_state (dict): A dictionary containing the current game state.
      It should have the following keys:
        - 'pieces': a 3D numpy array (batch x n_col x n_row) representing the game
          board. -1 -> food, 0 -> empty cell, and 1 -> critter.
        - 'scores': 1D numpy array of the score for each board in the batch.
        - 'rounds_left': a 1D numpy array of the rounds left for
          each board in the batch.
      critter: integer index to place on the expanded board state
      to_expand (list (int)): list of batch indices to have state expanded

    Returns:
      dict: A dictionary containing the expanded game state with the same keys
        as the input. The number of boards will be larger than the input state.
    """
    pieces = board_state['pieces'].copy()
    scores = board_state['scores'].copy()
    rounds_left = board_state['rounds_left'].copy()
    active_player = copy(board_state['active_player'])
    # Determine non-food locations
    non_food_locs = np.argwhere(pieces[to_expand] != -1)
    #scrub all existing critter locations,
    # maybe later only scrub specific critter type
    pieces[pieces >= 1] = 0
    # lists to store expanded states
    expanded_pieces = []
    expanded_scores = []
    expanded_rounds_left = []
    # Iterate over each non-food location
    for i in range(non_food_locs.shape[0]):
      # Create a copy of the board
      expanded_board = np.copy(pieces[to_expand])
      # Place the critter at the non-food location
      # later consider only placing at non-food,
      # non-other critter locs
      expanded_board[tuple(non_food_locs[i])] = critter
      # Add the expanded board to the list along score and rounds_left
      expanded_pieces.append(expanded_board)
      expanded_scores.append(scores[to_expand])
      expanded_rounds_left.append(rounds_left[to_expand])
    # Convert to arrays and create expanded board state
    expanded_state = {'pieces': np.stack(expanded_pieces),
                      'scores': np.array(expanded_scores),
                      'rounds_left': np.array(expanded_rounds_left),
                      'active_player': active_player}
    return expanded_state


  def play_game(self, players=[], collect_fov_data=False, fov_radius=2,
                visualize = False):
    """This method takes a list of players the same length as num_critters,
        and then plays a batch of games with them and returns the final board
        state"""
    if len(players) != self.num_critters:
      raise ValueError("number of players different than expected")

    board = self.get_init_board()
    if visualize == True:
      self.display(board, 0)

    if collect_fov_data is True:
      batch_size, n_rows, n_cols = board['pieces'].shape
      b = GridworldBoard(batch_size=self.batch_size, n_rows=self.n_rows,
                         n_cols=self.n_cols, num_food=self.num_food,
                         num_prey=self.num_prey, num_pred=self.num_pred,
                         lifetime=self.lifetime, rng=self.rng)
    for p_idx, player_ in enumerate(players):
      if player_.critter_index != p_idx+1:
        print(player_.critter_index)
        print(p_idx + 1)
        raise ValueError("player order does not match assigned critter index")

    for ii in range(self.lifetime):
      for player_ in players:
        old_scores = board['scores']
        if collect_fov_data is True:
          b.set_state(board)
          percepts = b.get_perceptions(fov_radius)

        a_player, _, _ = player_.play(board)
        board = self.get_next_state(board, player_.critter_index, a_player)
        if visualize == True:
          self.display(board, 0)
    return board




# @title player zoo
###########################################################################
# make a separate player zoo
###########################################################################

class FoodDriftPlayer():
  """
  A player the executes the drifting pattern of the food,
  Treating move made by the environment as though maybe by another player as
  a convenient coding abstraction... also just a good way to think about things
  This will drift the food on the board based on the given offsets probabilities.
  Collisions are handled by the execute moves logic of the board object

  Parameters:
  - offset_probs: Probabilities corresponding to each offset, note implicit
    order dependence here
  - wrapping, a boolean indicating whether drifting food can fall off the edge
    of the board and re-appear on the other side of the board.


  The play method returns a batch of moves
    - nothing, just updates self.pieces
  """

  def __init__(self, game, critter_index=-1, offset_probs=[1.0, 0, 0, 0, 0],
               wrapping=False, wrap_type='random'):
    self.game = game
    self.critter_index = critter_index
    assert (isinstance(critter_index, int) and
          self.game.num_food  <= critter_index <= -1), "Value is not a negative integer or exceeds the limit."
    self.offset_probs = offset_probs
    self.wrapping = wrapping
    self.wrap_type = wrap_type


  def play(self, board):
    possible_offsets = np.array([[ 0, -1,  0], # up
                                 [ 0,  1,  0], # down
                                 [ 0,  0, -1], # left
                                 [ 0,  0,  1], # right
                                 [ 0,  0,  0]]) # still
    batch_size, n_rows, n_cols = board['pieces'].shape
    # original food locations
    food_locations = np.argwhere(board['pieces'] == self.critter_index)
    # Sample offsets for each food location
    num_food = food_locations.shape[0]
    sampled_offsets = possible_offsets[self.rng.choice(
        np.arange(possible_offsets.shape[0]),
        size=num_food, replace=True, p=self.offset_probs)]
    # Possible new food locations
    p_new_locs = food_locations + sampled_offsets

    #out of bounds rows and cols
    oob_rows = (p_new_locs[:, 1] >= n_rows) | (p_new_locs[:, 1] < 0)
    oob_cols = (p_new_locs[:, 2] >= n_cols) | (p_new_locs[:, 2] < 0)

    if self.wrapping is True:
      if self.wrap_type == 'random':
        p_wrap_row_indexes = self.rng.choice(np.arange(n_rows),
                                             size=num_food)
        p_wrap_col_indexes = self.rng.choice(np.arange(n_cols),
                                             size=num_food)
        p_new_locs[oob_rows, 1] = p_wrap_row_indexes[oob_rows]
        p_new_locs[oob_cols, 1] = p_wrap_col_indexes[oob_cols]
      else:
        #deterministic wrapping
        p_new_locs[:, 1] = np.mod(p_new_locs[:, 1], n_rows)
        p_new_locs[:, 2] = np.mod(p_new_locs[:, 2], n_cols)
    else:
      # they don't move if they hit an edge
      p_new_locs[oob_rows, 1] = food_locations[oob_rows, 1]
      p_new_locs[oob_cols, 2] = food_locations[oob_cols, 2]

    a = p_new_locs[:,1] * n_cols + p_new_locs[:,2]
    a_1hots = np.zeros((batch_size, n_cols*n_rows))
    a_1hots[(range(batch_size), a)] = 1.0
    return a, a_1hots, a_1hots, p_new_locs


class RandomValidPlayer():
  """
  Instantiate random player for GridWorld, could be prey or pred... or even food
  It leans hard on the game's get valid method and then just samples from there
  """


  def __init__(self, game, critter_index=1, speed=1):
    self.game = game
    self.critter_index = critter_index
    self.speed = speed
    assert (isinstance(critter_index, int) and
        0 < critter_index <= game.num_critters), "Value is not a positive integer or exceeds the upper limit."


  def play(self, board):
    """
    Simulates a batch of random game plays based on the given board state.

    This function computes the probability of each valid move being played
    (uniform for valid moves, 0 for others), then selects a move randomly for
    each game in the batch based on these probabilities.

    Args:
      board (dict): A dictionary representing the state of the game. It
          contains:
          - 'pieces': A (batch_size, x_size, y_size) numpy array indicating
                      the pieces on the board.
          - 'scores' (not used directly in this function, but expected in dict)
          - 'rounds_left' (not used directly in this function, but expected in dict)

    Returns:
      tuple:
      - a (numpy array): An array of shape (batch_size,) containing randomly
                         chosen actions for each game in the batch.
      - a_1hots (numpy array): An array of shape (batch_size, action_size)
                               with one-hot encoded actions.
      - probs (numpy array): An array of the same shape as 'valids' containing
                             the probability of each move being played.
    """
    batch_size, x_size, y_size = board['pieces'].shape
    valids = self.game.get_valid_actions(board, self.critter_index, self.speed)
    action_size = self.game.get_action_size()

    probs = valids / np.sum(valids, axis=1).reshape(batch_size,1)

    a = [self.game.rng.choice(action_size, p=probs[ii])
                                for ii in range(batch_size)]
    a_1hots = np.zeros((batch_size, action_size))
    a_1hots[(range(batch_size), a)] = 1.0
    return np.array(a), a_1hots, probs




class GeneralLinearPlayer():
  """
  A Player playing a linear policy defined by the given weights. Content and
  size of percept is parameterized, as is speed. Whether it is treated as a
  prey or predator type by the game depends on the critter_index, specifically
  0 < critter_index <= game.pred_prey_threshold --> prey type
  game.pred_prey_threshold < critter_index --> predator type
  note that many game loops re-assign critter index based on the order of the
  player list handed to the game loop.
  """

  def __init__(self, game, critter_index=1, weights=None, fov_radius=1, speed=1,
               has_food_percept = True,  has_edge_percept=False,
               has_prey_percept = False, has_pred_percept=False,
               get_probs=False, deterministic=False):
    """
    Args:
      game: Gridworld Game instance
        Instance of the gridworldGame class above;
      weights: a numpy array that gives the connection strengths between the
      'perception' neurons and the direction 'neurons'
      fov_radius: int how far around itself the critter perceives
      speed: int how many grid cells a critter can move in a round
    Returns:
      Nothing
    """
    # all critters need these things
    self.game = game
    self.critter_index = critter_index
    assert (isinstance(critter_index, int) and
        0 < critter_index <= game.num_critters), "Value is not a positive integer or exceeds the upper limit."
    self.get_probs = get_probs
    # these things are specific to this kind of critter
    self.deterministic = deterministic
    self.fov_radius = fov_radius
    self.speed = speed
    self.W_out_shape = 2*self.speed**2 + 2*self.speed + 1
    self.W_in_shape = 2*self.fov_radius**2 + 2*self.fov_radius + 1
    self.has_food_percept = has_food_percept
    self.has_edge_percept = has_edge_percept
    self.has_prey_percept = has_prey_percept
    self.has_pred_percept = has_pred_percept
    self.W_layers = np.sum([has_food_percept, has_edge_percept,
                            has_prey_percept, has_pred_percept])
    if weights is None:
      self.W = np.ones((self.W_layers, self.W_out_shape, self.W_in_shape))
    else:
      self.W = weights
    if self.W.shape != (self.W_layers, self.W_out_shape, self.W_in_shape):
      raise ValueError("Weights don't match expected shape given fov, speed, and percepts")
    self.default_softmax_temp = 0.05


  def direction_value_from_percept(self, percepts, W):
    """
    Determine an action based on perception.

    Args:
      percept: A batch by self.W_in_shape array representing the perceptions of
        the organism. Indices correspond to spaces around the organism. The
        values in the array can be -200 (out-of-bounds), 0 (empty space),
        or negative integers (food), or positive integers below or equal
        game.pred_prey_threshold for prey organisms or positive integers above
        game.predy_prey_threshold for predator organisms
      W: a W_layers x W_out_shape x W_in_shape weight matrix of parameters
        representing the connection strengths between the perception inputs and
        the possible output actions.

    Returns:
      output: raw output activations, these filtered by which moves are valid
        and then softmax normalized later
    """
    expanded_percepts = []
    if self.has_food_percept:
      x_food = np.asarray((percepts <= -1) & (percepts > -200), float) # batch x len W_in
      expanded_percepts.append(x_food)
    if self.has_edge_percept:
      x_edge = np.asarray(percepts == -200, float) # batch x len W_in
      expanded_percepts.append(x_edge)
    if self.has_prey_percept:
      x_prey = np.asarray((percepts > 0) &
       (percepts <= self.game.pred_prey_threshold), float) # batch x len W_in
      expanded_percepts.append(x_prey)
    if self.has_pred_percept:
      x_pred = np.asarray(percepts > self.game.pred_prey_threshold, float) # batch x len W_in
      expanded_percepts.append(x_pred)
    percept_stack = np.stack(expanded_percepts) # W_layers x batch x len_W_in

    output_activations = np.tensordot(percept_stack, W, [[0, 2], [0, 2]])
    # output activations is batch by W_out, so each row gives the raw output
    # activations for that batch.
    return output_activations


  def play(self, board, temp=None, W=None):
    """
    Simulate Play on a Board

    Args:
      board: dict {'pieces':
      (batch x num_rows x num_cols) np.ndarray of board position,
                  'scores': batch len array of current scores,
                  'rounds_left': batch len array of rounds left

    Returns:
      sampled_actions: a batch, row, col index of the move taken
      by each player on each board
      a_1hots: a batch nrow*ncol array of 1hot indices of those same moves
      v_probs: sampling probabilities for those 1hots (If the policy
      is deterministic a_1hots is returned here as well... or if getting the
      probs is an un-needed fuss to compute)
    """
    if temp is None:
      temp = self.default_softmax_temp
    if W is None:
      W = self.W
    batch_size, n_rows, n_cols = board['pieces'].shape
    perceptions = self.game.get_perceptions(board, self.critter_index,
                                            self.fov_radius)
    direction_v = self.direction_value_from_percept(perceptions, W)
    #direction_v is batch x w_out, i.e. number of different moves
    legal_offsets, valid_moves_mask = self.game.get_legal_offsets(
        board, critter=self.critter_index, radius=self.speed)
    flat_dv = direction_v[valid_moves_mask]
    batch_indexes = legal_offsets[:,0]
    # turn offsets into flat indexes, i.e. ints in range(n_rows*n_cols)
    action_indexes = legal_offsets[:,1] * n_cols + legal_offsets[:,2]
    # Set invalid positions to -inf
    value_expand = np.ones((batch_size, n_rows*n_cols)) * -np.inf
    # Fragile order dependency here in how legal offsets and flat_dv 'line up'
    # and we can slot the direction values into value expand like this
    value_expand[(batch_indexes, action_indexes)] = flat_dv

    if self.deterministic:
      sampled_actions = np.argmax(value_expand, axis=1)
      a_1Hots = np.zeros((batch_size, n_rows * n_cols))
      a_1Hots[np.arange(batch_size), sampled_actions] = 1.0
      v_probs = a_1Hots
    else:
      # Subtract max for numerical stability
      value_expand_shift = value_expand - np.max(value_expand,
                                                 axis=1, keepdims=True)
      # softmax temp scaling
      value_expand_scale = value_expand_shift/temp
      exp_value = np.exp(value_expand_scale)
      # Normalize by the sum of the exponentiated values for each row
      v_probs = exp_value / np.sum(exp_value, axis=1, keepdims=True)
      v_probs = v_probs / np.sum(v_probs, axis=1, keepdims=True)
      samp = self.game.rng.uniform(size = batch_size).reshape((batch_size,1))
      sampled_actions = np.argmax(v_probs.cumsum(axis=1) > samp, axis=1)
      a_1Hots = np.zeros((batch_size, n_rows*n_cols))
      a_1Hots[(range(batch_size), sampled_actions)] = 1.0
    return sampled_actions, a_1Hots, v_probs




class EatNearPredPlayer():
  """
  A Player playing a parameterized policy defined by the given weights
  """


  def __init__(self, game, weights=None, fov_radius=1, critter_index=2,
               get_probs=False, deterministic=False):
    """
    Args:
      game: Gridworld Game instance
        Instance of the gridworldGame class above;
      weights: 4 x 12 numpy array (assumes fov_radius = 2), that gives the
        connection strengths between the 'perception' neurons and the direction
        'neurons'
      fov_radius: int how far around itself the critter perceives, weights is
        expecting fov_radius = 1
    Returns:
      Nothing
    """
    # all critters need these things
    self.game = game
    self.critter_index = critter_index
    assert (isinstance(critter_index, int) and
        0 < critter_index <= game.num_critters), "Value is not a positive integer or exceeds the upper limit."
    self.get_probs = get_probs
    # these things are specific to this kind of critter
    self.deterministic = deterministic
    if weights is None:
      self.W = np.array(
      [[20.,  0., 0.,  0.,  0.],
       [ 0., 20., 0.,  0.,  0.],
       [ 0.,  0., 0.,  0.,  0.],
       [ 0.,  0., 0., 20.,  0.],
       [ 0.,  0., 0.,  0., 20.]])
    else:
      self.W = weights
    self.fov_radius = fov_radius
    self.default_softmax_temp = 0.05


  def direction_value_from_percept(self, percepts, W):
    """
    Determine an action based on perception.

    Args:
      percept: A batch by 1D len 12 array representing the perceptions of the
      organism. Indices correspond to spaces around the organism. The values in
      the array can be -2 (out-of-bounds), 0 (empty space), or -1 (food).
      W: a 4 x 12 weight matrix parameter representing the connection strengths
        between the 12 perceptions inputs and the 4 possible output actions.

    Returns:
      direction_probs: array of probabilities of taking each action.
    """
    # a human interpretable overview of the percept structure
    #percept_struct = [
    #  'up', 'left', 'center', 'right',  'down']
    # a human interpretable overview of the out structure
    #output_struct = ['up', 'left', 'center' 'right', 'down']
    # boolean representation of percept, no edges, no food, no other predators
    # just 1's where prey is,
    # x is batch x 5
    x = np.asarray(((percepts >= 1) &
     (percepts <= self.game.pred_prey_threshold)), int)
    # W is 5 x 5
    # this does the broadcasting we want
    output_activations = (W @ x.T).T
    # output activations is batch by 4
    return output_activations


  def play(self, board, temp=None, W=None):
    """
    Simulate Play on a Board

    Args:
      board: dict {'pieces':
      (batch x num_rows x num_cols) np.ndarray of board position,
                  'scores': batch len array of current scores,
                  'rounds_left': batch len array of rounds left

    Returns:
      sampled_actions: a batch, row, col index of the move taken
      by each player on each board
      a_1hots: a batch nrow*ncol array of 1hot indices of those same moves
      v_probs: sampling probabilities for those 1hots (If the policy
      is deterministic a_1hots is returned here as well... or if getting the
      probs is an un-needed fuss to compute)
    """
    if temp is None:
      temp = self.default_softmax_temp
    if W is None:
      W = self.W
    batch_size, n_rows, n_cols = board['pieces'].shape
    perceptions = self.game.get_perceptions(board, self.critter_index,
                                            self.fov_radius)
    # note the fragile order based dependency on how legal offsets is written,
    # and how output activations are interpreted
    direction_v = self.direction_value_from_percept(perceptions, W)
    flat_ds = direction_v.T.ravel()

    critter_locs = np.array(np.where(board['pieces'] == self.critter_index))
    legal_offsets = np.stack([
    critter_locs + np.array([np.array([0, -1,  0])]*batch_size).T, # up
    critter_locs + np.array([np.array([0,  0, -1])]*batch_size).T, # left
    critter_locs + np.array([np.array([0,  0,  0])]*batch_size).T, # still
    critter_locs + np.array([np.array([0,  0,  1])]*batch_size).T, # right
    critter_locs + np.array([np.array([0,  1,  0])]*batch_size).T, # down
    ])
    legal_offsets = np.vstack(np.transpose(legal_offsets, (0, 2, 1)))

    # conditions for offsets on the board
    c1 = legal_offsets[:,1] >= 0
    c2 = legal_offsets[:,1] <= n_rows-1
    c3 = legal_offsets[:,2] >= 0
    c4 = legal_offsets[:,2] <= n_cols-1
    all_c = np.logical_and.reduce([c1, c2, c3, c4])

    batch_indexes = legal_offsets[:,0][all_c]
    # turn offsets into flat indexes, i.e. ints in range(n_rows*n_cols)
    action_indexes = legal_offsets[:,1][all_c] * n_cols + legal_offsets[:,2][all_c]
    direction_values = flat_ds[all_c]
    # the fragile order dependency is here in that legal offsets and flat_ds
    # 'line up' and can both be indexed by 'all_c' and we get what we want
    value_expand = np.zeros((batch_size, n_rows*n_cols))
    # i.e. we can slot the direction values into value expand like this
    value_expand[(batch_indexes, action_indexes)] = direction_values
    # valids is a batch x (n_rows * n_cols) binary np array, so we use it to
    # index value_expand and set non-valid values to -inf
    valids = gwg.get_valid_actions(board, self.critter_index)
    # Set invalid positions to -inf
    valid_value_expand = np.where(valids == 1, value_expand, -np.inf)
    if self.deterministic:
      sampled_actions = np.argmax(valid_value_expand, axis=1)
      a_1Hots = np.zeros((batch_size, n_rows * n_cols))
      a_1Hots[np.arange(batch_size), sampled_actions] = 1.0
      v_probs = a_1Hots
    else:
      # Subtract max for numerical stability
      value_expand_shift = valid_value_expand - np.max(valid_value_expand,
                                                       axis=1, keepdims=True)
      # softmax temp scaling
      value_expand_scale = value_expand_shift/temp
      exp_value = np.exp(value_expand_scale)
      # Normalize by the sum of the exponentiated values for each row
      v_probs = exp_value / np.sum(exp_value, axis=1, keepdims=True)
      v_probs = v_probs / np.sum(v_probs, axis=1, keepdims=True)
      samp = self.game.rng.uniform(size = batch_size).reshape((batch_size,1))
      sampled_actions = np.argmax(v_probs.cumsum(axis=1) > samp, axis=1)
      a_1Hots = np.zeros((batch_size, n_rows*n_cols))
      a_1Hots[(range(batch_size), sampled_actions)] = 1.0
    return sampled_actions, a_1Hots, v_probs




class RandomDirectionPlayer():
  """
  Instantiate random player for GridWorld
  """

  def __init__(self, game, critter_index=1):
    self.game = game
    self.critter_index = critter_index
    assert (isinstance(critter_index, int) and
        0 < critter_index <= game.num_critters), "Value is not a positive integer or exceeds the upper limit."

  def play(self, board):
    """
    Simulates a batch of random game plays based on the given board state.

    This function assigns a uniform probability to going up down left or right
    independent of whether it is at an edge or corner or not. Then because of
    bouncing off edges it will have a higher probability of moving away from
    edges as opposed to along them than the random valid move player.

    Args:
      board (dict): A dictionary representing the state of the game. It
          contains:
          - 'pieces': A (batch_size, x_size, y_size) numpy array indicating
                      the pieces on the board.
          - 'scores' (not used directly in this function, but expected in dict)
          - 'rounds_left' (not used directly in this function, but expected in dict)

    Returns:
      tuple:
      - a (numpy array): An array of shape (batch_size,) containing randomly
                         chosen actions for each game in the batch.
      - a_1hots (numpy array): An array of shape (batch_size, action_size)
                               with one-hot encoded actions.
      - probs (numpy array): An array of the same shape as 'valids' containing
                             the probability of each move being played.
    """
    batch_size, n_rows, n_cols = board['pieces'].shape
    action_probs = {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}

    critter_oriented_moves = self.game.rng.choice(list(action_probs.keys()),
                                                  size=(batch_size))
    direction_probs = [action_probs] * batch_size
    moves = self.game.critter_direction_to_move(board, critter_oriented_moves,
                                                self.critter_index)
    probs = self.game.direction_probs_to_flat_probs(board, direction_probs,
                                                    self.critter_index)
    sampled_actions = self.game.moves_to_actions(moves)
    a_1hots = np.zeros((batch_size, n_rows*n_cols))
    a_1hots[(range(batch_size), sampled_actions)] = 1.0

    return sampled_actions, a_1hots, probs




class MonteCarloBasedPlayer():
  """
  Simulate Player based on Monte Carlo Algorithm

  Note: Has dependencies in the gw_NN_RL.py util, namely a policy/value
  network and the Monte Carlo class.
  """

  def __init__(self, game, nnet,
               critter_index=1,
               default_depth=1,
               default_rollouts=1,
               default_K=4,
               default_temp=1.0,
               random_seed=None):
    """
    Initialize Monte Carlo Parameters

    Args:
      game: Gridworld Game instance
        Instance of the gridworldGame class above;
      nnet: gridworldNet instance
        Instance of the gridworldNNet class above;
      args: dictionary
        Instantiates number of iterations and episodes, controls temperature threshold, queue length,
        arena, checkpointing, and neural network parameters:
        learning-rate: 0.001, dropout: 0.3, epochs: 10, batch_size: 64,
        num_channels: 512

    Returns:
      Nothing
    """
    self.game = game
    self.critter_index = critter_index
    assert (isinstance(critter_index, int) and
        0 < critter_index <= game.num_critters), "Value is not a positive integer or exceeds the upper limit."
    self.nnet = nnet
    self.default_depth = default_depth
    self.default_rollouts = default_rollouts
    self.mc = MonteCarlo(self.game, self.nnet, self.default_depth)
    self.default_K = default_K
    self.default_temp = default_temp
    self.rng = np.random.default_rng(seed=random_seed)


  def play(self, board,
           num_rollouts=None,
           rollout_depth=None,
           K=None,
           softmax_temp=None):
    """
    Simulates a batch Monte Carlo based plays on the given board state.

    Computes the probability of each valid move being played using a softmax
    activation on the Monte Carlo based value (Q) of each action then selects a
    move randomly for each game in the batch based on those probabilities.

    Args:
      board (dict): A dictionary representing the state of the game. It
          contains:
          - 'pieces': A (batch_size, x_size, y_size) numpy array indicating
                      the pieces on the board.
          - 'scores' (not used directly in this function, but expected in dict)
          - 'rounds_left' (not used directly in this function, but expected in dict)

    Returns:
      tuple:
      - a (numpy array): An array of shape (batch_size,) containing randomly
                         chosen actions for each game in the batch.
      - a_1hots (numpy array): An array of shape (batch_size, action_size)
                               with one-hot encoded actions.
      - probs (numpy array): An array of the same shape as 'valids' containing
                             the probability of each move being played.
    """
    batch_size, n_rows, n_cols = board['pieces'].shape
    if num_rollouts is None:
      num_rollouts = self.default_rollouts
    if rollout_depth is None:
      rollout_depth = self.default_depth
    if K is None:
      K = self.default_K
    if softmax_temp is None:
      softmax_temp = self.default_temp

    # figure out top k actions according to normalize action probability
    # given by our policy network prediction
    #co_pieces = board['pieces'].copy()
    #this_critter_locs = np.where(co_pieces == self.critter_index+1)
    #all_critter_locs = np.where(co_pieces >= 1)
    # other critters are invisible to this player
    #co_pieces[all_critter_locs] = 0
    # nnet trained to see self as 1
    #co_pieces[this_critter_locs] = 1
    #scalar_rounds_left = board['rounds_left'][0]
    #co_rounds_left = scalar_rounds_left // self.game.num_critters
    #if self.critter_index-1 < scalar_rounds_left % self.game.num_critters:
       # add an extra if we haven't had this players turn yet in the round cycle
    #   co_rounds_left = co_rounds_left + 1
    #co_rounds_left = np.array([co_rounds_left]*batch_size)
    #pis, vs = self.nnet.predict(co_pieces,
    #                            board['scores'][:,self.critter_index-1],
    #                            co_rounds_left)
    pis, vs = self.mc.pis_vs_from_board(board, self.critter_index)
    valids = self.game.get_valid_actions(board, self.critter_index)
    masked_pis = pis * valids  # Masking invalid moves
    sum_pis = np.sum(masked_pis, axis=1)
    num_valid_actions = np.sum(valids, axis=1)
    effective_topk = np.array(np.minimum(num_valid_actions, K), dtype= int)
    probs = np.array([masked_pi / masked_pi.sum() if masked_pi.sum() > 0
                      else valid / valid.sum()
                      for valid, masked_pi in zip(valids, masked_pis)])
    partioned = np.argpartition(probs,-effective_topk)
    topk_actions = [partioned[g,-(ii+1)]
                      for g in range(batch_size)
                        for ii in range(effective_topk[g])]
    topk_actions_index = [ii
                            for ii, etk in enumerate(effective_topk)
                              for _ in range(etk)]
    values = np.zeros(len(topk_actions))
    # Do some rollouts
    for _ in range(num_rollouts):
      values = values + self.mc.simulate(board, topk_actions,
                                         topk_actions_index,
                                         critter=self.critter_index,
                                         depth=rollout_depth)
    values = values / num_rollouts

    value_expand = np.zeros((batch_size, n_rows*n_cols))
    value_expand[(topk_actions_index, topk_actions)] = values
    value_expand_shift = value_expand - np.max(value_expand, axis=1, keepdims=True)
    value_expand_scale = value_expand_shift/softmax_temp
    v_probs = np.exp(value_expand_scale) / np.sum(
        np.exp(value_expand_scale), axis=1, keepdims=True)
    v_probs = v_probs * valids
    v_probs = v_probs / np.sum(v_probs, axis=1, keepdims=True)
    samp = self.rng.uniform(size = batch_size).reshape((batch_size,1))
    sampled_actions = np.argmax(v_probs.cumsum(axis=1) > samp, axis=1)
    a_1Hots = np.zeros((batch_size, n_rows*n_cols))
    a_1Hots[(range(batch_size), sampled_actions)] = 1.0
    return sampled_actions, a_1Hots, v_probs




class SimpleRulePlayer():
  """
  A Predator Player based on the following simple policy:
  If there is any prey immediately nearby move towards it,
  otherwise move to a random valid location.
  """

  def __init__(self, game, fov_radius=1, critter_index=1):
    self.game = game
    self.critter_index = critter_index
    assert (isinstance(critter_index, int) and
        0 < critter_index <= game.num_critters), "Value is not a positive integer or exceeds the upper limit."
    self.fov_radius = fov_radius


  def simple_action_from_percept(self, percept):
    """
    Determine an action based on perception.

    Args:
      percept: A 1D array (len 4 if fov_radius = 1)representing the perception
        of the organism. Indices correspond to spaces around the organism. The
        values in the array can be -200 (out-of-bounds), 0 (empty space),
        negative integers > -200 (food),
        positive integers <= self.game.pred_prey_threshold are prey
        positive integers > self.game.pred_prey_threshold are predators

    Returns:
      action: a str, one of 'up', 'down', 'left', 'right'. If food in one or
        more of the spaces immediately beside the organism, the function will
        return a random choice among these directions. If there is no food
        nearby, the function will return a random direction.
    """
    # a human interpretable overview of the percept structure if fov = 2
    percept_struct = ['up', 'left', 'right', 'down']
    # Defines directions corresponding to different perception indices
    direction_struct = [
      'None', 'None', 'up', 'None',
      'None', 'left', 'right', 'None',
      'None', 'down', 'None', 'None']
    # these are what count as nearby in the percept
    nearby_directions = ['near up', 'near left', 'near right', 'near down']
    # Get the corresponding indices in the percept array
    nearby_indices = [percept_struct.index(dir_) for dir_ in nearby_directions]
    # Identify the directions where food is located
    food_indices = [index for index in nearby_indices if percept[index] <= -1]
    food_directions = [direction_struct[index] for index in food_indices]

    action_probs = {'up': 0.0, 'down': 0.0, 'left': 0.0, 'right': 0.0}
    if len(food_directions) > 0:  # If there is any food nearby
      # If there is any food nearby randomly choose a direction with food
      action = self.game.rng.choice(food_directions)  # Move towards a random one
      for direction in food_directions:
        action_probs[direction] = 1.0 /len(food_directions)
    else:
      # If there is no food nearby, move randomly
      action = self.game.rng.choice(['up', 'down', 'left', 'right'])
      for direction in ['up', 'down', 'left', 'right']:
        action_probs[direction] = 0.25

    return action, action_probs


  def play(self, board):
    """
    Simulate Play on a Board

    Args:
      board: dict {'pieces':
      (batch x num_rows x num_cols) np.ndarray of board position,
                  'scores': batch len array of current scores,
                  'rounds_left': batch len array of rounds left

    Returns:
      sampled_actions: a batch, row, col index of the move taken
      by each player on each board
      a_1hots: a batch nrow*ncol array of 1hot indices of those same moves
      probs: sampling probabilities for those 1hots (If the policy
      is deterministic a_1hots is returned here as well... or if getting the
      probs is an un-needed fuss to compute)

    """
    batch_size, n_rows, n_cols = board['pieces'].shape
    perceptions = self.game.get_perceptions(board, self.fov_radius,
                                            self.critter_index)

    critter_oriented_moves = []
    direction_probs = []
    for g in range(batch_size):
      action, action_probs = self.simple_action_from_percept(perceptions[g])
      critter_oriented_moves.append(action)
      direction_probs.append(action_probs)
    moves = self.game.critter_direction_to_move(board, critter_oriented_moves,
                                                direction_probs,
                                                self.critter_index)
    probs = self.game.direction_probs_to_flat_probs(board, direction_probs)
    sampled_actions = self.game.moves_to_actions(moves)
    a_1hots = np.zeros((batch_size, n_rows*n_cols))
    a_1hots[(range(batch_size), sampled_actions)] = 1.0

    return sampled_actions, a_1hots, probs




class PerceptParamPlayer():
  """
  A Player playing a parameterized policy defined by the given weights
  """


  def __init__(self, game, weights=None, fov_radius=2, critter_index=1):
    """
    Args:
      game: Gridworld Game instance
        Instance of the gridworldGame class above;
      weights: 4 x 12 numpy array (assumes fov_radius = 2), that gives the
        connection strengths between the 'perception' neurons and the direction
        'neurons'
      fov_radius: int how far around itself the critter perceives, weights is
        expecting fov_radius = 2
    Returns:
      Nothing
    """
    self.game = game
    self.critter_index = critter_index
    assert (isinstance(critter_index, int) and
        0 < critter_index <= game.num_critters), "Value is not a positive integer or exceeds the upper limit."
    if weights is None:
      self.W = np.array(
      [[1., 1., 4., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 4., 1., 1.],
       [0., 1., 0., 0., 1., 4., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 4., 1., 0., 0., 1., 0.]])
    else:
      self.W = weights
    self.fov_radius = fov_radius
    self.default_softmax_temp = 0.05


  def param_action_from_percept(self, percept, valid_directions, W,
                                softmax_temp=None):
    """
    Determine an action based on perception.

    Args:
      percept: A 1D len 12 array representing the perception of the organism.
        Indices correspond to spaces around the organism. The values in the
        array can be -2 (out-of-bounds), 0 (empty space), or -1 (food).
      W: a 4 x 12 weight matrix parameter representing the connection strengths
        between the 12 perceptions inputs and the 4 possible output actions.

    Returns:
      direction: a str, one of 'up', 'down', 'left', 'right'. If food in one or
        more of the spaces immediately beside the organism, the function will
        return a random choice among these directions. If there is no food
        nearby, the function will return a random direction.
      direction_probs: dictionary with probabilities of taking each action.
    """
    if len(valid_directions) == 0:
      # if there is no where legit to move, stay put
      return 'still', {direction: 0 for direction in output_struct}

    if softmax_temp is None:
      # very low temp, basically deterministic for this range of values
      softmax_temp = self.default_softmax_temp
    # a human interpretable overview of the percept structure
    percept_struct = [
      'far up', 'left up', 'near up', 'right up',
      'far left', 'near left', 'near right', 'far right',
      'left down', 'near down', 'right down', 'far down']
    # a human interpretable overview of the out structure
    output_struct = ['up', 'down', 'left', 'right']
    # boolean representation of percept, no edges, just 1's where food is,
    # zero otherwise, also means other organisms are invisible
    x = np.asarray(percept <= -1, int)
    output_activations = W @ x

    # softmax shift by max, scale by temp
    shift_scale_ex = np.exp((output_activations -
                             np.max(output_activations))/softmax_temp)
    # set invalid direction activations to zero
    invalid_directions = [direction for direction in output_struct
                           if direction not in valid_directions]
    invalid_indices = [output_struct.index(direction)
                        for direction in valid_directions]
    sm = shift_scale_ex / shift_scale_ex.sum() #normalized
    # set invalid direction probabilities to zero
    invalid_directions = [direction for direction in output_struct
                           if direction not in valid_directions]
    invalid_indices = [output_struct.index(direction)
                        for direction in invalid_directions]
    sm[invalid_indices] = 0
    probs_sm = sm / sm.sum(axis=0) #re-normalized again for fp issues
    direction = self.game.rng.choice(output_struct, p=probs_sm)
    direction_probs = {direction: prob
                        for direction, prob in zip(output_struct, probs_sm)}
    return direction, direction_probs


  def play(self, board, temp=None):
    """
    Simulate Play on a Board

    Args:
      board: dict {'pieces':
      (batch x num_rows x num_cols) np.ndarray of board position,
                  'scores': batch len array of current scores,
                  'rounds_left': batch len array of rounds left

    Returns:
      sampled_actions: a batch, row, col index of the move taken
      by each player on each board
      a_1hots: a batch nrow*ncol array of 1hot indices of those same moves
      v_probs: sampling probabilities for those 1hots (If the policy
      is deterministic a_1hots is returned here as well... or if getting the
      probs is an un-needed fuss to compute)
    """
    if temp is None:
      temp = self.default_softmax_temp
    batch_size, n_rows, n_cols = board['pieces'].shape
    perceptions = self.game.get_perceptions(board, self.fov_radius,
                                            self.critter_index)
    critter_oriented_moves = []
    direction_probs = []

    # Get valid actions for each game in the batch
    valid_directions = self.game.get_valid_directions(board, self.critter_index)
    for g in range(batch_size):
      direction, batch_direction_probs = self.param_action_from_percept(
        perceptions[g], valid_directions[g], self.W, softmax_temp=temp)
      critter_oriented_moves.append(direction)
      direction_probs.append(batch_direction_probs)
    moves = self.game.critter_direction_to_move(board, critter_oriented_moves,
                                                self.critter_index)
    probs = self.game.direction_probs_to_flat_probs(board, direction_probs, self.critter_index)
    sampled_actions = self.game.moves_to_actions(moves)
    a_1hots = np.zeros((batch_size, n_rows*n_cols))
    a_1hots[(range(batch_size), sampled_actions)] = 1.0

    return sampled_actions, a_1hots, probs




# @title interactive gridworld widgets

########################################
# widgets refactor for multi-critter
#########################################
# Interactive Gridworld Game Widgets




class InteractiveGridworld():
  """
  A widget based object for interacting with a gridworld game
  """

  def __init__(self, gridworld_game, init_board=None, has_fov=False,
               radius=2, fov_opaque=False, collect_fov_data=False,
               figsize=(6,5), critter_names=['Critter'], players=['human']):
    """
    Initializes a widget based object for interacting with a gridworld game

    Args:
      gridworld_game: an instance of GridworldGame object
        expects this to have batchsize 1
      init_board: (optional) a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
        if left out will initialize with a random board state
      has_fov: bool, whether or not to display fog of war around the critter
      radius: int, number of squares the critter can "see" around it
      figsize: tuple (int, int), size of the figure
      critter_names: a list of strings that determines what the critter is called
        in the plot legend, order should align with players
      player: a list of either 'human', None, or a player object with a play
        method and a critter_index attribute. If 'human' use buttons,  if None
        default to making a RandomValidPlayer object, otherwise use the
        player class provided to make the player objects and use a start button.
        The list needs to be as long as the gridworld_game.num_critters
        attribute. Order should align with critter_name.

      Note: fov is going to look pretty janky with more than one player, maybe
      we get fov to only turn on for the 'active' player?
    """

    # Set GridworldGame object and initialize the board state
    self.gwg = gridworld_game
    self.has_fov = has_fov
    self.radius = radius
    self.fov_opaque = fov_opaque
    self.percept_len = 2*self.radius*(self.radius+1)
    self.collect_fov_data = collect_fov_data
    self.figsize = figsize
    # initialize players and plotting specs together to ensure alignment
    self.players = []
    self.any_human_players = False
    self.active_player_index = 0
    self.crit_specs = []
    markers = ['h', 'd']  # hexagon and diamond
    colors = sns.color_palette("colorblind")
    for i in range(self.gwg.num_critters):
      spec = {'marker': markers[i % len(markers)],
              'color': colors[i // len(markers) % len(colors)],
              'name': critter_names[i],
              'int_id': i+1}
      self.crit_specs.append(spec)
      player = players[i] #implict check that players is at least long enough
      if player is None:
        self.players.append(RandomValidPlayer(self.gwg, critter_index=i+1))
      elif player == 'human':
        self.players.append('human')
        # right now only ever have on human player with index 1
        self.any_human_players = True
      else:
        # player objects expected to have a critter_index attribute
        # we set it appropriately here so it aligns with the players list
        # used to create the widget
        player.critter_index = i+1
        self.players.append(player)
    self.final_scores = []
    if init_board is None:
      self.board_state = self.gwg.get_init_board()
    else:
      self.board_state = init_board
    if self.collect_fov_data is True:
      # keep raw records of percept and eating for manipulation later
      self.percept_eat_records = []
      # keep data in contingency table of how many food items were in
      # the percept, and whether or not food was eaten
      self.fov_eat_table_data = np.zeros((2, self.percept_len+1))
    # Initialize widgets and buttons
    self.output = widgets.Output(layout=widgets.Layout(
      width = '20.0em', min_width='20.0em', max_width='21.0em',
      min_height='10.0em', overflow='auto'))
    self.scoreboard = widgets.Output(layout=widgets.Layout(
      min_width='12.5em', max_width='18.8em',
      min_height='6.3em', overflow='auto'))
    self.fov_eat_table_display = widgets.Output(layout=widgets.Layout(
      min_width='25.0em', min_height='18.8em', overflow='auto'))
    self.up_button = widgets.Button(description="Up",
      layout=widgets.Layout(width='6.3em'))
    self.down_button = widgets.Button(description="Down",
      layout=widgets.Layout(width='6.3em'))
    self.left_button = widgets.Button(description="Left",
      layout=widgets.Layout(width='6.3em'))
    self.right_button = widgets.Button(description="Right",
      layout=widgets.Layout(width='6.3em'))
    self.start_button = widgets.Button(description="Start",
      layout=widgets.Layout(width='6.3em'))

    # get plot canvas widgets and other plotting objects
    plt.ioff()
    if self.collect_fov_data and self.any_human_players:
      self.legend_type = None # don't keep regenerating the legend
      # do legend separately if showing observations and no human player
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov,
       self.b_fig_legend, self.b_ax_legend) = self.gwg.plot_board(
          self.board_state, g=0, critter_specs=self.crit_specs,
          legend_type='separate', figsize=self.figsize, has_fov=self.has_fov,
          radius=self.radius, fov_opaque=self.fov_opaque)
    elif len(self.players) > 1:
      self.legend_type=None # don't keep regenerating the legend
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov,
       self.b_fig_legend, self.b_ax_legend) = self.gwg.plot_board(
          self.board_state, g=0, critter_specs=self.crit_specs,
          has_fov=self.has_fov, legend_type='separate',
          radius=self.radius, fov_opaque=self.fov_opaque, figsize=self.figsize)
    else:
      self.legend_type = 'included'
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov
        ) = self.gwg.plot_board(self.board_state, g=0,
                                critter_specs=self.crit_specs,
                                has_fov=self.has_fov,
                                fov_opaque=self.fov_opaque,
                                radius=self.radius, figsize=self.figsize)
    # lump buttons together
    self.buttons = widgets.HBox([self.left_button,
                               widgets.VBox([self.up_button, self.down_button]),
                               self.right_button])
    # automatically pick different layouts for different situations
    if self.any_human_players:
      self.board_and_buttons = widgets.VBox([self.b_fig.canvas,
                                             self.buttons])
      if len(self.players) == 1:
        #one human player
        self.output_and_score = widgets.HBox([self.scoreboard, self.output])
        self.no_table_final_display = widgets.VBox([self.board_and_buttons,
                                                  self.output_and_score])
        if self.collect_fov_data == True:
          # a single human player collecting data
          self.final_display = widgets.HBox([self.no_table_final_display,
                                           self.fov_eat_table_display])
        else: # self.collect_fov_data == False:
          # a single human player not collecting data
          self.final_display = self.no_table_final_display
      else:
        # more than one player, one of them human
        self.V_board_outbput = widgets.VBox([self.board_and_buttons,
                                             self.output])
        self.V_scoreboard_start_legend = widgets.VBox([
        self.scoreboard, self.start_button, self.b_fig_legend.canvas])
        self.final_display = widgets.HBox([self.V_board_outbput,
                                             self.V_scoreboard_start_legend])
    else: # player is some kind of ai
      if self.collect_fov_data == True:
        # an ai player with recording
        # in this case legend is separate
        self.V_score_start_output_legend = widgets.VBox([self.scoreboard,
          self.start_button,  self.output, self.b_fig_legend.canvas])
        self.V_board_table = widgets.VBox([self.b_fig.canvas,
                                           self.fov_eat_table_display])
        self.final_display = widgets.HBox([self.V_board_table,
                                           self.V_score_start_output_legend])
      else:
        if len(self.players) == 1:
          # an ai player without recording
          self.H_score_output_start = widgets.HBox([
            self.scoreboard, self.output, self.start_button])
          self.final_display = widgets.VBox([
            self.b_fig.canvas, self.H_score_output_start])
        else:
          # more than one ai player
          self.V_board_outbput = widgets.VBox([self.b_fig.canvas, self.output])
          self.V_scoreboard_start_legend = widgets.VBox([
              self.scoreboard, self.start_button, self.b_fig_legend.canvas])
          self.final_display = widgets.HBox([self.V_board_outbput,
                                             self.V_scoreboard_start_legend])

    # initialize text outputs
    with self.scoreboard:
      table = [['High Score:'] + ['--'] * self.gwg.num_critters,
               ['Last Score:'] + ['--'] * self.gwg.num_critters,
               ['Average Score:'] + ['--'] * self.gwg.num_critters,]
      if len(self.players) > 1:
        headers = [''] + [f'P{i+1}' for i in range(self.gwg.num_critters)]
        print(tabulate(table, headers=headers))
      else: # len(self.players) == 1
        print(tabulate(table))
    with self.output:
      if self.any_human_players:
        print('Click a button to start playing')
      else:
        print('Click the start button to run the simulation')
    with self.fov_eat_table_display:
      printmd("**Observations**")
      table_data = [[str(ii),
                     str(self.fov_eat_table_data[0,ii]),
                     str(self.fov_eat_table_data[1,ii])] for ii in range(11)]
      table = ([['Food in Percept', 'Food Not Eaten', 'Food Eaten']] +
               table_data)
      print(tabulate(table))

    # Connect the buttons to functions that do something
    self.up_button.on_click(self.on_up_button_clicked)
    self.down_button.on_click(self.on_down_button_clicked)
    self.left_button.on_click(self.on_left_button_clicked)
    self.right_button.on_click(self.on_right_button_clicked)
    self.start_button.on_click(self.on_start_button_clicked)


  def button_output_update(self, which_button):
    old_board = self.board_state.copy()
    # index of players is 0 through num_critter-1,
    # same player represented by value of index + 1 in
    old_scores = old_board['scores'][0]
    if self.collect_fov_data is True:
      batch_size, n_rows, n_cols = old_board['pieces'].shape
      b = GridworldBoard(batch_size, n_rows, n_cols,
                         self.gwg.num_food, self.gwg.lifetime,
                         rng=self.gwg.rng)
      b.set_state(old_board)
      percept = b.get_perceptions(self.radius)[0]

    if (isinstance(self.players[self.active_player_index], str) and
        'human' in self.players[self.active_player_index]):
      direction = which_button
    else:
      a_player, _, _ = self.players[self.active_player_index].play(old_board)
      # print(a_player)
      a_player = self.gwg.action_to_critter_direction(old_board,
                                                      self.active_player_index+1,
                                                      a_player)
      # but we only want to apply their move to the appropriate board
      direction = a_player[0]

    self.board_state = self.gwg.critter_oriented_get_next_state(
          self.board_state, self.active_player_index+1, [direction])
    new_scores = self.board_state['scores'][0] #first batch first critter type
    rounds_left = self.board_state['rounds_left'][0]
    num_moves = np.floor(self.gwg.lifetime -
                         rounds_left / self.gwg.num_critters)
    if new_scores[self.active_player_index] > old_scores[self.active_player_index]:
      #eating happened
      eating_string = "They ate the food/prey there!"
      did_eat = 1
    else: #eating didn't happen
      eating_string = "There's no food/prey there."
      did_eat = 0
    row, col = self.gwg.get_critter_rc(self.board_state, 0,
                                       self.active_player_index+1)
    (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov
     ) = self.gwg.plot_board(self.board_state, g=0,
                             fig=self.b_fig, ax=self.b_ax,
                             critter_specs=self.b_crit_specs, food=self.b_food,
                             fov=self.b_fov, has_fov=self.has_fov,
                             fov_opaque=self.fov_opaque,
                             radius=self.radius, legend_type=self.legend_type)
    if self.collect_fov_data is True:
      p_e_data = {'perception': percept.copy(),
                  'state': old_board,
                  'did_eat': bool(did_eat)}
      self.percept_eat_records.append(p_e_data)
      percept_int = np.sum(percept==-1) # number of food items in FoV
      self.fov_eat_table_data[did_eat, percept_int] += 1

    with self.output:
      clear_output()
      if len(self.players) == 1:
        print("The critter (tried) to move " + direction +
              " and is now at ({}, {}).".format(row,col))
        print(eating_string)
        print("Rounds Left: {}\nFood Eaten: {}\nFood Per Move: {:.2f}".format(
            rounds_left, new_scores[self.active_player_index],
            new_scores[self.active_player_index] / num_moves))
      else: # more than one players
        print("Critter {} (tried) to move ".format(self.active_player_index+1) +
              direction +
              " and is now at ({}, {}).".format(row, col))
        print(eating_string)
        print("Rounds Left: {}\nFood Eaten: {}".format(
            rounds_left, new_scores))
    if rounds_left == 0:
      self.final_scores.append(new_scores)
      with self.output:
        clear_output
        print('Game Over. Final Score {}'.format(new_scores))
        print('Resetting the board for another game')
        self.board_state = self.gwg.get_init_board()
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov
       ) = self.gwg.plot_board(self.board_state, 0, self.b_fig, self.b_ax,
                               self.b_crit_specs, self.b_food, self.b_fov,
                               has_fov=self.has_fov, radius=self.radius,
                               fov_opaque=self.fov_opaque,
                               legend_type=self.legend_type)
    with self.scoreboard:
        clear_output()
        print('Games Played: ' + str(len(self.final_scores)))
        if len(self.players) == 1:
          if len(self.final_scores) > 0:
            table = [
              ['High Score:', str(np.max(np.array(self.final_scores)))],
              ['Last Score:', str(self.final_scores[-1])],
              ['Average Score',
              '{:.2f}'.format(np.mean(np.array(self.final_scores)))]]
          else:
            table = [['High Score:', '--'],
                     ['Last Score:', '--'],
                     ['Average Score:', '--']]
          print(tabulate(table))
        else: # len(self.players) > 1
          headers = [''] + [f'P{i+1}' for i in range(self.gwg.num_critters)]
          if len(self.final_scores) > 0:
            table = []
            # Assuming the batch size is 1 for now
            current_scores = self.final_scores[-1]
            max_scores = np.max(np.array(self.final_scores), axis=0)
            average_scores = np.mean(np.array(self.final_scores), axis=0)
            table.append(['High Scores:'] +
              [str(score) for score in max_scores])
            table.append(['Last Scores:'] +
              [str(score) for score in current_scores])
            table.append(['Average Scores:'] +
              ['{:.2f}'.format(score) for score in average_scores])
          else:
            table = [
              ['High Score:'] + ['--'] * self.gwg.num_critters,
              ['Last Score:'] + ['--'] * self.gwg.num_critters,
              ['Average Score:'] + ['--'] * self.gwg.num_critters,]
          print(tabulate(table, headers=headers))

    with self.fov_eat_table_display:
      clear_output()
      printmd("**Observations**")
      table_data = [[str(ii),
                     str(self.fov_eat_table_data[0,ii]),
                     str(self.fov_eat_table_data[1,ii])] for ii in range(11)]
      table = ([['Food in Percept', 'Food Not Eaten', 'Food Eaten']] +
               table_data)
      print(tabulate(table))

  def disable_direction_buttons(self):
    self.up_button.disabled = True
    self.down_button.disabled = True
    self.left_button.disabled = True
    self.right_button.disabled = True

  def enable_direction_buttons(self):
    self.up_button.disabled = False
    self.down_button.disabled = False
    self.left_button.disabled = False
    self.right_button.disabled = False

  def human_ai_player_loop(self, direction):
    self.disable_direction_buttons()  # Disable buttons, no double clicks
    # Execute the move of the human who clicked the button
    self.button_output_update(direction)
    # Move to the next player
    def update_player_and_rounds():
      """Update the player index and decrement rounds if a full loop is completed."""
      self.active_player_index = (self.active_player_index + 1) % len(self.players)
      if self.active_player_index == 0:
        self.board_state['rounds_left'] -= 1
    update_player_and_rounds()
    # Do AI moves if there are any
    while self.players[self.active_player_index] != 'human':
      self.button_output_update('tbd')
      # Move to the next player
      update_player_and_rounds()
    # Next player is human turn buttons on for them
    self.enable_direction_buttons()

  def on_up_button_clicked(self, *args):
    self.human_ai_player_loop('up')

  def on_down_button_clicked(self, *args):
    self.human_ai_player_loop('down')

  def on_left_button_clicked(self, *args):
    self.human_ai_player_loop('left')

  def on_right_button_clicked(self, *args):
    self.human_ai_player_loop('right')

  def on_start_button_clicked(self, *args):
    self.start_button.disabled = True
    for ii in range(self.gwg.lifetime*self.gwg.num_critters):
      self.button_output_update('tbd')
      time.sleep(0.2)
    self.start_button.disabled = False




class Head2HeadGridworld():
  """
  A widget for interacting with a gridworld game while an artificial player
  plays on an identical board or watching two artificial players play, again
  with identical starting positions (though RNG not synched between the two
  boards, so not like duplicate bridge). We are not going to worry about having
  more than 1 critter type playing in head to head, (maybe we will to talk
  about cooperation... maybe).
  """

  def __init__(self, gridworld_game, init_board=None,
               player0='human', p0_short_name='YOU', p0_long_name=None,
               player1=None, p1_short_name='THEM', p1_long_name=None,
               has_fov=False, radius=2, fov_opaque=False,
               collect_fov_data=False, critter_name='Critter',
               figsize=(5,4.5), has_temp_slider=False):
    """
    Initializes a widget based object for interacting with a gridworld game

    Args:
      gridworld_game: an instance of GridworldGame object
        expects this to have batch_size of 2
      init_board: (optional) a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
        if left out will initialize with a random board state
      player0: object with a play method that takes a board state
        as an argument and returns a move. If none will use a random player
        if the special string 'human' is passed make arrow keys for that player
      player1: same deal as player0, never more than 1 human player
      has_fov: bool, whether or not to display field of view around the critter
      radius: int, number of squares the critter can "see" around it
    """
    # Set GridworldGame object and initialize the board state
    self.gwg = gridworld_game
    self.final_scores = []
    self.player0 = player0
    self.p0_short_name = p0_short_name
    self.p0_long_name = p0_long_name
    self.player1 = player1
    self.p1_short_name = p1_short_name
    self.p1_long_name = p1_long_name
    self.no_human = True
    if self.player0 == 'human':
      assert self.player1 != 'human'
      self.no_human = False
    if self.player1 == 'human':
      assert self.player0 != 'human'
      self.no_human = False
    self.p0_next_move = None
    self.p1_next_move = None
    self.has_fov = has_fov
    self.radius = radius
    self.fov_opaque = fov_opaque
    self.percept_len = 2*self.radius*(self.radius+1)
    self.collect_fov_data = collect_fov_data
    self.critter_name = critter_name
    self.figsize = figsize
    if player0 is None:
      self.player0 = RandomValidPlayer(self.gwg)
    else:
      self.player0 = player0
    if player1 is None:
      self.player1 = RandomValidPlayer(self.gwg)
    else:
      self.player1 = player1
    self.has_temp_slider = has_temp_slider

    if self.collect_fov_data is True:
      self.percept_eat_records = []
      self.fov_eat_table_data = np.zeros((2, self.percept_len+1))
    if init_board is None:
      self.board_state = self.gwg.get_init_board()
    else:
      self.board_state = init_board
    #print(self.board_state)

    # both players have same starting board
    self.board_state['pieces'][1] = self.board_state['pieces'][0].copy()

    # Initialize widgets and buttons
    if self.has_temp_slider:
      self.sft_slider_label = widgets.Label(value='Softmax Temperature')
      self.sft_slider = widgets.FloatSlider(value=1.0, min=0.05,
                                            max=5.0, step=0.05)
      self.softmax_temp_slider = widgets.VBox([self.sft_slider_label,
                                               self.sft_slider])
    self.output0 = widgets.Output(layout=widgets.Layout(
      width = '20.0em', min_width='20.0em', max_width='21.0em',
      min_height='10.0em', overflow='auto'))
    self.output1 = widgets.Output(layout=widgets.Layout(
      width = '20.0em', min_width='20.0em', max_width='21.0em',
      min_height='10.0em', overflow='auto'))
    self.scoreboard = widgets.Output(layout=widgets.Layout(
      min_width='20em', max_width='21em', min_height='6.3em', overflow='auto'))
    self.up_button = widgets.Button(description="Up",
                                    layout=widgets.Layout(width='6.3em'))
    self.down_button = widgets.Button(description="Down",
                                      layout=widgets.Layout(width='6.3em'))
    self.left_button = widgets.Button(description="Left",
                                      layout=widgets.Layout(width='6.3em'))
    self.right_button = widgets.Button(description="Right",
                                       layout=widgets.Layout(width='6.3em'))
    self.start_button = widgets.Button(description="Start",
      layout=widgets.Layout(width='6.3em', margin='0.6em 0 0 0'))  # 0.6em top margin

    #set up buttons and outputs and layouts
    self.buttons = widgets.HBox([self.left_button,
                               widgets.VBox([self.up_button, self.down_button]),
                               self.right_button])
    plt.ioff()
    (self.b_fig0, self.b_ax0, self.b_crit_specs0, self.b_food0, self.b_fov0,
     self.b_fig_legend, self.b_ax_legend) = self.gwg.plot_board(
        self.board_state, g=0, legend_type='separate', figsize=self.figsize,
        has_fov=self.has_fov, radius=self.radius, fov_opaque=self.fov_opaque,
        name=self.critter_name, title=self.p0_long_name)
    (self.b_fig1, self.b_ax1, self.b_crit_specs1, self.b_food1, self.b_fov1
     ) = self.gwg.plot_board(self.board_state, g=1, legend_type=None,
                             figsize=self.figsize, has_fov=self.has_fov,
                             radius=self.radius, fov_opaque=self.fov_opaque,
                             title=self.p1_long_name)
    # player 0 is human
    self.board_buttons_and_output0 = widgets.VBox(
      [self.b_fig0.canvas, self.buttons, self.output0])
    # player 1 is human
    self.board_buttons_and_output1 = widgets.VBox(
      [self.b_fig1.canvas, self.buttons, self.output1])
    # non human players
    self.board_and_output0 = widgets.VBox([self.b_fig0.canvas, self.output0])
    self.board_and_output1 = widgets.VBox([self.b_fig1.canvas, self.output1])

    self.legend_and_scores = widgets.VBox([self.b_fig_legend.canvas,
                                           self.scoreboard])
    if self.has_temp_slider:
      self.legend_scores_start = widgets.VBox([self.b_fig_legend.canvas,
                                               self.scoreboard,
                                               self.softmax_temp_slider,
                                               self.start_button])
    else:
      self.legend_scores_start = widgets.VBox([self.b_fig_legend.canvas,
                                               self.scoreboard,
                                               self.start_button])
    if self.player0 == 'human':
      self.final_display = widgets.HBox([self.board_buttons_and_output0,
                                         self.legend_and_scores,
                                         self.board_and_output1])
    elif self.player1 == 'human':
      self.final_display = widgets.HBox([self.board_and_output0,
                                         self.legend_and_scores,
                                         self.board_buttons_and_output1])
    else: # no human player
      self.final_display = widgets.HBox([self.board_and_output0,
                                          self.legend_scores_start,
                                          self.board_and_output1])
    # initial text outputs
    # if there's a temp slider check who, if anyone uses it
    self.p0_uses_temp = False
    self.p1_uses_temp = False
    if self.has_temp_slider:
      if self.player0=='human':
        pass
      else:
        try:
          _ = self.player0.play(self.board_state, temp=1.0)
          self.p0_uses_temp = True
        except TypeError: pass
      if self.player1 == 'human':
        pass
      else:
        try:
          _ = self.player1.play(self.board_state, temp=1.0)
          self.p1_uses_temp = True
        except TypeError: pass
      if not self.p0_uses_temp and not self.p1_uses_temp:
        with self.output0:
          print("Warning: neither player supports temperature adjustment. "
                "The slider will have no effect.")
    with self.output0:
      if self.no_human == False:
        print('Click a button to start.')
      else:
        print('Click the start button to run the simulation')
    with self.scoreboard:
      print('Games Played: ' + str(len(self.final_scores)))
      table = [['', self.p0_short_name, self.p1_short_name],
          ['High Score:', '--', '--'],
          ['Last Score:', '--', '--'],
          ['Avg. Score:', '--', '--']]
      print(tabulate(table))

    # Connect the buttons to functions that do something
    self.up_button.on_click(self.on_up_button_clicked)
    self.down_button.on_click(self.on_down_button_clicked)
    self.left_button.on_click(self.on_left_button_clicked)
    self.right_button.on_click(self.on_right_button_clicked)
    self.start_button.on_click(self.on_start_button_clicked)


  def button_output_update(self, which_button):
    old_board = self.board_state.copy()
    old_scores = old_board['scores'][:,0] #both batches only one critter type
    self.active_player = old_board['active_player']
    self.disable_buttons()
    if self.player0 == 'human':
      a_player0 = which_button
    else:
      if self.p0_next_move is not None:
        a_player0_ = self.p0_next_move
        self.p0_next_move = None
      else:
        with self.output0:
          print("AI is thinking...")
        if self.p0_uses_temp:
          a_player0_, _, _ = self.player0.play(old_board,
                                               temp=self.sft_slider.value)
        else:
          a_player0_, _, _ = self.player0.play(old_board)
      a_player0_ = self.gwg.action_to_critter_direction(old_board,
                                                        self.active_player+1,
                                                        a_player0_)
      a_player0 = a_player0_[0]
    if self.player1 == 'human':
      a_player1 = which_button
    else:
      if self.p1_next_move is not None:
        a_player1_ = self.p1_next_move
        self.p1_next_move = None
      else:
        with self.output1:
          print("AI is thinking...")
        if self.p1_uses_temp:
          a_player1_, _, _ = self.player1.play(old_board,
                                               temp=self.sft_slider.value)
        else:
          a_player1_, _, _ = self.player1.play(old_board)
      a_player1_ = self.gwg.action_to_critter_direction(old_board,
                                                        self.active_player+1,
                                                        a_player1_)
      a_player1 = a_player1_[1]
    self.enable_buttons()

    self.board_state = self.gwg.critter_oriented_get_next_state(
        self.board_state, self.active_player+1, [a_player0, a_player1])

    # Try to precompute next AI player move(s) if there are any rounds left
    if self.board_state['rounds_left'][0] > 0:
      if self.player0 != 'human':
        if self.p0_uses_temp:
          self.p0_next_move, _, _ = self.player0.play(
            self.board_state, temp=self.sft_slider.value)
        else:
          self.p0_next_move, _, _ = self.player0.play(self.board_state)
      if self.player1 != 'human':
        if self.p1_uses_temp:
          self.p1_next_move, _, _ = self.player1.play(
            self.board_state, temp=self.sft_slider.value)
        else:
          self.p1_next_move, _, _ = self.player1.play(self.board_state)

    if self.collect_fov_data is True:
      batch_size, n_rows, n_cols = old_board['pieces'].shape
      b = GridworldBoard(batch_size, n_rows, n_cols,
                         self.gwg.num_food, self.gwg.lifetime,
                         rng=self.gwg.rng)
      b.set_state(old_board)
      percept = b.get_perceptions(self.radius)

    new_scores = self.board_state['scores'][:,0] #both batches one critter type
    rounds_left = self.board_state['rounds_left'][0]
    num_moves = self.gwg.lifetime - rounds_left

    if new_scores[0] > old_scores[0]:
      eating_string0 = "They ate the food there!"
    else:
      eating_string0 = "There's no food there."
    if new_scores[1] > old_scores[1]:
      eating_string1 = "They ate the food there!"
    else:
      eating_string1 = "There's no food there."
    did_eat = int(new_scores[0] > old_scores[0])

    row0, col0 = self.gwg.get_critter_rc(self.board_state, 0, 1)
    (self.b_fig0, self.b_ax0, self.b_crit_specs0, self.b_food0, self.b_fov0
     ) = self.gwg.plot_board(self.board_state, 0, self.b_fig0, self.b_ax0,
                             self.b_crit_specs0, self.b_food0, self.b_fov0,
                             has_fov=self.has_fov, radius=self.radius,
                             fov_opaque=self.fov_opaque,
                             legend_type=None)
    row1, col1 = self.gwg.get_critter_rc(self.board_state, 1, 1)
    (self.b_fig1, self.b_ax1, self.b_crit_specs1, self.b_food1, self.b_fov1
     ) = self.gwg.plot_board(self.board_state, 1, self.b_fig1, self.b_ax1,
                             self.b_crit_specs1, self.b_food1, self.b_fov1,
                             has_fov=self.has_fov, radius=self.radius,
                             fov_opaque=self.fov_opaque,
                             legend_type=None)

    with self.output0:
      clear_output()
      if self.player0 == 'human':
        print("You clicked the " + which_button +
              " button and your critter is now at ({}, {}).".format(row0,col0))
      else:
        print("This player (tried) to move " + a_player0 +
              " and is now at ({}, {}).".format(row0,col0))
      print(eating_string0)
      print("Rounds Left: {} \nFood Eaten: {} \nFood Per Move: {:.2f}".format(
          rounds_left, new_scores[0], new_scores[0] / num_moves))
    with self.output1:
      clear_output()
      if self.player1 == 'human':
        print("You clicked the " + which_button +
              " button and your critter is now at ({}, {}).".format(row1,col1))
      else:
        print("This player (tried) to move " + a_player1 +
              " and is now at ({}, {}).".format(row1,col1))
      print(eating_string1)
      print("Rounds Left: {} \nFood Eaten: {} \nFood Per Move: {:.2f}".format(
        rounds_left, new_scores[1], new_scores[1] / num_moves))

    if self.collect_fov_data is True:
      p_e_data = (percept.copy(), did_eat, old_board)
      self.percept_eat_records.append(p_e_data)
      percept_int = np.sum(percept==-1, axis=1)
      self.fov_eat_table_data[did_eat, percept_int] += 1

    if rounds_left == 0:
      self.final_scores.append(new_scores)
      self.board_state = self.gwg.get_init_board()
      self.board_state['pieces'][1] = self.board_state['pieces'][0].copy()
      (self.b_fig0, self.b_ax0, self.b_crit_specs0, self.b_food0, self.b_fov0
       ) = self.gwg.plot_board(self.board_state, 0, self.b_fig0, self.b_ax0,
                              self.b_crit_specs0, self.b_food0, self.b_fov0,
                              has_fov=self.has_fov, radius=self.radius,
                              fov_opaque=self.fov_opaque,
                              legend_type=None)
      (self.b_fig1, self.b_ax1, self.b_crit_specs1, self.b_food1, self.b_fov1
       ) = self.gwg.plot_board(self.board_state, 1, self.b_fig1, self.b_ax1,
                               self.b_crit_specs1, self.b_food1, self.b_fov1,
                               has_fov=self.has_fov, radius=self.radius,
                               fov_opaque=self.fov_opaque,
                               legend_type=None)
      with self.output0:
        clear_output
        print('Game Over. Final Score {}'.format(new_scores[0]))
        print('Resetting the board for another game')
      with self.output1:
        clear_output
        print('Game Over. Final Score {}'.format(new_scores[1]))
        print('Resetting the board for another game')
    with self.scoreboard:
      clear_output()
      self.b_fig_legend.canvas.draw()
      print('Games Played: ' + str(len(self.final_scores)))
      if len(self.final_scores) > 0:
        table = [['', self.p0_short_name, self.p1_short_name],
          ['High Score:', str(np.max(np.array(self.final_scores)[:,0])),
                          str(np.max(np.array(self.final_scores)[:,1]))],
          ['Last Score:', str(self.final_scores[-1][0]),
                          str(self.final_scores[-1][1])],
          ['Average Score',
            '{:.2f}'.format(np.mean(np.array(self.final_scores)[:,0])),
            '{:.2f}'.format(np.mean(np.array(self.final_scores)[:,1]))]]
      else:
        table = [['', self.p0_short_name, self.p1_short_name],
          ['High Score:', '--', '--'],
          ['Last Score:', '--', '--'],
          ['Average Score:', '--', '--']]
      print(tabulate(table))


  def on_up_button_clicked(self, *args):
    self.button_output_update('up')

  def on_down_button_clicked(self, *args):
    self.button_output_update('down')

  def on_left_button_clicked(self, *args):
    self.button_output_update('left')

  def on_right_button_clicked(self, *args):
    self.button_output_update('right')

  def on_start_button_clicked(self, *args):
    self.start_button.disabled = True
    if self.has_temp_slider:
      self.softmax_temp_slider.disabled = True
    for ii in range(self.gwg.lifetime):
      self.button_output_update('tbd')
      time.sleep(0.2)
    self.start_button.disabled = False
    if self.has_temp_slider:
      self.softmax_temp_slider.disabled = False

  def disable_buttons(self):
    self.up_button.disabled = True
    self.down_button.disabled = True
    self.left_button.disabled = True
    self.right_button.disabled = True

  def enable_buttons(self):
    self.up_button.disabled = False
    self.down_button.disabled = False
    self.left_button.disabled = False
    self.right_button.disabled = False




class ExploreWeightsWidget:
  def __init__(self, game):
    self.game = game
    self.n_rows, self.n_cols = 4, 12  # four directions, twelve fov cells
    self.row_labels = ['Up', 'Down', 'Left', 'Right']
    self.col_labels = [
      'Far<br>Up', 'Left<br>Up', 'Near<br>Up', 'Right<br>Up',
      'Far<br>Left', 'Near<br>Left', 'Near<br>Right', 'Far<br>Right',
      'Left<br>Down', 'Near<br>Down', 'Right<br>Down', 'Far<br>Down'
      ]

    # Create column headers
    col_header = [widgets.Label(value='', layout=widgets.Layout(width='50px'))] + \
                 [widgets.HTML(value=label, layout=widgets.Layout(width='60px', min_height='60px')) for label in self.col_labels]

    self.text_fields = [widgets.HBox(col_header)]

    for label in self.row_labels:
      row_fields = [widgets.FloatText(value=0.0, step=0.1, layout=widgets.Layout(width='60px'))
                    for _ in range(self.n_cols)]
      row_label = widgets.Label(value=f"{label}:", layout=widgets.Layout(width='50px'))
      self.text_fields.append(widgets.HBox([row_label] + row_fields))

    # Create a button to start the game
    self.run_button = widgets.Button(description="Run Game")
    self.run_button.on_click(self.run_game)

    # set up fig and create placeholders for vertical lines and histograms
    colors = sns.color_palette("colorblind")
    self.current_color = colors[0]
    self.best_color = colors[1]
    self.prev_color = colors[2]
    self.fig, self.ax = plt.subplots(figsize=(6,4))
    self.ax.set_xlim([0,25])
    self.ax.set_ylim([0,1])
    remove_ip_clutter(self.fig)
    self.current_avg_line = self.ax.axvline(-1, color=self.current_color,
                                            linestyle='dashed', linewidth=3,
                                            label='Current')
    self.prev_avg_line = self.ax.axvline(-1, color=self.prev_color,
                                         linestyle='dashed', linewidth=3,
                                         label='Previous')
    self.best_avg_line = self.ax.axvline(-1, color=self.best_color,
                                         linestyle='dashed', linewidth=3,
                                         label='Best')
    if self.game.batch_size > 1:
      #only do hist for batches
      self.current_hist_bars = self.ax.bar([0]*10, [0]*10,
                                           color=self.current_color,
                                           label='Current Run')
      self.prev_hist_bars = self.ax.bar([0]*10, [0]*10, color=self.prev_color,
                                        label='Previous Run', alpha=0.5)
    self.fig.legend(loc='outside right upper')
    self.fig.canvas.draw()

    # Output widget to display game output and any other information
    self.out = widgets.Output()

    # keep track of important values
    self.best_avg_score = float('-inf')
    self.best_params = None
    self.prev_scores = []
    self.scores = np.array([])

    # Button to set the best weights
    self.best_button = widgets.Button(description="Set Best Weights")
    self.best_button.on_click(self.set_best_weights)

    # Add a ToggleButton for symmetry
    self.symmetry_toggle = widgets.ToggleButton(value=False,
                                                description='Enforce Symmetry',
                                                disabled=False,
                                                button_style='',
                                                tooltip='Toggle symmetry enforcement',
                                                icon='check')
    self.symmetry_toggle.observe(self.toggle_symmetry, 'value')

    self.final_display = widgets.VBox([
      *self.text_fields,
      widgets.HBox([self.run_button, self.best_button, self.symmetry_toggle]),
      self.fig.canvas,
      self.out])

    self.links = []  # To keep track of the jslink objects


  def run_game(self, *args):
    weights = []
    for hbox in self.text_fields[1:]:  # Skip the header row
      row_weights = [field.value for field in hbox.children[1:]]  # Skip the label at the first position
      weights.append(row_weights)
    weights = np.array(weights)
    ppp = PerceptParamPlayer(self.game, weights=weights)
    # Run the game
    final_board = self.game.play_game(players=[ppp], visualize=False)

    self.scores = final_board['scores'].flatten()
    avg_score = np.mean(self.scores)

    if avg_score > self.best_avg_score:
      self.best_avg_score = avg_score
      self.best_params = weights

    if self.game.batch_size > 1:
      # Compute and update histogram data
      counts, edges = np.histogram(self.scores, bins=10)
      counts = counts/np.sum(counts)
      prev_counts, prev_edges = np.histogram(self.prev_scores[-1], bins=10) if len(self.prev_scores) > 0 else ([0]*10, edges)
      prev_sum = np.sum(prev_counts)
      if prev_sum > 0:
        prev_counts = prev_counts / np.sum(prev_counts)
      # Update the height of bars for the current scores
      for bar, height, left in zip(self.current_hist_bars, counts, edges[:-1]):
          bar.set_height(height)
          bar.set_x(left)
          bar.set_width(edges[1] - edges[0])
      # Update the height of bars for the previous scores
      for bar, height, left in zip(self.prev_hist_bars, prev_counts, prev_edges[:-1]):
          bar.set_height(height)
          bar.set_x(left)
          bar.set_width(prev_edges[1] - prev_edges[0])
    # set vline data
    self.current_avg_line.set_xdata([avg_score])
    if len(self.prev_scores) > 0:
        prev_avg_score = np.mean(self.prev_scores[-1])
        self.prev_avg_line.set_xdata([prev_avg_score])
    self.best_avg_line.set_xdata([self.best_avg_score])

    #update the fig
    self.fig.legend(loc='outside right upper')
    self.fig.canvas.draw()
    # Display the output
    with self.out:
      clear_output()

      if self.game.batch_size > 1:
        print(f"Average Score This Time: {avg_score}")
        if len(self.prev_scores) > 0:
          prev_avg_score = np.mean(self.prev_scores[-1])
          print(f"Average Score Last Time: {prev_avg_score}")
        print(f"Best Ever Average Score: {self.best_avg_score}")
      else:
        print(f"Score This Run: {avg_score}")
        if len(self.prev_scores) > 0:
          print(f"Score Last Run: {prev_avg_score}")
        print(f"Best Score: {self.best_avg_score}")

    self.prev_scores.append(self.scores.copy())


  def link_symmetric_widgets(self, change):
    # Each row's symmetry permutation indices
    symmetry_indices = {
      'Up':    [ 0,  1,  2,  1,  3,  4,  4,  3,  5,  6,  5,  7],
      'Down':  [ 7,  5,  6,  5,  3,  4,  4,  3,  1,  2,  1,  0],
      'Left':  [ 3,  1,  4,  5,  0,  2,  6,  7,  1,  4,  5,  3],
      'Right': [ 3,  5,  4,  1,  7,  6,  2,  0,  5,  4,  1,  3]}

    if change['new']:  # If the toggle button is activated
      base_row = self.text_fields[1].children[1:]  # The 'Up' row
      base_perm = np.array(symmetry_indices['Up'])  # Convert to a NumPy array for easier manipulation

      for row_label, hbox in zip(self.row_labels, self.text_fields[1:]):  # Include all rows
        link_row = hbox.children[1:]  # Skip the label
        perm = np.array(symmetry_indices[row_label])  # Convert to a NumPy array for easier manipulation
        for i, j in enumerate(perm):
          base_index = np.flatnonzero(base_perm == j)[0]
          if row_label != 'Up' or base_index != i:  # Skip self-links
            link = widgets.jslink((base_row[base_index], 'value'), (link_row[i], 'value'))
            self.links.append(link)


  def unlink_symmetric_widgets(self):
    for link in self.links:
      link.unlink()
    self.links.clear()


  def toggle_symmetry(self, change):
    if change.new:
      self.link_symmetric_widgets(change)
    else:
      self.unlink_symmetric_widgets()


  def set_best_weights(self, *args):
    if self.best_params is not None:
      for i, hbox in enumerate(self.text_fields[1:]):
        for j, field in enumerate(hbox.children[1:]):
          field.value = self.best_params[i][j]





class ComplexMoveInteractiveGridworld():
  """
  A widget based object for interacting with a gridworld game when more
  complicated (fast) moves are allowed
  """

  def __init__(self, gridworld_game, init_board=None, has_fov=False,
               radius=2, fov_opaque=False, collect_fov_data=False,
               figsize=(6,5), critter_names=['Critter'], players=['human']):
    """
    Initializes a widget based object for interacting with a gridworld game

    Args:
      gridworld_game: an instance of GridworldGame object
        expects this to have batchsize 1
      init_board: (optional) a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
        if left out will initialize with a random board state
      has_fov: bool, whether or not to display fog of war around the critter
      radius: int, number of squares the critter can "see" around it
      figsize: tuple (int, int), size of the figure
      critter_names: a list of strings that determines what the critter is called
        in the plot legend, order should align with players
      player: a list of either 'human', None, or a player object with a play
        method and a critter_index attribute. If 'human' use buttons,  if None
        default to making a RandomValidPlayer object, otherwise use the
        player class provided to make the player objects and use a start button.
        The list needs to be as long as the gridworld_game.num_critters
        attribute. Order should align with critter_name.

      Note: fov is going to look pretty janky with more than one player, maybe
      we get fov to only turn on for the 'active' player?
    """

    # Set GridworldGame object and initialize the board state
    self.gwg = gridworld_game
    self.has_fov = has_fov
    self.radius = radius
    self.fov_opaque = fov_opaque
    self.percept_len = 2*self.radius*(self.radius+1)
    self.collect_fov_data = collect_fov_data
    self.figsize = figsize
    # initialize players and plotting specs together to ensure alignment
    self.players = []
    self.any_human_players = False
    self.active_player_index = 0
    self.crit_specs = []
    markers = ['h', 'd']  # hexagon and diamond
    colors = sns.color_palette("colorblind")
    for i in range(self.gwg.num_critters):
      spec = {'marker': markers[i % len(markers)],
              'color': colors[i // len(markers) % len(colors)],
              'name': critter_names[i],
              'int_id': i+1}
      self.crit_specs.append(spec)
      player = players[i] #implict check that players is at least long enough
      if player is None:
        self.players.append(RandomValidPlayer(self.gwg, critter_index=i+1))
      elif player == 'human':
        self.players.append('human')
        # right now only ever have on human player with index 1
        self.any_human_players = True
      else:
        # player objects expected to have a critter_index attribute
        # we set it appropriately here so it aligns with the players list
        # used to create the widget
        player.critter_index = i+1
        self.players.append(player)
    self.final_scores = []
    if init_board is None:
      self.board_state = self.gwg.get_init_board()
    else:
      self.board_state = init_board
    if self.collect_fov_data is True:
      # keep raw records of percept and eating for manipulation later
      self.percept_eat_records = []
      # keep data in contingency table of how many food items were in
      # the percept, and whether or not food was eaten
      self.fov_eat_table_data = np.zeros((2, self.percept_len+1))
    # Initialize widgets and buttons
    self.output = widgets.Output(layout=widgets.Layout(
      width = '21.0em', min_width='21.0em', max_width='22.0em',
      min_height='10.0em', overflow='auto'))
    self.scoreboard = widgets.Output(layout=widgets.Layout(
      min_width='12.5em', max_width='18.8em',
      min_height='6.3em', overflow='auto'))
    self.fov_eat_table_display = widgets.Output(layout=widgets.Layout(
      min_width='25.0em', min_height='18.8em', overflow='auto'))
    self.up_button = widgets.Button(description="Up",
      layout=widgets.Layout(width='6.3em'))
    self.down_button = widgets.Button(description="Down",
      layout=widgets.Layout(width='6.3em'))
    self.left_button = widgets.Button(description="Left",
      layout=widgets.Layout(width='6.3em'))
    self.right_button = widgets.Button(description="Right",
      layout=widgets.Layout(width='6.3em'))
    self.move_options = ['Stay', 'Up', 'Down', 'Left', 'Right']
    self.move1_dropdown = widgets.Dropdown(
      options=self.move_options,
      value='Stay',
      description='First Move Part:')
    self.move2_dropdown = widgets.Dropdown(
      options=self.move_options,
      value='Stay',
      description='Second Move Part:')
    self.confirm_button = widgets.Button(description="Confirm Move")
    self.start_button = widgets.Button(description="Start",
      layout=widgets.Layout(width='6.3em'))
    self.game_running = False

    # get plot canvas widgets and other plotting objects
    plt.ioff()
    if self.collect_fov_data and self.any_human_players:
      self.legend_type = None # don't keep regenerating the legend
      # do legend separately if showing observations and no human player
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov,
       self.b_fig_legend, self.b_ax_legend) = self.gwg.plot_board(
          self.board_state, g=0, critter_specs=self.crit_specs,
          legend_type='separate', figsize=self.figsize, has_fov=self.has_fov,
          radius=self.radius, fov_opaque=self.fov_opaque,
          focal_critter_index=self.active_player_index)
    elif len(self.players) > 1:
      self.legend_type=None # don't keep regenerating the legend
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov,
       self.b_fig_legend, self.b_ax_legend) = self.gwg.plot_board(
          self.board_state, g=0, critter_specs=self.crit_specs,
          has_fov=self.has_fov, legend_type='separate',
          radius=self.radius, fov_opaque=self.fov_opaque, figsize=self.figsize,
          focal_critter_index=self.active_player_index)
    else:
      self.legend_type = 'included'
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov
        ) = self.gwg.plot_board(self.board_state, g=0,
                                critter_specs=self.crit_specs,
                                has_fov=self.has_fov,
                                fov_opaque=self.fov_opaque,
                                radius=self.radius, figsize=self.figsize,
                                focal_critter_index=self.active_player_index)
    # lump buttons together
    self.buttons = widgets.HBox([self.left_button,
                               widgets.VBox([self.up_button, self.down_button]),
                               self.right_button])
    # automatically pick different layouts for different situations
    if self.any_human_players:
      self.board_and_buttons = widgets.VBox([self.b_fig.canvas, self.buttons])
      if self.players[0] != 'human':
        # first player isn't human disable direction buttons
        self.disable_direction_buttons()

      if len(self.players) == 1:
        #one human player
        self.output_and_score = widgets.HBox([self.scoreboard, self.output])
        self.no_table_final_display = widgets.VBox([self.board_and_buttons,
                                                  self.output_and_score])
        if self.collect_fov_data == True:
          # a single human player collecting data
          self.final_display = widgets.HBox([self.no_table_final_display,
                                           self.fov_eat_table_display])
        else: # self.collect_fov_data == False:
          # a single human player not collecting data
          self.final_display = self.no_table_final_display
      else:
        # more than one player, at least one of them human
        self.V_board_outbput = widgets.VBox([self.board_and_buttons,
                                             self.output])
        self.V_scoreboard_start_legend = widgets.VBox([
        self.scoreboard, self.start_button, self.b_fig_legend.canvas])
        self.final_display = widgets.HBox([self.V_board_outbput,
                                             self.V_scoreboard_start_legend])
    else: # player is some kind of ai
      if self.collect_fov_data == True:
        # an ai player with recording
        # in this case legend is separate
        self.V_score_start_output_legend = widgets.VBox([self.scoreboard,
          self.start_button,  self.output, self.b_fig_legend.canvas])
        self.V_board_table = widgets.VBox([self.b_fig.canvas,
                                           self.fov_eat_table_display])
        self.final_display = widgets.HBox([self.V_board_table,
                                           self.V_score_start_output_legend])
      else:
        if len(self.players) == 1:
          # an ai player without recording
          self.H_score_output_start = widgets.HBox([
            self.scoreboard, self.output, self.start_button])
          self.final_display = widgets.VBox([
            self.b_fig.canvas, self.H_score_output_start])
        else:
          # more than one ai player
          self.V_board_outbput = widgets.VBox([self.b_fig.canvas, self.output])
          self.V_scoreboard_start_legend = widgets.VBox([
              self.scoreboard, self.start_button, self.b_fig_legend.canvas])
          self.final_display = widgets.HBox([self.V_board_outbput,
                                             self.V_scoreboard_start_legend])

    # initialize text outputs
    with self.scoreboard:
      table = [['High Score:'] + ['--'] * self.gwg.num_critters,
               ['Last Score:'] + ['--'] * self.gwg.num_critters,
               ['Average Score:'] + ['--'] * self.gwg.num_critters,]
      if len(self.players) > 1:
        headers = [''] + [f'P{i+1}' for i in range(self.gwg.num_critters)]
        print(tabulate(table, headers=headers))
      else: # len(self.players) == 1
        print(tabulate(table))
    with self.output:
      if self.any_human_players:
        print('Click a button to start playing')
      else:
        print('Click the start button to run the simulation')
    with self.fov_eat_table_display:
      printmd("**Observations**")
      table_data = [[str(ii),
                     str(self.fov_eat_table_data[0,ii]),
                     str(self.fov_eat_table_data[1,ii])] for ii in range(11)]
      table = ([['Food in Percept', 'Food Not Eaten', 'Food Eaten']] +
               table_data)
      print(tabulate(table))

    # Connect the buttons to functions that do something
    self.up_button.on_click(self.on_up_button_clicked)
    self.down_button.on_click(self.on_down_button_clicked)
    self.left_button.on_click(self.on_left_button_clicked)
    self.right_button.on_click(self.on_right_button_clicked)
    self.start_button.on_click(self.on_start_button_clicked)


  def button_output_update(self, which_button):
    old_board = self.board_state.copy()
    next_player_index = (self.active_player_index + 1) % len(self.players)
    # index of players is 0 through num_critter-1,
    # same player represented by value of index + 1 in
    old_scores = old_board['scores'][0]
    if self.collect_fov_data is True:
      batch_size, n_rows, n_cols = old_board['pieces'].shape
      b = GridworldBoard(batch_size, n_rows, n_cols,
                         self.gwg.num_food, self.gwg.lifetime,
                         rng=self.gwg.rng)
      b.set_state(old_board)
      percept = b.get_perceptions(self.radius)[0]

    direction = None
    if (isinstance(self.players[self.active_player_index], str) and
        'human' in self.players[self.active_player_index]):
      direction = which_button
      self.board_state = self.gwg.critter_oriented_get_next_state(
          self.board_state, self.active_player_index+1, [direction])
    else:
      a_player, _, _ = self.players[self.active_player_index].play(old_board)
      self.board_state = self.gwg.get_next_state(
          self.board_state, self.active_player_index+1, a_player)
      a_r = np.floor_divide(a_player[0], self.gwg.n_cols)
      a_c = np.remainder(a_player[0], self.gwg.n_cols)

    new_scores = self.board_state['scores'][0] #first batch first critter type
    rounds_left = self.board_state['rounds_left'][0]
    num_moves = np.floor(self.gwg.lifetime -
                         rounds_left / self.gwg.num_critters)
    if new_scores[self.active_player_index] > old_scores[self.active_player_index]:
      #eating happened
      eating_string = "They ate the food/prey there!"
      did_eat = 1
    else: #eating didn't happen
      eating_string = "There's no food/prey there."
      did_eat = 0
    row, col = self.gwg.get_critter_rc(self.board_state, 0,
                                       self.active_player_index+1)
    (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov
     ) = self.gwg.plot_board(self.board_state, g=0,
                             fig=self.b_fig, ax=self.b_ax,
                             critter_specs=self.b_crit_specs, food=self.b_food,
                             fov=self.b_fov, has_fov=self.has_fov,
                             fov_opaque=self.fov_opaque,
                             radius=self.radius, legend_type=self.legend_type,
                             focal_critter_index=next_player_index)
    if self.collect_fov_data is True:
      p_e_data = {'perception': percept.copy(),
                  'state': old_board,
                  'did_eat': bool(did_eat)}
      self.percept_eat_records.append(p_e_data)
      percept_int = np.sum(percept==-1) # number of food items in FoV
      self.fov_eat_table_data[did_eat, percept_int] += 1

    with self.output:
      clear_output()
      if len(self.players) == 1:
        if direction is not None:
          print("The critter (tried) to move " +
                direction +
                " and is now at ({}, {}).".format(row, col))
        else:
          print("The critter (tried) to move " +
                "to " + "({}, {})".format(a_r, a_c) +
                " and is now at ({}, {}).".format(row, col))
        print(eating_string)
        print("Rounds Left: {}\nScores: {:.2f}".format(
            rounds_left, new_scores[self.active_player_index]))
      else: # more than one players
        if direction is not None:
          print("Critter {} (tried) to move ".format(self.active_player_index+1) +
                direction +
                " and is now at ({}, {}).".format(row, col))
        else:
          print("Critter {} (tried) to move ".format(self.active_player_index+1) +
                "to " + "({}, {})".format(a_r, a_c) +
                " and is now at ({}, {}).".format(row, col))
        print(eating_string)
        print("Rounds Left: {}\nScores: {}".format(
            rounds_left, new_scores))
    if rounds_left == 0:
      self.final_scores.append(new_scores)
      self.game_running = False
      with self.output:
        clear_output
        print('Game Over. Final Score {}'.format(new_scores))
        print('Resetting the board for another game')
        self.board_state = self.gwg.get_init_board()
      (self.b_fig, self.b_ax, self.b_crit_specs, self.b_food, self.b_fov
       ) = self.gwg.plot_board(self.board_state, 0, self.b_fig, self.b_ax,
                               self.b_crit_specs, self.b_food, self.b_fov,
                               has_fov=self.has_fov, radius=self.radius,
                               fov_opaque=self.fov_opaque,
                               legend_type=self.legend_type,
                               focal_critter_index=next_player_index)
      # start_button can be clicked now
      self.start_button.disabled = False
    with self.scoreboard:
        clear_output()
        print('Games Played: ' + str(len(self.final_scores)))
        if len(self.players) == 1:
          if len(self.final_scores) > 0:
            table = [
              ['High Score:', str(np.max(np.array(self.final_scores)))],
              ['Last Score:', str(self.final_scores[-1])],
              ['Average Score',
              '{:.2f}'.format(np.mean(np.array(self.final_scores)))]]
          else:
            table = [['High Score:', '--'],
                     ['Last Score:', '--'],
                     ['Average Score:', '--']]
          print(tabulate(table))
        else: # len(self.players) > 1
          headers = [''] + [f'P{i+1}' for i in range(self.gwg.num_critters)]
          if len(self.final_scores) > 0:
            table = []
            # Assuming the batch size is 1 for now
            current_scores = self.final_scores[-1]
            max_scores = np.max(np.array(self.final_scores), axis=0)
            average_scores = np.mean(np.array(self.final_scores), axis=0)
            table.append(['High Scores:'] +
              [str(score) for score in max_scores])
            table.append(['Last Scores:'] +
              [str(score) for score in current_scores])
            table.append(['Average Scores:'] +
              ['{:.2f}'.format(score) for score in average_scores])
          else:
            table = [
              ['High Score:'] + ['--'] * self.gwg.num_critters,
              ['Last Score:'] + ['--'] * self.gwg.num_critters,
              ['Average Score:'] + ['--'] * self.gwg.num_critters,]
          print(tabulate(table, headers=headers))

    with self.fov_eat_table_display:
      clear_output()
      printmd("**Observations**")
      table_data = [[str(ii),
                     str(self.fov_eat_table_data[0,ii]),
                     str(self.fov_eat_table_data[1,ii])] for ii in range(11)]
      table = ([['Food in Percept', 'Food Not Eaten', 'Food Eaten']] +
               table_data)
      print(tabulate(table))

    # last thing after all display stuff is done
    self.update_player_and_rounds()

  def disable_direction_buttons(self):
    self.up_button.disabled = True
    self.down_button.disabled = True
    self.left_button.disabled = True
    self.right_button.disabled = True

  def enable_direction_buttons(self):
    self.up_button.disabled = False
    self.down_button.disabled = False
    self.left_button.disabled = False
    self.right_button.disabled = False

  def update_player_and_rounds(self):
    """Update the player index and decrement rounds if a full loop is completed."""
    self.active_player_index = (self.active_player_index + 1) % len(self.players)
    if self.active_player_index == len(self.players) - 1:
      self.board_state['rounds_left'] -= 1

  def human_ai_player_loop(self, direction):
    self.disable_direction_buttons()  # Disable buttons, no double clicks
    self.game_running = True
    # Execute the move of the human who clicked the button
    self.button_output_update(direction)
    # Move to the next player
    # Do AI moves if there are any
    while self.players[self.active_player_index] != 'human' and self.game_running:
      self.button_output_update('tbd')
      time.sleep(0.5)
    # Next player is human turn buttons on for them
    self.enable_direction_buttons()

  def start_game(self):
    # If first player is AI start button kicks off this loop
    self.game_running = True
    while self.players[self.active_player_index] != 'human' and self.game_running:
      self.button_output_update('tbd')
      time.sleep(0.5)
    # Next player is human, turn buttons on for them
    self.enable_direction_buttons()

  def on_up_button_clicked(self, *args):
    self.human_ai_player_loop('up')

  def on_down_button_clicked(self, *args):
    self.human_ai_player_loop('down')

  def on_left_button_clicked(self, *args):
    self.human_ai_player_loop('left')

  def on_right_button_clicked(self, *args):
    self.human_ai_player_loop('right')

  def on_start_button_clicked(self, *args):
    self.start_button.disabled = True
    self.start_game()
    self.enable_direction_buttons()

Random seed 2021 has been set.
This notebook isn't using and doesn't need a GPU. Good.
Running in colab


In [52]:
# @title Strike No Strike Game Logic
# The unerlying logic used by the evolution simulator and and the
# 'twiddle genome' widget

class StrikeNoStrikeEnv():

  def __init__(self, num_receptors=4, num_food_items=4, num_non_food_items=4,
               food_markers=None, non_food_markers=None, food_freq=0.5,
               food_items_dist=None, non_food_items_dist=None,
               reward=1, cost=1, rng=None, random_init=False):
    """
    Args:
      num_receptors (int): Number of receptors
      num_food_items (int): Number of food items
      num_non_food_items (int): Number of non-food items
      food_markers (np.ndarray): Food markers shape (num_food_items, num_receptors)
      non_food_markers (np.ndarray): Non-food markers shape (num_non_food_items, num_receptors)
      food_freq (float): Food frequency
      reward (float): reward for succesful strike
      cost (float): cost of strike at non food
    """
    if rng is None:
      self.rng = np.random.default_rng()
    else:
      self.rng = rng

    self.num_receptors = num_receptors

    # Create food markers
    if food_markers is not None:
      self.food_markers = food_markers
    else:
      first_half_food = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                        size=(num_food_items, num_receptors // 2),
                                        replace=True)
      second_half_food = np.ones((num_food_items, num_receptors - num_receptors // 2),
                                  dtype=np.bool_)
      food_markers = np.hstack([first_half_food, second_half_food])
    if non_food_markers is not None:
      self.non_food_markers = non_food_markers
    else:
      # Create non-food markers
      first_half_non_food = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                            size=(num_non_food_items, num_receptors // 2),
                                            replace=True)
      second_half_non_food = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                             size=(num_non_food_items, num_receptors // 2),
                                             replace=True)
      # Ensure second half is not all 1's
      while np.any(np.all(second_half_non_food, axis=1)):
        idx = np.where(np.all(second_half_non_food, axis=1))[0]
        second_half_non_food[idx] = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                                    size=(len(idx), num_receptors - num_receptors // 2),
                                                    replace=True)
        # keep checking condition because we set with random each time
      non_food_markers = np.hstack([first_half_non_food, second_half_non_food])

    # default items distribution is uniform, but can be otherwise
    if food_items_dist == 'dirichlet_sample':
      self.food_items_dist = self.rng.dirichlet(np.ones(len(self.food_markers)),size=1)[0]
    elif food_items_dist is not None:
      self.food_item_dist = food_items_dist
    else:
      self.food_item_dist = np.ones(num_food_items) / num_food_items

    if non_food_items_dist == 'dirichlet_sample':
      self.non_food_items_dist = self.rng.dirichlet(np.ones(len(self.non_food_markers)),size=1)[0]
    elif non_food_items_dist is not None:
      self.non_food_item_dist = non_food_items_dist
    else:
      self.non_food_item_dist = np.ones(num_non_food_items) / num_non_food_items

    # Initialize rewards, costs, and food frequency
    if random_init:
      self.reward = self.rng.integers(1, 10)
      self.cost = self.rng.integers(1, 10)
      self.food_freq = np.round(self.rng.uniform(0.01, 0.99), 2)

  def compute_score(self, genotype,  n_trials=100):
    tp, fp = 0, 0

    n_food_trials = self.rng.binomial(n=n_trials, p=self.food_freq)
    n_non_food_trials = n_trials - n_food_trials
    food_indices = self.rng.choice(len(self.food_markers),
                                   size=n_food_trials,
                                   p=self.food_item_dist)
    non_food_indices = self.rng.choice(len(self.non_food_markers),
                                       size=n_non_food_trials,
                                       p=self.non_food_item_dist)

    food_items = self.food_markers[food_indices]
    non_food_items = self.non_food_markers[non_food_indices]

    # Using broadcasting to compute number of strikes in each case
    # this happens when genotype lines up with food marker for all
    # spots where genotype specifies a receptor
    tp = np.sum(np.all((genotype & food_items)[genotype], axis=1))
    fp = np.sum(np.all((genotype & non_food_items)[genotype], axis=1))

    return tp * self.reward - fp * self.cost




In [63]:
# @title Genome Twidler Widget

##########
# Define GenomeTwiddle locally before integrating into utils
##########


class GenomeTwiddle:
  def __init__(self, seed=None, show_env=True, num_receptors=4,
               num_food_types=4, num_non_food_types=4):
    self.rng = np.random.default_rng(seed)
    self.show_env = show_env
    self.num_receptors = num_receptors
    step_for_freq_food = 0.01

    if show_env and num_receptors==4:
      self.food_markers = np.array([[1, 1, 1, 1],
                                    [1, 0, 1, 1],
                                    [0, 0, 1, 1],
                                    [0, 1, 1, 1]],
                                   dtype=np.bool_)
      self.non_food_markers = np.array([[0, 0, 1, 0],
                                        [1, 0, 0, 1],
                                        [0, 1, 0, 1],
                                        [0, 0, 1, 0]],
                                       dtype=np.bool_)
      self.food_item_dist = np.ones(len(self.food_markers)) / len(self.food_markers)
      self.non_food_item_dist = np.ones(len(self.non_food_markers)) / len(self.non_food_markers)
    else:
      first_half_food = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                        size=(num_food_types, num_receptors // 2),
                                        replace=True)
      second_half_food = np.ones((num_food_types, num_receptors - num_receptors // 2),
                                  dtype=np.bool_)
      self.food_markers = np.hstack([first_half_food, second_half_food])
      self.food_item_dist = self.rng.dirichlet(np.ones(len(self.food_markers)),size=1)[0]

      first_half_non_food = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                            size=(num_non_food_types, num_receptors // 2),
                                            replace=True)
      second_half_non_food = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                             size=(num_non_food_types, num_receptors // 2),
                                             replace=True)
      # Ensure second half is not all 1's
      while np.any(np.all(second_half_non_food, axis=1)):
        idx = np.where(np.all(second_half_non_food, axis=1))[0]
        second_half_non_food[idx] = self.rng.choice(np.array([0, 1],dtype=np.bool_),
                                                    size=(len(idx), num_receptors - num_receptors // 2),
                                                    replace=True)
        # keep checking condition because we set with random each time
      self.non_food_markers = np.hstack([first_half_non_food, second_half_non_food])
      self.non_food_item_dist = self.rng.dirichlet(np.ones(len(self.non_food_markers)),size=1)[0]

    self.genotype_buttons = widgets.HBox([widgets.RadioButtons(
        options=['0', '1'], description=f'Receptor {i+1}:',) for i in range(num_receptors)])

    if show_env is True:
      default_r = 1
      default_c = 1
      default_ff = 0.5
    else:
      default_r = self.rng.integers(1, 10)
      default_c = self.rng.integers(1, 10)
      default_ff = np.round(self.rng.uniform(0.01, 0.99) / step_for_freq_food) * step_for_freq_food

    self.freq_food = widgets.FloatSlider(
        value=default_ff, min=0, max=1.0, step=0.01,
        description='Food Freq:',)
    self.reward_r = widgets.IntSlider(
        value=default_r, min=0, max=10, description='Reward (r):',)
    self.cost_c = widgets.IntSlider(
        value=default_c, min=0, max=10, description='Cost (c):',)

    self.food_marker_output = widgets.Output()
    self.non_food_marker_output = widgets.Output()
    food_markers_with_labels = [[f'Food Item {i+1}'] + list(marker) for i, marker in enumerate(self.food_markers)]
    non_food_markers_with_labels = [[f'Non-Food Item {i+1}'] + list(marker) for i, marker in enumerate(self.non_food_markers)]
    headers = [''] + [f'R{i+1}' for i in range(self.num_receptors)]

    with self.food_marker_output:
      print(f"Food Markers:\n{tabulate(food_markers_with_labels, headers=headers, tablefmt='grid')}")
    with self.non_food_marker_output:
      print(f"Non-Food Markers:\n{tabulate(non_food_markers_with_labels, headers=headers, tablefmt='grid')}")

    self.marker_display = widgets.HBox([self.food_marker_output, self.non_food_marker_output])
    self.output = widgets.Output()
    self.compute_button = widgets.Button(description='Compute', disabled=False,
                                         button_style='',)
    self.compute_button.on_click(self.compute_trials)
    self.sliders = widgets.VBox([widgets.Label('Parameters:'),
        widgets.HBox([self.freq_food, self.reward_r, self.cost_c])])
    self.compute_result = widgets.VBox([ widgets.Label('Result:'),
                                        self.compute_button,
                                         self.output])
    # Conditional display
    if show_env:
      self.widget_box = widgets.VBox([self.genotype_buttons,
                                      self.marker_display,
                                      self.sliders,
                                      self.compute_result])
    else:
      self.widget_box = widgets.VBox([self.genotype_buttons,
                                      self.compute_result])

  def display(self):
    display(self.widget_box)

  def compute_trials(self, button):
    with self.output:
      self.output.clear_output()
      n_trials = 100

      genotype = np.array(
          [int(button.value) for button in self.genotype_buttons.children],
          dtype=np.bool_)

      n_food_trials = self.rng.binomial(n=n_trials, p=self.freq_food.value)
      n_non_food_trials = n_trials - n_food_trials
      food_indices = self.rng.choice(len(self.food_markers),
                                     size=n_food_trials,
                                     p=self.food_item_dist)
      non_food_indices = self.rng.choice(len(self.non_food_markers),
                                         size=n_non_food_trials,
                                         p=self.non_food_item_dist)

      food_items = self.food_markers[food_indices]
      non_food_items = self.non_food_markers[non_food_indices]

      # Using broadcasting to compute number of strikes in each case
      # this happens when genotype lines up with food marker for all
      # spots where genotype specifies a receptor

      tp = np.sum(np.all((genotype & food_items)[:, genotype], axis=1))
      fp = np.sum(np.all((genotype & non_food_items)[:, genotype], axis=1))
      fn = n_food_trials - tp
      tn = n_non_food_trials - fp

      # Using broadcasting to compute number of strikes in each case
      # this happens when genotype lines up with food marker for all
      score = tp * self.reward_r.value - fp * self.cost_c.value
      if self.show_env:
        contingency_table = [["", "Strike", "No-Strike"],
                             ["Food", tp, fn],
                             ["Non-Food", fp, tn]]
        print(f"Contingency Table:\n{tabulate(contingency_table, headers='firstrow', tablefmt='grid')}")
      print(f"Score: {score}")


# 1.3.1.1: A Bite-Sized Evolution Example

We're going to explore the fundamentals of evolution using the following simple scenario. Drawing inspiration from the sand-striker worm (Eunice aphroditois), imagine an organism that lurks in its burrow, with just its chemosensory antennae sticking out. Whenever food that 'smells good' (as determined by its antennae receptors) passes by, the worm lunges from its lair to snatch the morsel, incapacitate it if need be, and drag it back home for a feast. Yum! We model this organism with a simplified 'genome'—an array of bits where each bit codes for the presence (1) or absence (0) of a distinct chemosensory receptor on the worm's antennae.

The organism's life centers around one core problem: to 'strike' or 'not strike'. A successful strike at edible food yields a reward, $r$. While the immediate benefits of this come from fats, proteins, and calories, and the impact of these resources on survival and reproduction fluctuates based on the worm's condition, we simplify by saying each successful capture increases the number of viable gametes by $r$, thereby affecting the chances of the organisms offspring being in the next generation. Conversely, a missed strike costs $c$. While these immediate costs manifest as increased predation risk or lost opportunity, which may have complex impacts on survival and reproduction depending on the worm's condition, we again simplify this cost as a constant gamete decrement by $c$.

The environment our organism finds itself in, in this first minimal example, consists of a series of encounters with several different kinds of food and non-food items. Each kind of item is tagged with distinct chemical markers, akin to different 'scents' or 'flavors'. Note that these markers will sometimes partially (but never totally) overlap between items regardless of food / non-food categorization, depending on the particular details of the environment, creating potential for misjudgment. Our organism's chemosensory antennae, coded by its genome, are sensitive to these markers. In each of these encounters, the organism faces its core 'strike' or 'not strike' problem, which it 'solves' with the following decision rule: the worm strikes only if all of its receptors pick up a marker and refrains from striking if any of its receptors are left unactivated, (Note that this means it will always strike if it has no receptors). In the simulation below see if you can find the optimal 'genome' for an organism.  To start you can see the 'markers' on the different food and non-food items.

In [64]:
# @title Twiddle the Genes
# @markdown Run this cell to tweak the genome. Click the compute button to see how the organism fares over 100 random trials
gt = GenomeTwiddle(show_env=True, num_receptors=4)
gt.display()


So already in this very simple example some interesting things are happening. In our example, by construction, each location on the genome can be thought of as a 'gene' for a particular receptor with two possible alleles, presence or absense of the corresponding chemosenory receptor on an antenna. However, we might also zoom out our analysis and look at traits at the behavioural level, e.g. we say that 'Striking at Food Item 2' (with markers [1, 0, 1]) is a trait, then we find that either the gene for receptor 1 or the gene for receptor 2 might equally be considered a 'Gene for Striking at Food Item 2'. In a different environment, where the markers for food and non-food items are slightly different—for example, if 'Food Item 2' has markers [1, 0, 0]—now only the gene for receptor 1 might be considered the 'Gene for Striking at Food Item 2'. This hints at the potential complexity of gene-behavior mapping (through ontogeny and mechanisms) but also emphasizes how this mapping is strongly context-dependent, driven by the specifics of the environment.

In spite of this complexity, the evolutionary process is still able to shape and select for complext traits, as we shall soon see. When you were twiddling the genes in the exercise above, you had the benefit of know what the markers were for the food and non-food items, the costs and rewards of different outcomes, and the relative frequencies of food versus non-food items appearing. The evolutionary process is effectively blind to all of this, the problem that evolution faces is more like the following exercise.

In [4]:
# @title Twiddle the Genes in Ignorance
# @markdown Run this cell to tweak the genome. Click the compute button to see how the organism fares over 100 random trials
gt = GenomeTwiddle(show_env=False)
gt.display()


So with just three receptors this problem isn't so difficult. It's relatively easy to test combinations and see what works well and what doesn't, even with the highly stochastic feedback. But what about when there are more receptors?

In [5]:
# @title An Overwheliming Number of Gene's and Receptors
# @markdown Run this cell to tweak the genome. Click the compute button to see how the organism fares over 100 random trials
gt = GenomeTwiddle(show_env=False, num_receptors=10)
gt.display()

The curse of dimensionality strikes again.

In [6]:
# @markdown Submit your feedback
content_review(f"{feedback_prefix}_M1")

# 1.3.1.2: Evolution acts on Populations

One of the ways that evolution is able to search high-dimensional parameters spaces (big genomes) despite the curse of dimensionality is because it is massively parrellel. Many variants (possibly as many as there are individiuals in a population) are evaluated in parrellel within each generation.

In a very [coarse first approximation](# "This is a good first approximiation but completely fails to address many critical aspects of evoltuion, in particular the emergence of new genetic loci."), evolution can be thought of as allele frequency changes within a population. In this simplified framing evolution can be seen as the aggregate affects at the population level of selection acting on individuals. Populations evolve, individuals are selected. To see how evolution can solve this problem, even when the search space is large and opaque, we will simulate a population of organisms, apply the 'environmental seive' by having those individuals with the highest score in the 'strike' or 'no-strike' game be the ones most likely to have their offspring in the subsequent generation. In this first simplified simulation we will work with asexual haploid organisms. This is only to ease simulation and thinking about the model though. Note that sex--and other forms of gene transfer-- together with multiple chromosomes are the general rule in life. In the next sequences we will look at how sex (and re-combination enabled by diplody) radically amplifies the effectiveness of pure brute force parrellelism.

In [44]:
# @markdown Run this cell to see brute force parrellelism of evolution find good solution(s) to a 3-receptor/marker organism/environment system.

def compute_score(genotype, food_markers, non_food_markers,
                  reward_r, cost_c, food_freq, rng=None, n_trials=100):
    tp, fp = 0, 0
    if rng is None:
        rng = np.random.default_rng()

    n_food_trials = rng.binomial(n=n_trials, p=food_freq)
    n_non_food_trials = n_trials - n_food_trials
    food_indices = rng.choice(len(food_markers), size=n_food_trials)
    non_food_indices = rng.choice(len(non_food_markers), size=n_non_food_trials)

    food_items = food_markers[food_indices]
    non_food_items = non_food_markers[non_food_indices]

    # Using broadcasting to compute number of strikes in each case
    tp = np.sum(np.any(genotype & food_items, axis=1))
    fp = np.sum(np.any(genotype & non_food_items, axis=1))

    return tp * reward_r - fp * cost_c


def evolutionary_simulation(population_size=1000, num_generations=100,
                            mutation_rate=0.01, num_receptors=3,
                            reward_r=1, cost_c=1, food_freq=0.5,
                            temperature=5.0, has_mutation=False,
                            random_init=True, seed=None):

    rng = np.random.default_rng(seed)
    # Initialize population
    if random_init:
        population = rng.integers(0, 2, size=(population_size, num_receptors),
                                  dtype=np.bool_)
    elif num_receptors == 3 and population_size == 400 and not random_init:
        population = np.array([[1, 1, 1],
                               [1, 0, 1],
                               [0, 0, 1],
                               [0, 0, 0]],
                              dtype=np.bool_)
        population = np.repeat(population, 100, axis=0)
    else:
        population = rng.integers(0, 2, size=(population_size, num_receptors),
                                  dtype=np.bool_)

    # Initialize environment
    if num_receptors == 3 and not random_init:
        food_markers = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 0], [0, 1, 0]],
                                dtype=np.bool_)
        non_food_markers = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 1], [0, 0, 1]],
                                    dtype=np.bool_)
    else:
        if num_food_items is None:
            num_food_items = 2 ** (num_receptors // 2)

        if num_non_food_items is None:
            num_non_food_items = 2 ** (num_receptors // 2)

        # Create food markers
        first_half_food = rng.choice(np.array([0, 1],dtype=np.bool_),
                                     size=(num_food_items, num_receptors // 2),
                                     replace=True)
        second_half_food = np.ones((num_food_items, num_receptors - num_receptors // 2),
                                   dtype=np.bool_)
        food_markers = np.hstack([first_half_food, second_half_food])
        # Create non-food markers
        first_half_non_food = rng.choice(np.array([0, 1],dtype=np.bool_),
                                         size=(num_non_food_items, num_receptors // 2),
                                         replace=True)
        second_half_non_food = rng.choice(np.array([0, 1],dtype=np.bool_),
                                         size=(num_non_food_items, num_receptors // 2),
                                         replace=True)
        # Ensure second half is not all 1's
        while np.any(np.all(second_half_non_food, axis=1)):
            idx = np.where(np.all(second_half_non_food, axis=1))[0]
            second_half_non_food[idx] = rng.choice(np.array([0, 1],dtype=np.bool_),,
                                                   size=(len(idx), num_receptors - num_receptors // 2),
                                                   replace=True)
            # keep checking condition because we set with random each time
        non_food_markers = np.hstack([first_half_non_food, second_half_non_food])

    # Initialize rewards, costs, and food frequency
    if random_init:
        reward_r = rng.integers(1, 10)
        cost_c = rng.integers(1, 10)
        food_freq = np.round(rng.uniform(0.01, 0.99), 2)

    print(f"Food markers: {np.array(food_markers,dtype=int)}")
    print(f"Non-food markers: {np.array(non_food_markers, dtype=int)}")
    print(f"Reward: {reward_r}")
    print(f"Cost: {cost_c}")
    print(f"Food frequency: {food_freq}")
    print("---------------------------------------")

    # Run simulation
    for generation in range(num_generations):
        # Compute scores for each individual
        scores = [compute_score(geno, food_markers, non_food_markers,
                                reward_r, cost_c, food_freq, rng) for geno in population]

        # Normalize scores to probabilities with temperature adjustment
        exp_scores = np.exp(np.array(scores) / temperature)
        prob_scores = exp_scores / np.sum(exp_scores)

        # Count and display population makeup
        pop_counter = collections.Counter([tuple(np.array(ind, dtype=int))
                                            for ind in population])

        # Calculate the average score for each genotype
        genotype_to_scores = collections.defaultdict(list)
        for ind, score in zip(population, scores):
            genotype_str = tuple(np.array(ind, dtype=int))
            genotype_to_scores[genotype_str].append(score)

        avg_scores_by_type = {geno: np.mean(scs) for geno, scs in genotype_to_scores.items()}

        print(f"Generation {generation+1}: Mean Score = {np.mean(scores)}")
        print(f"Population makeup: {pop_counter}")
        print(f'Average scores by type: {avg_scores_by_type}')
        print('')

        if len(pop_counter) == 1:
            print(f"Population has converged to a single genotype in generation {generation + 1}. Stopping simulation.")
            break

        # Resample population based on scores
        population = population[
            np.random.choice(np.arange(population_size),
                             size=population_size, p=prob_scores)]

        # mutation applied to next generation using XOR
        # to introduce new variants
        if has_mutation:
          mutation_mask = rng.random(population.shape) < mutation_rate
          population ^= mutation_mask

# Example usage
evolutionary_simulation(temperature=5.0, random_init=False, num_generations=25,
                        has_mutation=False)



Food markers: [[1 1 1]
 [1 0 1]
 [1 1 0]
 [0 1 0]]
Non-food markers: [[0 0 0]
 [1 0 0]
 [0 1 1]
 [0 0 1]]
Reward: 1
Cost: 1
Food frequency: 0.5
---------------------------------------
Generation 1: Mean Score = 13.807
Population makeup: Counter({(1, 1, 1): 140, (1, 1, 0): 134, (0, 0, 0): 132, (1, 0, 1): 127, (0, 1, 0): 126, (0, 0, 1): 121, (1, 0, 0): 116, (0, 1, 1): 104})
Average scores by type: {(1, 0, 1): -0.4251968503937008, (0, 0, 1): 0.8925619834710744, (1, 1, 1): 14.014285714285714, (0, 1, 1): 24.21153846153846, (0, 1, 0): 24.76984126984127, (0, 0, 0): 0.0, (1, 1, 0): 24.559701492537314, (1, 0, 0): 24.663793103448278}

Generation 2: Mean Score = 23.997
Population makeup: Counter({(1, 1, 0): 364, (0, 1, 0): 210, (0, 1, 1): 181, (1, 0, 0): 177, (1, 1, 1): 62, (1, 0, 1): 5, (0, 0, 1): 1})
Average scores by type: {(1, 1, 0): 24.96153846153846, (1, 0, 0): 24.56497175141243, (1, 1, 1): 12.806451612903226, (1, 0, 1): -0.4, (0, 1, 0): 24.685714285714287, (0, 1, 1): 25.298342541436465, (0

So the above worked great, there we're eight different types to evaluate, we had lot's of copies of each type, and our selective process quickly zeroed in on the 'best' two types, [1 1 0] and [0 1 1]. We had a population size of 1000 in this case. But what can we do when the population size is smaller than the number of possible types? If there are 12 receptors then there are will be roughly four times as many possible genotype variants ($4096 = 2^{12}$) as there were individuals in this simulated population. The process above will select good variants from the existing population, but will never introduce new variants to the population, so if there are no good variants in the intial population, the process will not discover a good solution. Run the cell to see this particular failing in action.

In [45]:
# @markdown Run this cell to see how our evolutionary simulation fails to discover good solutions that aren't in the initial population.
evolutionary_simulation(temperature=5.0, random_init=False, population_size=400)


Food markers: [[1 1 1]
 [1 0 1]
 [1 1 0]
 [0 1 0]]
Non-food markers: [[0 0 0]
 [1 0 0]
 [0 1 1]
 [0 0 1]]
Reward: 1
Cost: 1
Food frequency: 0.5
---------------------------------------
Generation 1: Mean Score = 2.4925
Population makeup: Counter({(1, 1, 1): 100, (1, 0, 1): 100, (0, 0, 1): 100, (0, 0, 0): 100})
Average scores by type: {(1, 1, 1): 11.65, (1, 0, 1): -1.56, (0, 0, 1): -0.12, (0, 0, 0): 0.0}

Generation 2: Mean Score = 10.595
Population makeup: Counter({(1, 1, 1): 355, (1, 0, 1): 27, (0, 0, 1): 15, (0, 0, 0): 3})
Average scores by type: {(1, 1, 1): 11.983098591549295, (0, 0, 1): 1.1333333333333333, (1, 0, 1): -1.2222222222222223, (0, 0, 0): 0.0}

Generation 3: Mean Score = 12.555
Population makeup: Counter({(1, 1, 1): 399, (1, 0, 1): 1})
Average scores by type: {(1, 1, 1): 12.593984962406015, (1, 0, 1): -3.0}

Generation 4: Mean Score = 12.1875
Population makeup: Counter({(1, 1, 1): 400})
Average scores by type: {(1, 1, 1): 12.1875}

Population has converged to a single geno

What our evolutionary simulation is missing is a way of introducing variation into the population. We have selection, but selection can only ever reduce the number of distinct varients in the population, never introduce novel variants. This is where mutation comes in. Mutations (and recombination events in more complext replication processes) are necissary to create the heritable variation that is selected for in the first place. Though note that if mutation rates are too high variation become less heritable. So some mutation, but not too much is critical to an effective (in the sense of finding phenotypes that are well suited to the environment) evolutionary process. Let's add mutation to our evolutionary simulation and see if that helps in the case above where a small population does not start with a good solution.

In [46]:
evolutionary_simulation(temperature=5.0, random_init=False, population_size=400,
                        has_mutation=True)




Food markers: [[1 1 1]
 [1 0 1]
 [1 1 0]
 [0 1 0]]
Non-food markers: [[0 0 0]
 [1 0 0]
 [0 1 1]
 [0 0 1]]
Reward: 1
Cost: 1
Food frequency: 0.5
---------------------------------------
Generation 1: Mean Score = 3.29
Population makeup: Counter({(1, 1, 1): 100, (1, 0, 1): 100, (0, 0, 1): 100, (0, 0, 0): 100})
Average scores by type: {(1, 1, 1): 12.29, (1, 0, 1): 0.68, (0, 0, 1): 0.19, (0, 0, 0): 0.0}

Generation 2: Mean Score = 9.735
Population makeup: Counter({(1, 1, 1): 282, (1, 0, 1): 91, (0, 0, 1): 10, (0, 0, 0): 8, (1, 1, 0): 4, (0, 1, 1): 4, (1, 0, 0): 1})
Average scores by type: {(1, 1, 1): 13.070921985815604, (1, 0, 1): 0.24175824175824176, (1, 1, 0): 18.75, (0, 0, 1): -2.3, (0, 1, 1): 28.5, (0, 0, 0): 0.0, (1, 0, 0): 20.0}

Generation 3: Mean Score = 14.2775
Population makeup: Counter({(1, 1, 1): 336, (1, 1, 0): 28, (0, 1, 1): 26, (1, 0, 1): 10})
Average scores by type: {(1, 1, 1): 12.833333333333334, (1, 0, 1): -0.9, (1, 1, 0): 24.928571428571427, (0, 1, 1): 27.307692307692307}

Mutation introduces the effective [1, 1, 0] and [0, 1, 1], and then selection makes it so that (at least one of) these variants comes to dominate the population. Note that most of the other variants that persist in the population are a single mutation step away from the dominant variant(s).

So adding mutation allowed this evolutionary process to find a good solution even when there was not a good solution in the initial population. Let's see now if we are able to search a pretty big solution space, i.e. one where there are 12 receptors so, 4096 possible variants to try.

In [47]:
evolutionary_simulation(num_receptors=12, temperature=5.0, random_init=False,
                        population_size=1000, has_mutation=True)


Food markers: [[0 1 1 ... 1 1 1]
 [0 0 1 ... 0 1 0]
 [0 0 1 ... 1 0 1]
 ...
 [0 1 1 ... 0 0 1]
 [1 0 0 ... 1 0 0]
 [0 1 0 ... 0 1 1]]
Non-food markers: [[1 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [1 1 1 ... 1 1 0]
 ...
 [1 0 0 ... 0 1 1]
 [1 0 1 ... 1 0 0]
 [1 1 0 ... 1 1 0]]
Reward: 1
Cost: 1
Food frequency: 0.5
---------------------------------------
Generation 1: Mean Score = -0.142
Population makeup: Counter({(0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1): 4, (1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1): 3, (0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0): 3, (1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1): 3, (1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1): 3, (0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0): 3, (1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0): 3, (0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0): 3, (0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0): 3, (1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1): 3, (1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1): 3, (0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1): 3, (0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1): 3, (1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0): 3, (0, 0, 0, 0, 1

In [ ]:
np.random.randint(0, 2, size=(population_size, num_receptors))

Before starting on policy optimization, we first need to extend and generalize our parameterized policy player from earlier sequences.
Specifically, instead of solely perceiving food, a policy should now also respond to the presence of other organisms and the board's edges, augmenting the initial food-only perception. Moreover, to allow for diverse policies to emerge, we're enhancing the prey's speed, allowing movement across two grid cells. This means the prey can target any of the 12 nearest cells, as opposed to just the 4 adjacent ones, effectively reaching any grid cell within its sight. Likewise, the predator's perceptive field is expanded, now encompassing 24 surrounding grid cells—effectively covering half the board when centered. Previously we didn't allow for organisms to stay in one spot, but now this will also be enabled for both prey and predator types. Below, we define a general class for these players (feel free to skim). Both the prey and predator organisms will be constructed from this foundational class using suitable parameters.

In [ ]:
class GeneralLinearPlayer():
  """
  Represents a player in a grid-world game with a linear policy based on given weights.
  The player's perception and actions are determined by its parameters.
  """

  def __init__(self, game, critter_index=1, weights=None, fov_radius=1, speed=1,
               has_food_percept=True, has_edge_percept=False,
               has_prey_percept=False, has_pred_percept=False,
               get_probs=False, deterministic=False):
    """
    Initialize the player.

    Args:
      game (GridworldGame instance): The game the player is part of.
      critter_index (int): Determines the role (prey or predator) of the player.
      ... (Other parameters are self-explanatory)
    """
    # Validate critter index
    if not (isinstance(critter_index, int) and 0 < critter_index <= game.num_critters):
      raise ValueError("Invalid critter index value. Ensure it's a positive integer within game limits.")

    # Essential attributes
    self.game = game
    self.critter_index = critter_index
    self.get_probs = get_probs

    # Player-specific attributes
    self.deterministic = deterministic
    self.fov_radius = fov_radius
    self.speed = speed
    self.W_out_shape = 2 * self.speed**2 + 2 * self.speed + 1
    self.W_in_shape = 2 * self.fov_radius**2 + 2 * self.fov_radius + 1
    self.has_food_percept = has_food_percept
    self.has_edge_percept = has_edge_percept
    self.has_prey_percept = has_prey_percept
    self.has_pred_percept = has_pred_percept
    self.W_layers = sum([has_food_percept, has_edge_percept, has_prey_percept, has_pred_percept])
    self.W_shape = (self.W_layers, self.W_out_shape, self.W_in_shape)
    self.W = weights if weights is not None else np.ones(self.W_shape)
    if self.W.shape != self.W_shape:
      raise ValueError("Weights shape mismatch. Check fov, speed, and percepts.")
    self.default_softmax_temp = 0.05

  def direction_value_from_percept(self, percepts, W):
    """
    Calculate action values based on player's perception.

    Args:
      percepts (np.array): Surroundings the player perceives.
      W (np.array): Connection strengths between perceptions and actions.

    Returns:
      np.array: Raw output activations for possible actions.
    """
    expanded_percepts = []
    # Check if the player can perceive each type and append the perception
    if self.has_food_percept:
      x_food = np.asarray((percepts <= -1) & (percepts > -200), float) # batch x len W_in
      expanded_percepts.append(x_food)
    if self.has_edge_percept:
      x_edge = np.asarray(percepts == -200, float) # batch x len W_in
      expanded_percepts.append(x_edge)
    if self.has_prey_percept:
      x_prey = np.asarray((percepts > 0) &
       (percepts <= self.game.pred_prey_threshold), float) # batch x len W_in
      expanded_percepts.append(x_prey)
    if self.has_pred_percept:
      x_pred = np.asarray(percepts > self.game.pred_prey_threshold, float) # batch x len W_in
      expanded_percepts.append(x_pred)
    # Stack all perceptions together in an array
    percept_stack = np.stack(expanded_percepts) # W_layers x batch x len_W_in

    # `np.tensordot` computes weighted sums of perceptions,
    # determining the influence of each perception on potential actions.
    # It efficiently does this as a tensor product between the perception stack
    # and the weights, across all the batches.
    output_activations = np.tensordot(percept_stack, self.W, [[0, 2], [0, 2]]) # batch x W_out
    # `np.tensordot` is packing a lot under the hood. Specifically it is doing
    # the following far more efficiently:
    # output_activations = np.zeros((self.game.batch_size, self.W_out))
    # for b in range(self.game.batch_size): #each batch
    #   for l in range(self.W_layers):      #each percept type
    #     for o in range(self.W_out):       #each output direction
    #       sum_val = 0
    #       for i in range(W_in):           #each value in the percept
    #         # increment the value of this direction in this batch by the
    #         # percept value * the weight strength
    #         sum_val += percept_stack[b, l, i] * W[l, o, i]
    #       output_activations[b, o] += sum_val
    # each row gives the raw output activations for that batch.
    return output_activations

  def play(self, board, temp=None, W=None):
    """
    Simulate a play on the board.

    Args:
      board (dict): Current state of the game board.

    Returns:
      tuple: Action taken, one-hot representation, and action probabilities.
    """
    temp = temp if temp is not None else self.default_softmax_temp
    W = W if W is not None else self.W
    perceptions = self.game.get_perceptions(board, self.critter_index, self.fov_radius)
    direction_values = self.direction_value_from_percept(perceptions, W)

    batch_size, n_rows, n_cols = board['pieces'].shape
    legal_offsets, valid_moves_mask = self.game.get_legal_offsets(board, critter=self.critter_index, radius=self.speed)
    flat_values = direction_values[valid_moves_mask]
    action_values = np.ones((batch_size, n_rows * n_cols)) * -np.inf
    action_values[(legal_offsets[:,0], legal_offsets[:,1] * n_cols + legal_offsets[:,2])] = flat_values

    # Determine action based on the mode (deterministic or probabilistic)
    if self.deterministic:
      sampled_actions = np.argmax(action_values, axis=1)
      one_hot_actions = np.zeros((batch_size, n_rows * n_cols))
      one_hot_actions[np.arange(batch_size), sampled_actions] = 1.0
      return sampled_actions, one_hot_actions, one_hot_actions
    else:
      action_probs = self.get_action_probabilities(action_values, temp)
      random_samples = self.game.rng.uniform(size=batch_size).reshape((batch_size, 1))
      sampled_actions = np.argmax(action_probs.cumsum(axis=1) > random_samples, axis=1)
      one_hot_actions = np.zeros((batch_size, n_rows * n_cols))
      one_hot_actions[(range(batch_size), sampled_actions)] = 1.0
      return sampled_actions, one_hot_actions, action_probs

  def get_action_probabilities(self, action_values, temp):
    """Compute the softmax probabilities of the actions."""
    shifted_values = action_values - np.max(action_values, axis=1, keepdims=True)
    scaled_values = shifted_values / temp
    exponentiated_values = np.exp(scaled_values)
    return exponentiated_values / np.sum(exponentiated_values, axis=1, keepdims=True)

In [ ]:
################################################################################
# TODO for students: Complete the lines with ... to instantiate two differ..ent
# organisms. A prey type with field of view and speed 2, and a predator type
# with a field of view 3 and a speed of 1. Both predator and prey are able to
# percieve and distinguish between food, edges, prey and predators in their
# perceptive field..
raise NotImplementedError("Exercise: instantiate a predator and a prey player")
################################################################################
gwg = GridworldGame(batch_size=1, num_prey=..., num_pred=...)
prey_player_params = {'critter_index': 1, 'fov_radius': ..., 'speed': ...,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
prey_player = GeneralLinearPlayer(gwg, **prey_player_params)

pred_player_params = {'critter_index': 2, 'fov_radius': ..., 'speed': ...,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
pred_player = GeneralLinearPlayer(gwg, **pred_player_params)

In [ ]:
#to_remove solution
gwg = GridworldGame(batch_size=1, num_prey=1, num_pred=1)
prey_player_params = {'critter_index': 1, 'fov_radius': 2, 'speed': 2,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
prey_player = GeneralLinearPlayer(gwg, **prey_player_params)

pred_player_params = {'critter_index': 2, 'fov_radius': 3, 'speed': 1,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
pred_player = GeneralLinearPlayer(gwg, **pred_player_params)

Now that we are going to be evaluating two distinct organism types, we also need to extend and generalize our evaluation function.

In [ ]:
def evaluate_2_types(prey_player, prey_flat_W, pred_player, pred_flat_W,
                     game, plt_hist=False):
  ''' Assumes player objects were instantiated with the passed in game '''

  prey_player.W = prey_flat_W.reshape(prey_player.W_shape)
  pred_player.W = pred_flat_W.reshape(pred_player.W_shape)

  final_board = game.play_game(players=[prey_player, pred_player], visualize=False)
  # Evaluate the score
  prey_scores = final_board['scores'][:, 0]
  pred_scores = final_board['scores'][:, 1]
  avg_prey_score = np.mean(prey_scores)
  avg_pred_score = np.mean(pred_scores)

  if plt_hist is True:
    fig, ax = plt.subplots()
    weights_prey = np.ones_like(prey_scores) / float(len(prey_scores))
    weights_pred = np.ones_like(pred_scores) / float(len(pred_scores))
    ax.hist(prey_scores, weights=weights_prey, alpha=0.5, label='Prey Scores')
    ax.hist(pred_scores, weights=weights_pred, alpha=0.5, label='Predator Scores')
    ax.legend(loc='upper right')
    ax.set_title("Distribution of Scores")
    ax.set_xlabel("Score")
    ax.set_ylabel("Probability")
    remove_ip_clutter(fig)
    plt.show()
  return(avg_prey_score, avg_pred_score)

gwg = GridworldGame(batch_size=900, num_prey=1, num_pred=1,
                    rng=np.random.default_rng(202))
prey_player_params = {'critter_index': 1, 'fov_radius': 1, 'speed': 1,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
prey_player = GeneralLinearPlayer(gwg, **prey_player_params)
pred_player_params = {'critter_index': 2, 'fov_radius': 1, 'speed': 1,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
pred_player = GeneralLinearPlayer(gwg, **pred_player_params)

avg_prey_score, avg_pred_score = evaluate_2_types(
    prey_player, np.ravel(np.zeros(prey_player.W_shape)),
    pred_player, np.ravel(np.zeros(pred_player.W_shape)),
    gwg, plt_hist=True)
print(f'Average prey score: {avg_prey_score}')
print(f'Average pred score: {avg_pred_score}')

Observing both organisms executing a random drift policy, we notice that the average predator score is slightly positive, and the average prey score is slightly negative. This outcome aligns with expectations. While predator-prey encounters are infrequent, they result in minor gains for predators but substantial losses for prey. These significant losses aren't totally balanced by the prey's more common, but lower scoring, food-eating events.

With our evaluation function in place, we're ready to properly start on the optimization process. For optimization, we'll use an enhanced version of the evolutionary algorithm from sequence 1.3.1, known as the Covariance Matrix Adapted Evolutionary Strategy (CMA-ES). While CMA-ES primarily mirrors our basic evolutionary algorithm its distinction lies in its 'mutation' distribution. Guided by an evolving covariance matrix, this distribution changes over time, leading to mutants with an increased likelihood of fitness improvement. This changing 'mutation' distribution can be seen as a kind of meta-evolution, where mutability itself evolves.

In [ ]:
# @markdown You don't need to know how this code works right now, but you do need to run this cell, and you might want to skim this code out of curiosity.
def cma_es(n, eval_func, focal_pop=0, init_m=None, init_C=None, init_sigma=None,
           pop_size=None, select_frac=0.5, weight_scheme='skew', # other magic word is flat
           c_1=None, c_mu=None, c_c = None, c_sig = None, d_sig = None,
           max_gen=100, sigma_tol=0.0001, value_tol=0.0001,
           rng = np.random.default_rng(),
           verbose = True):
  # assumes eval_func returns scores for multiple populations
  # focal_pop indexes which of these scores is the one we are optimizing for
  # use heuristics to set constants if none provided
  # these are the same as in a simple ES algo
  if pop_size is None:
    pop_size = 4 + np.floor(3 * np.log(n))
    if verbose: print(f"pop_size set to: {pop_size}")
  select_size = int(np.floor(pop_size * select_frac))

  if weight_scheme == 'skew':
    mu_w = np.array([np.log(select_size + 0.5) - np.log(i)
                      for i in range(1, select_size + 1)])
    mu_w = mu_w / np.sum(mu_w)
    mu_eff = 1 / np.sum(mu_w**2)
    if verbose: print(f"mu_w set using skew scheme: {mu_w}")
    if verbose: print(f"mu_eff set to: {mu_eff}")
  elif weight_scheme == 'flat':
    mu_w = np.ones(select_size) / float(select_size)
    mu_eff = select_size / 2
    if verbose: print(f"mu_w set using flat scheme: {mu_w}")
    if verbose: print(f"mu_eff set to: {mu_eff}")

  # this is fancy stuff for updating the mutation distribution
  if c_1 is None:
    c_1 = 2 / n**2
    if verbose: print(f"c_1 set to: {c_1}")
  if c_mu is None:
    c_mu = min(1 - c_1, 2 * (mu_eff - 2 + 1/mu_eff) / (n + 2))
    if verbose: print(f"c_mu set to: {c_mu}")
  if c_c is None:
    c_c = (4 + mu_eff/n) / (n + 4 + 2 * mu_eff/n)
    if verbose: print(f"c_c set to: {c_c}")
  if c_sig is None:
    c_sig = (mu_eff + 2) / (n + mu_eff + 5)
    if verbose: print(f"c_sig set to: {c_sig}")
  if d_sig is None:
    d_sig = 1 + c_sig + 2 * max(0, np.sqrt((mu_eff - 1) / (n + 1)) - 1)
    if verbose: print(f"d_sig set to: {d_sig}")

  # Initialize variables of the process to these defaults if none given
  # (weighted) population mean parameters
  if init_m is None:
    m = np.zeros(n)
  else:
    m = init_m
  if init_C is None:
    # inially the Covariance of the proposal/offspring distribution is just
    # a standard i.i.d. Gaussian
    C = np.eye(n)
  else:
    C = init_C
  if init_sigma is None:
    sigma = 1.0
  else:
    sigma = init_sigma
  # p_c is used as a kind of momentum to inform how C is updated
  p_c = np.zeros(n)


  # Main loop
  start_time = time.time()
  mean_scores_history = []
  for gen in range(max_gen):
    # Steps 1-3 are standard Evolutionary Optimization
    # 1. Sampe params of this generation
    samples = rng.multivariate_normal(m, sigma**2 * C, size=int(pop_size))
    # 2. Evaluate each member of the new population
    fitness_values = np.vstack([eval_func(x) for x in samples])
    focal_fitness_values = fitness_values[:, focal_pop]
    sorted_indices = np.argsort(-focal_fitness_values) #argsort does low to high, so sort neg
    mean_scores = np.mean(fitness_values, axis=0)
    mean_scores_history.append(mean_scores)
    # If verbose flag is true, print generation info and mean score
    elapsed_time = time.time() - start_time
    if verbose:
      all_mean_scores = ", ".join([f"{score:.4f}" for score in mean_scores])
      print(f"Generation {gen + 1}/{max_gen}. Mean Scores: [{all_mean_scores}]. Elapsed Time: {elapsed_time:.2f} seconds")
    # 3. Update mean based on selection
    old_m = m.copy()
    selected_samples = samples[sorted_indices[:select_size]]
    m = (mu_w[:, np.newaxis].T @ selected_samples).squeeze()
    # Steps 4-7 are all about cleverly updating the sampling distribution
    # 4. Compute weighted sample deviations from the old mean
    y = np.sqrt(mu_w[:, np.newaxis]) * (selected_samples - old_m) / sigma
    # 5. Update the covariance matrix
    C = (1 - c_1 - c_mu) * C + c_1 * np.outer(p_c, p_c)
    C += c_mu * sum(np.outer(yi, yi) for yi in y)
    # 6. Update the evolution path
    p_c = (1 - c_c) * p_c + np.sqrt(c_c * (2 - c_c) * mu_eff) * (m - old_m) / sigma
    # 7. Adapt the step size sigma
    sigma *= np.exp(c_sig/d_sig * (np.linalg.norm(p_c) / np.sqrt(n) - 1))
    # 8. Check stopping criteria
    if sigma < sigma_tol:
        print("Stopping due to sigma below threshold.")
        break
    if np.abs(np.max(fitness_values) - np.min(fitness_values)) < value_tol:
        print("Stopping due to objective function values convergence.")
        break
  return m, C, mean_scores

## Optimized Prey Policy when the Predator Drifts Randomly

In [ ]:
# This optimization process takes about 1 minute to run.
# You can skip this and a canned result will be used in the following code cells.
# If you want to get a better result, or love watching progress bars, you can
# let this optimization process run longer by setting `max_gen` to a higher value.

game = GridworldGame(batch_size=16, n_rows=7, n_cols=7,
                     num_food=10, num_prey=1, num_pred=1,
                     lifetime=30, rng=np.random.default_rng(40))
# Initialize players
prey_player_params = {'critter_index': 1, 'fov_radius': 2, 'speed': 2,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
pred_player_params = {'critter_index': 2, 'fov_radius': 3, 'speed': 1,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
prey_player = GeneralLinearPlayer(game, **prey_player_params)
pred_player = GeneralLinearPlayer(game, **pred_player_params)
n_prey = np.prod(prey_player.W_shape)
n_pred = np.prod(pred_player.W_shape)

def eval_wrapper_for_prey(params):
  pred_flat_W = pred_player.W.flatten()  # Using weights for predator
  scores = evaluate_2_types(
  prey_player, params, pred_player, pred_flat_W, game)
  return scores

opt_prey_weights, prey_cov, score_evo = cma_es(n_prey,
                                               eval_wrapper_for_prey,
                                               focal_pop=0,
                                               max_gen=50)

In [ ]:
# @title Watch Random Predator and Optimized Prey
# @markdown Run this cell to watch a random predator organism play against an optimized prey organism.

main_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/main/sequences/P1C2_OptimizationAndEnvironment/data/opt_prey_for_rand_pred.npy"
branch_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/P1C2S3_Competition/sequences/P1C2_OptimizationAndEnvironment/data/opt_prey_for_rand_pred.npy"
response = requests.get(main_url, allow_redirects=True)
if response.status_code == 200:
  with BytesIO(response.content) as f:
    opt_prey_weights = np.load(f, allow_pickle=False)
else:
  print(f"Failed to download with status code: {response.status_code}")
  print('Trying next url')
  response = requests.get(branch_url, allow_redirects=True)
  if response.status_code == 200:
    with BytesIO(response.content) as f:
      opt_prey_weights = np.load(f, allow_pickle=False)
  else:
    print(f"Failed to download with status code: {response.status_code}")

gwg = GridworldGame(batch_size=1, n_rows=7, n_cols=7,
                    num_pred=1, num_prey=1, num_food=10, lifetime=30,
                    rng=np.random.default_rng(11))


opt_prey_player = GeneralLinearPlayer(
    gwg, critter_index=1, fov_radius=2, speed=2, has_food_percept=True,
    has_edge_percept=True, has_prey_percept=True, has_pred_percept=True)
opt_prey_player.W = opt_prey_weights.reshape(opt_prey_player.W_shape)
rand_pred_player = GeneralLinearPlayer(
    gwg, critter_index=2, fov_radius=3, speed=1, has_food_percept=True,
    has_edge_percept=True, has_prey_percept=True, has_pred_percept=True)

opt_prey_igwg = ComplexMoveInteractiveGridworld(
    gwg, players=[opt_prey_player, rand_pred_player], critter_names=['Prey', 'Pred'],
    figsize=(5,4), has_fov=True, radius=0, fov_opaque=False)
display(opt_prey_igwg.b_fig.canvas)
display(opt_prey_igwg.b_fig_legend.canvas)
clear_output()
display(opt_prey_igwg.final_display)

The prey organism has 'evolved' to avoid being eaten and does a pretty good job of eating lots of food despite its limited perceptual field, and efforts to avoid the predator. Now lets look at the reverse situation where the prey organism executes a random drift policy and the predator organism is optimized.

## Optimized Predator Policy when the Prey Drifts Randomly

In [ ]:
# This optimization process also takes about 1 minute to run.
# You can skip this and a canned result will be used in the following code cells.
# If you want to get a better result, or love watching progress bars, you can
# let this optimization process run longer by setting `max_gen` to a higher value.

game = GridworldGame(batch_size=16, n_rows=7, n_cols=7,
                     num_food=10, num_prey=1, num_pred=1,
                     lifetime=30, rng=np.random.default_rng(40))
# Initialize players
prey_player_params = {'critter_index': 1, 'fov_radius': 2, 'speed': 2,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
pred_player_params = {'critter_index': 2, 'fov_radius': 3, 'speed': 1,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
prey_player = GeneralLinearPlayer(game, **prey_player_params)
pred_player = GeneralLinearPlayer(game, **pred_player_params)
n_prey = np.prod(prey_player.W_shape)
n_pred = np.prod(pred_player.W_shape)

def eval_wrapper_for_pred(params):
  prey_flat_W = prey_player.W.flatten()  # Using default weights for predator
  scores = evaluate_2_types(
  prey_player, prey_flat_W, pred_player, params, game)
  return scores

opt_pred_weights, pred_cov, score_evo = cma_es(n_pred,
                                               eval_wrapper_for_pred,
                                               focal_pop=1,
                                               max_gen=50)

In [ ]:
# @title Watch Random Prey and Optimized Predator
# @markdown Run this cell to watch a random prey organism play against an optimized predator organism.

main_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/main/sequences/P1C2_OptimizationAndEnvironment/data/opt_pred_for_rand_prey.npy"
branch_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/P1C2S3_Competition/sequences/P1C2_OptimizationAndEnvironment/data/opt_pred_for_rand_prey.npy"
response = requests.get(main_url, allow_redirects=True)
if response.status_code == 200:
  with BytesIO(response.content) as f:
    opt_pred_weights = np.load(f, allow_pickle=False)
else:
  print(f"Failed to download with status code: {response.status_code}")
  print('Trying next url')
  response = requests.get(branch_url, allow_redirects=True)
  if response.status_code == 200:
    with BytesIO(response.content) as f:
      opt_pred_weights = np.load(f, allow_pickle=False)
  else:
    print(f"Failed to download with status code: {response.status_code}")

gwg = GridworldGame(batch_size=1, n_rows=7, n_cols=7,
                    num_pred=1, num_prey=1, num_food=10, lifetime=30,
                    rng=np.random.default_rng(11))


rand_prey_player = GeneralLinearPlayer(
    gwg, critter_index=1, fov_radius=2, speed=2, has_food_percept=True,
    has_edge_percept=True, has_prey_percept=True, has_pred_percept=True)
opt_pred_player = GeneralLinearPlayer(
    gwg, critter_index=2, fov_radius=3, speed=1, has_food_percept=True,
    has_edge_percept=True, has_prey_percept=True, has_pred_percept=True)
opt_pred_player.W = opt_pred_weights.reshape(opt_pred_player.W_shape)

opt_pred_igwg = ComplexMoveInteractiveGridworld(
    gwg, players=[rand_prey_player, opt_pred_player], critter_names=['Prey', 'Pred'],
    figsize=(5,4), has_fov=True, radius=0, fov_opaque=False)
display(opt_pred_igwg.b_fig.canvas)
display(opt_pred_igwg.b_fig_legend.canvas)
clear_output()
display(opt_pred_igwg.final_display)

In this scenario as well the predator organism gets better at tracking down and eating the unpredictable prey, even though the prey is moves unpredictably and more quickly than the predator. Finally we are going to look at a simultaneous evolution scenario where both the prey and predator populations constantly co-adapt to each-other.

In [ ]:
# @markdown Submit your feedback
content_review(f"{feedback_prefix}_M2")

# 1.2.3.3: Prey and Predator Co-Evolve

We need to adapt our CMA-ES algo to update both populations, predator and prey, simultaneously.

In [ ]:
def two_pop_cma_es(n1, n2, eval_func, init_m1=None, init_m2=None,
                   init_C1=None, init_C2=None,
                   init_sigma1=None, init_sigma2=None,
                   pop_size=None, select_frac=0.5, weight_scheme='skew', # other magic word is flat
                   c_1=None, c_mu=None, c_c = None, c_sig = None, d_sig = None,
                   max_gen=100, sigma_tol=0.0001, value_tol=-0.0001,
                   rng = np.random.default_rng(), verbose = True):

  # use heuristics to set constants if none provided
  if pop_size is None:
    pop_size = 8 + np.floor(3 * np.log(n1)) + np.floor(3 * np.log(n2))
    if verbose: print(f"pop_size set to: {pop_size}")
  select_size = int(np.floor(pop_size * select_frac))

  if weight_scheme == 'skew':
    mu_w = np.array([np.log(select_size + 0.5) - np.log(i)
                      for i in range(1, select_size + 1)])
    mu_w = mu_w / np.sum(mu_w)
    mu_eff = 1 / np.sum(mu_w**2)
    if verbose: print(f"mu_w set using skew scheme: {mu_w}")
    if verbose: print(f"mu_eff set to: {mu_eff}")
  elif weight_scheme == 'flat':
    mu_w = np.ones(select_size) / float(select_size)
    mu_eff = select_size / 2
    if verbose: print(f"mu_w set using flat scheme: {mu_w}")
    if verbose: print(f"mu_eff set to: {mu_eff}")

  n_max = max(n1, n2)
  if c_1 is None:
    c_1 = 2 / n_max**2
    if verbose: print(f"c_1 set to: {c_1}")
  if c_mu is None:
    c_mu = min(1 - c_1, 2 * (mu_eff - 2 + 1/mu_eff) / (n_max + 2))
    if verbose: print(f"c_mu set to: {c_mu}")
  if c_c is None:
    c_c = (4 + mu_eff/n_max) / (n_max + 4 + 2 * mu_eff/n_max)
    if verbose: print(f"c_c set to: {c_c}")
  if c_sig is None:
    c_sig = (mu_eff + 2) / (n_max + mu_eff + 5)
    if verbose: print(f"c_sig set to: {c_sig}")
  if d_sig is None:
    d_sig = 1 + c_sig + 2 * max(0, np.sqrt((mu_eff - 1) / (n_max + 1)) - 1)
    if verbose: print(f"d_sig set to: {d_sig}")

  # Initialize variables of the process to these defaults if none given
  if init_m1 is None:
    m1 = np.zeros(n1)
  else:
    m1 = init_m1
  if init_m2 is None:
    m2 = np.zeros(n2)
  else:
    m2 = init_m2
  if init_C1 is None:
    C1 = np.eye(n1)
  else:
    C1 = init_C1
  if init_C2 is None:
    C2 = np.eye(n2)
  else:
    C2 = init_C2
  if init_sigma1 is None:
    sigma1 = 1.0
  else:
    sigma1 = init_sigma1
  if init_sigma2 is None:
    sigma2 = 1.0
  else:
    sigma2 = init_sigma2
  p_c1 = np.zeros(n1)
  p_c2 = np.zeros(n2)


  # Main loop
  start_time = time.time()
  mean_scores_hist = []
  for gen in range(max_gen):
    # 1. Sampe params of this generation
    samples1 = rng.multivariate_normal(m1, sigma1**2 * C1, size=int(pop_size))
    samples2 = rng.multivariate_normal(m2, sigma2**2 * C2, size=int(pop_size))
    # 2. Evaluate each member of the new population
    fitness_values  = np.vstack([eval_func(x1, x2)
                                 for x1, x2 in zip(samples1, samples2)])
    sorted_indices1 = np.argsort(-fitness_values[:,0]) #argsort does low to high, so sort neg
    sorted_indices2 = np.argsort(-fitness_values[:,1]) #argsort does low to high, so sort neg
    mean_scores = np.mean(fitness_values, axis=0)
    mean_scores_hist.append(mean_scores)
    # If verbose flag is true, print generation info and mean score
    if verbose:
      elapsed_time = time.time() - start_time
      print(f"Generation {gen + 1}/{max_gen} | Mean Population Scores: [{mean_scores[0]:.4f}, {mean_scores[1]:.4f}] | Time Elapsed: {elapsed_time:.2f} seconds")
    # 3. Update mean as a weighted sum of the selected samples
    old_m1 = m1.copy()
    old_m2 = m2.copy()
    selected_samples1 = samples1[sorted_indices1[:select_size]]
    selected_samples2 = samples2[sorted_indices2[:select_size]]
    m1 = (mu_w[:, np.newaxis].T @ selected_samples1).squeeze()
    m2 = (mu_w[:, np.newaxis].T @ selected_samples2).squeeze()
    # 4. Compute weighted sample deviations from the old mean
    y1 = np.sqrt(mu_w[:, np.newaxis]) * (selected_samples1 - old_m1) / sigma1
    y2 = np.sqrt(mu_w[:, np.newaxis]) * (selected_samples2 - old_m2) / sigma2
    # 5. Update the covariance matrix
    C1 = (1 - c_1 - c_mu) * C1 + c_1 * np.outer(p_c1, p_c1)
    C1 += c_mu * sum(np.outer(yi, yi) for yi in y1)
    C2 = (1 - c_1 - c_mu) * C2 + c_1 * np.outer(p_c2, p_c2)
    C2 += c_mu * sum(np.outer(yi, yi) for yi in y2)
    # 6. Update the evolution path
    p_c1 = (1 - c_c) * p_c1 + np.sqrt(c_c * (2 - c_c) * mu_eff) * (m1 - old_m1) / sigma1
    p_c2 = (1 - c_c) * p_c2 + np.sqrt(c_c * (2 - c_c) * mu_eff) * (m2 - old_m2) / sigma2
    # 7. Adapt the step size sigma
    sigma1 *= np.exp(c_sig/d_sig * (np.linalg.norm(p_c1) / np.sqrt(n1) - 1))
    sigma2 *= np.exp(c_sig/d_sig * (np.linalg.norm(p_c2) / np.sqrt(n2) - 1))
    # 8. Check stopping criteria
    if sigma1 < sigma_tol and sigma2 < sigma_tol:
        print("Stopping due to sigma below threshold.")
        break
    if np.abs(np.max(fitness_values) - np.min(fitness_values)) < value_tol:
        print("Stopping due to objective function values convergence.")
        break
  return m1, C1, m2, C2, mean_scores_hist

In [ ]:
# This optimization process takes about 3 minutes to run.
# You can skip this and a canned result will be used in the following code cells.
# If you want to get a better result, or love watching progress bars, you can
# let this optimization process run longer by setting `max_gen` to a higher value.

game = GridworldGame(batch_size=16, n_rows=7, n_cols=7,
                     num_food=10, num_prey=1, num_pred=1,
                     lifetime=30, rng=np.random.default_rng(40))
# Initialize players
prey_player_params = {'critter_index': 1, 'fov_radius': 2, 'speed': 2,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
pred_player_params = {'critter_index': 2, 'fov_radius': 3, 'speed': 1,
                      'has_food_percept': True, 'has_edge_percept': True,
                      'has_prey_percept': True, 'has_pred_percept': True}
prey_player = GeneralLinearPlayer(game, **prey_player_params)
pred_player = GeneralLinearPlayer(game, **pred_player_params)
n_prey = np.prod(prey_player.W_shape)
n_pred = np.prod(pred_player.W_shape)

def dual_eval_wrapper(params1, params2):
  scores = evaluate_2_types(
  prey_player, params1, pred_player, params2, game)
  return scores


(co_opt_prey_weights, co_prey_cov,
 co_opt_pred_weights, co_pred_cov,
 co_score_evo) = two_pop_cma_es(n_prey, n_pred,
                                dual_eval_wrapper,
                                max_gen=25)

In [ ]:
# @title Watch Co-Adapted Prey and Predator
# @markdown Run this cell to watch a co-adapted prey and predator organism play each other.

# get prey weights
main_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/main/sequences/P1C2_OptimizationAndEnvironment/data/co_opt_prey_W.npy"
branch_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/P1C2S3_Competition/sequences/P1C2_OptimizationAndEnvironment/data/co_opt_prey_W.npy"
response = requests.get(main_url, allow_redirects=True)
if response.status_code == 200:
  with BytesIO(response.content) as f:
    co_opt_prey_weights = np.load(f, allow_pickle=False)
else:
  print(f"Failed to download with status code: {response.status_code}")
  print('Trying next url')
  response = requests.get(branch_url, allow_redirects=True)
  if response.status_code == 200:
    with BytesIO(response.content) as f:
      co_opt_prey_weights = np.load(f, allow_pickle=False)
  else:
    print(f"Failed to download with status code: {response.status_code}")

# get pred weights
main_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/main/sequences/P1C2_OptimizationAndEnvironment/data/co_opt_pred_W.npy"
branch_url = "https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/raw/P1C2S3_Competition/sequences/P1C2_OptimizationAndEnvironment/data/co_opt_pred_W.npy"
response = requests.get(main_url, allow_redirects=True)
if response.status_code == 200:
  with BytesIO(response.content) as f:
    co_opt_pred_weights = np.load(f, allow_pickle=False)
else:
  print(f"Failed to download with status code: {response.status_code}")
  print('Trying next url')
  response = requests.get(branch_url, allow_redirects=True)
  if response.status_code == 200:
    with BytesIO(response.content) as f:
      co_opt_pred_weights = np.load(f, allow_pickle=False)
  else:
    print(f"Failed to download with status code: {response.status_code}")

gwg = GridworldGame(batch_size=1, n_rows=7, n_cols=7,
                    num_pred=1, num_prey=1, num_food=10, lifetime=30,
                    rng=np.random.default_rng(11))

co_opt_prey_player = GeneralLinearPlayer(
    gwg, critter_index=1, fov_radius=2, speed=2, has_food_percept=True,
    has_edge_percept=True, has_prey_percept=True, has_pred_percept=True)
co_opt_prey_player.W = co_opt_prey_weights.reshape(co_opt_prey_player.W_shape)

co_opt_pred_player = GeneralLinearPlayer(
    gwg, critter_index=2, fov_radius=3, speed=1, has_food_percept=True,
    has_edge_percept=True, has_prey_percept=True, has_pred_percept=True)
co_opt_pred_player.W = co_opt_pred_weights.reshape(co_opt_pred_player.W_shape)

co_opt_igwg = ComplexMoveInteractiveGridworld(
    gwg, players=[co_opt_prey_player, co_opt_pred_player], critter_names=['Prey', 'Pred'],
    figsize=(5,4), has_fov=True, radius=0, fov_opaque=False)
display(co_opt_igwg.b_fig.canvas)
display(co_opt_igwg.b_fig_legend.canvas)
clear_output()
display(co_opt_igwg.final_display)

The predator organism, with its slower movement, can not directly chase down the faster prey. To successfully hunt, the predator must stay out of the prey's field of view, hoping that the prey will move within striking distance. Meanwhile, the prey organism faces its own challenges, constantly deciding between two conflicting approaches:

1. Slow, cautious movement that reduces food intake but minimizes the risk of being preyed upon.
2. Quick, recklessly optimistic movement that boosts food consumption but increases vulnerability to predators.

Note that relative predation risks depend critically on the predator's policy. The resulting equilibrium from such an evolutionary tug-of-war is a set of policies where no single policy can unilaterally improve its outcome. In game theory, this stable state of the system—based on game payoffs—is termed a Nash Equilibrium. (In Game Theory the term "strategies" is typically used instead of policy, but we will continue to use "policy" in line with standard reinforcement learning terminology.)

A few clarifications are necessary about what this Nash Equilibrium is not:

1. A set of policies maximizing the prey's score—prey would fare much better against less skilled predators.
2. The best set of policies for the predator; they'd have higher success rates against less evasive prey.
3. The set of policies optimizing the average payoff across both populations.

It's due to such nuances that we say evolution doesn't "optimize" in a traditional sense. Yet, while technically correct, a Nash Equilibrium is a set of policies that the "best response" to the each other. In this sense, evolutionary outcomes can be thought of as "optimal."

In Sequence 1.2.3, we explored the idea that optima depict an ideal state. The concept of equilibria, as illustrated here, extends and refines this notion. While our world remains in perpetual change, the underlying evolutionary dynamics constantly steer systems towards these "mutual best response" equilibrium points.

This concludes our exploration of how competition and interaction introduce complexities into optimization. While evolutionary processes are more than just optimization processes, as we progress in this book, we'll primarily sidestep these game-theoretic considerations. Our primary take-away is that because living organisms are embedded in such evolutionary systems, which inherently drive change, there are selective pressures not just for adaptation to changing environments but ***rapid*** adaptation. In the next sequence we will see how within lifetime learning can play an integral role in adapting to rapidly changing environments.

In [ ]:
# @markdown Submit your feedback
content_review(f"{feedback_prefix}_M3")

# Quiz

In [ ]:
# @markdown **Run this cell** to take the quiz
comprehension_quiz = [
  {
    "question": "What overarching principles was illustrated in these evolutionary Gridworld simulation?",
    "type": "multiple_choice",
    "answers": [
      {
        "answer": "Static behaviours result in predictable outcomes.",
        "correct": False,
        "feedback": "While this is sometimes true, these simulations were primarily about adaptive and evolving behaviours."
      },
      {
        "answer": "Organisms constantly adapt their behaviours in response to their environment and other organisms.",
        "correct": True,
        "feedback": "Exactly! The predator-prey dynamic in the Gridworld simulation demonstrates the continuous adaptation of behaviors in response to both environmental and inter-organismal factors."
      },
      {
        "answer": "Organisms with faster movement are always dominant in their environment.",
        "correct": False,
        "feedback": "Speed might be advantageous, but adaptation and strategy can often overcome sheer speed."
      },
      {
        "answer": "Evolutionary outcomes are entirely random and lack predictability.",
        "correct": False,
        "feedback": "While there might be elements of randomness, evolutionary outcomes are largely driven by environmental pressures and are not entirely random."
      }
    ]
  },
  {
    "question": "What is a primary goal of using evolutionary strategies like CMA-ES in optimization problems?",
    "type": "multiple_choice",
    "answers": [
      {
        "answer": "To maintain a static solution throughout iterations.",
        "correct": False,
        "feedback": "Evolutionary strategies aim to adapt and refine solutions, not keep them static."
      },
      {
        "answer": "To iteratively refine solutions, increasing the likelihood of better outcomes over time.",
        "correct": True,
        "feedback": "Correct! Evolutionary strategies like CMA-ES work iteratively, refining solutions to yield better outcomes as they progress."
      },
      {
        "answer": "To always ensure a perfect solution in the first iteration.",
        "correct": False,
        "feedback": "Evolutionary strategies work over multiple iterations and don't guarantee perfection in the first attempt."
      },
      {
        "answer": "To decrease the randomness in evolutionary algorithms.",
        "correct": False,
        "feedback": "While they refine solutions, evolutionary strategies still incorporate randomness or stochastic processes to discover optimal solutions."
      }
    ]
  },
  {
    "question": "When both prey and predator populations are allowed to evolve simultaneously, this process of mutual adaptation can best be described as:",
    "type": "multiple_choice",
    "answers": [
      {
        "answer": "Mono-evolution, where only one species evolves while the other remains static.",
        "correct": False,
        "feedback": "Mono-evolution implies evolution in isolation, which is not the case here."
      },
      {
        "answer": "Static evolution, where both species remain unchanged despite environmental pressures.",
        "correct": False,
        "feedback": "Static evolution doesn't capture the essence of mutual adaptation observed in the discussed scenario."
      },
      {
        "answer": "Co-evolution, where both species undergo evolutionary adjustments in response to each other.",
        "correct": True,
        "feedback": "Exactly! Co-evolution refers to the process where two or more species reciprocally affect each other's evolution."
      },
      {
        "answer": "Random evolution, where changes in species are entirely unpredictable and lack direction.",
        "correct": False,
        "feedback": "The process described is systematic and driven by environmental and interspecies dynamics, not entirely random."
      }
    ]
  }
]



display_quiz(comprehension_quiz)